In [2]:
from datasets import load_dataset, load_metric
from nltk.tokenize import word_tokenize


In [3]:
from transformers import (
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments,
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    T5TokenizerFast,
)

In [4]:
tokenizer = AutoTokenizer.from_pretrained("t5-base")
input_ids=tokenizer(["hello there general kenobi", "hello world"])["input_ids"]
print(input_ids)
print(tokenizer.batch_decode(input_ids))

[[21820, 132, 879, 3, 2217, 6690, 1], [21820, 296, 1]]
['hello there general kenobi</s>', 'hello world</s>']


/root/miniconda3/envs/sum/lib/python3.8/site-packages/transformers/models/t5/tokenization_t5_fast.py:156: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [5]:
T5TokenizerFast.max_model_input_sizes

{'t5-small': 512, 't5-base': 512, 't5-large': 512, 't5-3b': 512, 't5-11b': 512}

In [6]:
t5 = AutoModelForSeq2SeqLM.from_pretrained("t5-base", use_cache=False)

# Bleu

In [7]:
predictions = ["hello there general kenobi", "foo bar foobar"]
references = [
    ["hello there general kenobi", "hello there !", "Hi general kenobi"],
    ["foo bar foobar", "foobar"]
]
pred=[tokenizer.tokenize(pred) for pred in predictions]
print(predictions, pred)
refer=[[tokenizer.tokenize(refer) for refer in references_per_label] for references_per_label in references]

print(references, refer)

['hello there general kenobi', 'foo bar foobar'] [['▁hello', '▁there', '▁general', '▁', 'ken', 'obi'], ['▁fo', 'o', '▁bar', '▁fo', 'o', 'bar']]
[['hello there general kenobi', 'hello there !', 'Hi general kenobi'], ['foo bar foobar', 'foobar']] [[['▁hello', '▁there', '▁general', '▁', 'ken', 'obi'], ['▁hello', '▁there', '▁', '!'], ['▁Hi', '▁general', '▁', 'ken', 'obi']], [['▁fo', 'o', '▁bar', '▁fo', 'o', 'bar'], ['▁fo', 'o', 'bar']]]


In [8]:
bleu = load_metric("bleu")
results = bleu.compute(predictions=pred, references=refer)
print(results)
print(results['bleu'])
print(results['precisions'][2])

{'bleu': 1.0, 'precisions': [1.0, 1.0, 1.0, 1.0], 'brevity_penalty': 1.0, 'length_ratio': 1.7142857142857142, 'translation_length': 12, 'reference_length': 7}
1.0
1.0


In [9]:
sentence1 = ["the cat sat on the mat"]
sentence2 = ["the cat ate the mat"]
pred=[tokenizer.tokenize(sen) for sen in sentence1]
refer=[[tokenizer.tokenize(sen) for sen in sentence2]]

print("pred", pred)
print("refer", refer)
result = bleu.compute(predictions=pred, references=refer)
print(result)

pred [['▁the', '▁cat', '▁', 's', 'at', '▁on', '▁the', '▁mat']]
refer [[['▁the', '▁cat', '▁', 'ate', '▁the', '▁mat']]]
{'bleu': 0.0, 'precisions': [0.625, 0.42857142857142855, 0.16666666666666666, 0.0], 'brevity_penalty': 1.0, 'length_ratio': 1.3333333333333333, 'translation_length': 8, 'reference_length': 6}


In [37]:
pred_str = "It is a guide to action which ensures that the military always obeys the commands of the party"
ref_str = "It is a guide to action which ensures that the military always obeys the commands of the party"
pred_li=[pred_str, pred_str]
ref_li=[ref_str, ref_str]
# pred_token=tokenizer.tokenize(prediction[0])
# refer_token=tokenizer.tokenize(reference[0])

from nltk.translate import meteor_score
from datasets.config import importlib_metadata, version
NLTK_VERSION = version.parse(importlib_metadata.version("nltk"))
print(NLTK_VERSION)

pred_token=tokenizer.tokenize(pred_str)
ref_token=tokenizer.tokenize(ref_str)
print("pred_token", pred_token)
meteor_results=meteor_score.single_meteor_score(
                    ref_token, pred_token, alpha=0.9, beta=3, gamma=0.5
                )
print("meteor_results", meteor_results)


meteor_results = [meteor_score.single_meteor_score(
    tokenizer.tokenize(ref_str), tokenizer.tokenize(pred_str), alpha=0.9, beta=3, gamma=0.5
)
          for ref_str, pred_str in zip(ref_li, pred_li)
         ]
print("meteor_results", meteor_results)

# meteor = load_metric('meteor')
# meteor_results = meteor.compute(predictions=[pred_str],
#                         references=[ref_str])
# print("meteor_results", meteor_results)

3.7
pred_token ['▁It', '▁is', '▁', 'a', '▁guide', '▁to', '▁action', '▁which', '▁ensure', 's', '▁that', '▁the', '▁military', '▁always', '▁obey', 's', '▁the', '▁commands', '▁of', '▁the', '▁party']
meteor_results 0.9999460101500918
meteor_results [0.9999460101500918, 0.9999460101500918]


# Rouge

In [10]:
predictions = ["hello there", "general kenobi"]
references = ["hello there", "general kenobi"]
pred=[tokenizer.tokenize(pred) for pred in predictions]
print(predictions, pred)
refer=[tokenizer.tokenize(refer)  for refer in references]
print(references, refer)

['hello there', 'general kenobi'] [['▁hello', '▁there'], ['▁general', '▁', 'ken', 'obi']]
['hello there', 'general kenobi'] [['▁hello', '▁there'], ['▁general', '▁', 'ken', 'obi']]


In [11]:
rouge = load_metric('rouge')
results = rouge.compute(predictions=predictions,
                        references=references)
print(results)
print(results['rouge1'].mid.fmeasure)

{'rouge1': AggregateScore(low=Score(precision=1.0, recall=1.0, fmeasure=1.0), mid=Score(precision=1.0, recall=1.0, fmeasure=1.0), high=Score(precision=1.0, recall=1.0, fmeasure=1.0)), 'rouge2': AggregateScore(low=Score(precision=1.0, recall=1.0, fmeasure=1.0), mid=Score(precision=1.0, recall=1.0, fmeasure=1.0), high=Score(precision=1.0, recall=1.0, fmeasure=1.0)), 'rougeL': AggregateScore(low=Score(precision=1.0, recall=1.0, fmeasure=1.0), mid=Score(precision=1.0, recall=1.0, fmeasure=1.0), high=Score(precision=1.0, recall=1.0, fmeasure=1.0)), 'rougeLsum': AggregateScore(low=Score(precision=1.0, recall=1.0, fmeasure=1.0), mid=Score(precision=1.0, recall=1.0, fmeasure=1.0), high=Score(precision=1.0, recall=1.0, fmeasure=1.0))}
1.0


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


ArrowTypeError: Expected bytes, got a 'list' object

In [ ]:
rouge = load_metric('rouge')
results = rouge.compute(predictions=pred,
                        references=refer)
print(results)

In [79]:
pred=[['▁"', 'Norm', 'al', '_', 'frame', '"', ':', '▁false', '"', 'A', 'l', 'titude', '▁(', 'm', ')', '"', ':', '▁', '0.5', ',', '▁"', 'f', 'ly', 'ing', '▁over', '"', ':', '▁"', 'water', '"', '"', 'time', '_', 's', 't', 'amp', '"', ':', '▁*', '0', ':', '01', '"', 'name', '"', ':', '▁**', '0', ':', '02', '"', 'd', 'i', 'stance', '"', ':', '▁2.0', '▁"', 'ship', '▁3', '",', '▁"', 'D', 'i', 'stance', '")', ':', '▁*', '2.0', '▁', "'", '"', 'name', "'", ':', "'", 'ship', '▁3', "'", "'", '▁', "'", "'", 'name', "'", "'", ':', '▁"', 'ship', '▁2', "'", "'", '"', 'date', '_', 'time', '"', ':', '▁#', '0', ':', '.', '.', '▁(', '-', '---', '---', '---', '---', '---', '▁--', '▁', '-', '▁', '-', '▁', '---', '---', '-', '---', '"', '-', '"', '-', '-', '"', '▁', '/', '-', '-', '"', '-', '"', '/', '/', '"', '▁', '/', '"', '"', '▁"', '"', '▁', '-', '▁"', '▁"', "'", '"', '(', '/', '"', '-', "'", "'", '"', ':', "'", '"', "'", '"', '"', "'", "'", "'", '▁"', '▁"', "'", '"', '▁"', '"', '."', '"', '"', '▁"', "'", "'", '(', '"', '"', '(', '"', '(', "'", '"', '(', '(', '(', "'", '(', '(', '"', '▁(', '(', '(', '*', '"', '(', '▁"', '(', '()', '"', '"', ':', '"', '")', '"', "'", ':', '"', "'", '(', "'", "'", '-', "'", '"', '▁(', "'", "'", ')', '"', '()', "'", "'", ';', '"', "'", ';', "'", "'", '.', "'", "'", ',', "'", "'", '/', "'", "'", '_', "'", "'", '?', "'", "'", '!', "'", "'", '&', "'", "'", '0', "'", "'", '1', "'", "'", '2', "'", "'", '3', "'", "'", '6', "'", "'", '7', "'", "'", '8', "'", "'", '9', "'", "'", '10', "'", "'", '11', "'", "'", '12', "'", "'", '13', "'", "'", '14', "'", "'", '09', "'", "'", '00', "'", "'", '01', "'", "'", '02', "'", "'", '03', "'", "'", '04', "'", "'", '08', "'", "'", '32', "'", "'", '72', "'", "'", '64', "'", "'", '69', "'", "'", '71', "'", "'", '82', "'", "'", '86', "'", "'", '91', "'", "'", '81', "'", "'", '61', "'", "'", '62', "'", "'", '84', "'", "'", '89', "'", "'", '88', "'", "'", '▁(', "'", '.', '-', "'", '.', '.', "'", ';', '▁"', "'", ';', '▁', "'", ';', ';', "'", '"', ';', '"', ';', '▁"', '"', ';', "'", ')', '▁"', "'", ':', "'", "'", ');', '▁"', '"', "'", '."', "'", ')', "'", '"', '▁', "'", '(', '.', '."', '"', ';', '▁', "'", '(', '▁"', "'", '(', '")', '"', "'", '▁("', "'", '")', '▁"', "'", ')', '(', '"', "'", '▁"', '(', '")', '▁"', '"', '"', '▁("', '(', '")', '▁"', '"', '(', ':', '▁"', '"', '▁"', '(', "'", '")', '"', '"', '")', '"', '▁', '")', '"', '▁"', '▁"', '()', '▁"', '(', '▁"', '"', '▁(', ')', '▁"', '▁"', '")', "'", '"', '"', '▁', '&', '"', '"', '&', '"', '▁"', '&', '"', "'", '&', '"', '&', "'", '"', '&', '▁"', '"', '&', '&', '"', '(', '"', '(', '&', '"', ';', '(', '"', '&', ';', '"', '&', '(', '"', ';', '&', "'", '&', '&', "'", ';', '&', '"', '▁', '0', "'", '"', '1', "'", '"', '2', "'", '"', '3', "'", '"', '4"', '"', '"', '4"', "'", '"', '0', "'", '0', '"', '"', '0', '"', "'", '1', '"', '"', '1', '"', "'", '2', '"', '"', '2', '"', "'", '3', '"', '"', '3', '"', "'", '4"', '4"', '"', '4"', '4"', '4"', "'", "'", '4"', '"', '1', '▁"', '"', '2', '▁"', '0', '"', '▁"', '1', '"', '▁"', '2', '"', '▁"', '3', '"', '▁"', '4"', '"', '▁"', '6', '"', '▁"', '0', "'", '4"', ':', '▁"', '1', "'", '4"', ';', '"', '"', '6', '"', '"', '-', '0', '"', ';', '1', "'", '0', ';', '"', '(', '1', "'", '2', '▁"', '"', '0', ':', '1', "'", '1', '▁"', '0', ':', '0', "'", '1', ':', '0', '"', ':', '▁1', "'", '0', '▁"', '0', ')', '"', '1', '"', '2', '"', '3', '"', '4"', '4"', '"', '2', '"', '123', '"', '34', '"', '"', '2', ':', '22', "'", '0', '"', '02', '&', '"', '2', '&', '2', '"', '0', '&', '2', "'", '2', '&', '3', "'", '0', '&', '"', '1', '&', '"', '3', '&', '"', '4"', '0', '"', '0', ')', '0', '"', '2', '(', '0', '"', '(', '0', "'", '2', '(', '"', '0', ';', '0', '"', '0', '"', '▁', ')', '"', '0', '(', '"', '1', '(', '"', '3', '(', '"', '2', '+', '"', '"', '4"', '(', '2', '"'], ['▁Na', 'N', ',', '▁"', 'D', 'i', 'stance', '"', ':', '▁', '1.0', '▁"', 'time', '_', 's', 't', 'amp', '"', ':', '▁"', '0', ':', '04', '",', '▁"', 'name', '"', ':', '▁Na', 'N', '▁', '-', '▁"', 'd', 'i', 'stance', ':', '▁', '0.5', '"', '▁time', '_', 'level', '"', ':', '▁10', '"', 'Norm', 'al', '_', 'frame', '"', ':', '▁false', '"', 'Win', 'd', 'Speed', '▁(', 'm', '/', 's', ')', '"', ':', '▁16', '"', 'Time', '_', 'level', '▁(', 'm', ')', ':', '▁', '0', ':', '03', '"', ':', '▁1', '"', 'car', 'go', '"', ':', '▁2', '"', 'normal', '_', 'frame', ')', ':', '▁1', ':', '04', '"', ':', '▁3', ':', '04', '"', 'd', '-', '-', '.', '.', '▁"', '"', '"', '▁', "'", '"', '"', '▁"', '"', '▁"', '-', '-', '"', '▁"', '▁"', '▁', '.', '."', '/', '"', '-', '"', "'", '"', '."', ':', '"', '*', '"', '(', '"', "'", '"', '"', '"', ':', '"', '".', '▁"', '.', '/', '"', '"', '▁"', '"', ':', '..."', ':', ':', '"', ':', "'", '"', ':', '-', '"', '"', '-', '-', '"', '-', ':', '"', '-', '▁"', '"', '"', '(', '"', '(', '▁"', '"', '(', '(', '(', '"', '"', '/', '(', '(', "'", '"', '(', "'", "'", '(', '"', '/', '"', '(', '/', '"', '/', '/', '"', "'", "'", "'", '/', "'", "'", '"', "'", '/', '"', '▁"', "'", "'", '.', "'", "'", '▁"', "'", '"', '/', "'", '"', '▁"', '/', "'", '/', '/', "'", '.', '/', "'", ':', "'", "'", ':', '"', "'", ':', '▁"', "'", '.', '▁"', "'", '/', '.', '▁"', '...', "'", '."', '"', "'", '.', '.', ':', '▁"', '".', '.', '!', '................', '................', '................', '................', '................', '................', '................', '................', '................', '................', '................', '................', '................', '---', '---', '---', '---', '---', '---', '---', '"', '---', '---', "'", "'", '---', "'", "'", '...', "'", '-', '"', "'", '-', "'", '"', '-', "'", '-', '▁"', "'", '-', '---', '"', '-', '---', "'", '-', '(', '-', '-', '(', '"', '-', '/', '"', '-', '(', "'", '-', '/', '-', '"', '▁"', '-', '-', '/', "'", '-', ':', '▁"', '-', "'", '.', '-', '-', '▁"', '-', '---', '▁"', '-', '"', '▁', '-', '"', '/', '.', '.', '▁"', '-', '/', '.', '▁--', '▁', '-', '---', '▁', '-', '▁', "'", "'", '▁', '-', "'", '▁', "'", '"', '▁', "'", '.', "'", '"', '▁', "'", '-', '.', '.', '-', "'", '▁"', '-', '▁"', '.', '-', '▁"', '▁"', '-', '.', '-', '"', '▁--', '"', '-', '.', '-', '."', '-', '_', '"', '-', '▁--', "'", '-', '_', '-', '-', '▁', '-', '---', 'F', '-', '-', 'N', '-', '-', 'I', '-', '-', 'D', '-', '-', 'A', '-', '-', 'C', '-', '-', 'S', '-', '-', 'P', '-', '-', 'Re', '-', '-', 'E', '-', '-', 'B', '-', '-', ':', '-', '-', 'Y', '-', '-', 'V', '-', '-', 'H', '-', '-', 'W', '-', '-', 'R', '-', '-', 'K', '-', '-', 'L', '-', '-', 'M', '-', '-', 'T', '-', '-', 'NO', '-', '-', 'NE', '-', '-', 'No', '-', '-', 'O', '-', '-', 'n', '-', '-', '▁(', '-', '-', '")', '-', '-', ')', '-', '"', '(', '-', '"', 'D', '-', '"', 'N', '-', '"', 'F', '-', '"', 'I', '-', "'", '(', '-', "'", 'N', '-', "'", 'F', '-', "'", 'D', '-', "'", ':', '-', "'", ',', '-', "'", "'", '-', 'F', "'", '-', 'N', '"', '-', 'N', "'", '-', 'P', "'", '-', 'C', "'", '-', 'I', "'", '-', 'D', "'", '-', 'A', '"', '-', 'F', '"', '-', 'C', '"', '-', 'D', '"', '-', 'P', '"', '-', 'A', "'", '-', 'S', '"', '-', 'I', '"', '-', 'S', "'", '"', 'I', "'", '"', 'F', '"', '▁"', 'N', '-', 'F', '.', '-', 'I', '.', '-', 'F', '".', '-', '"', 'A', '-', 'F', '."', '-', '"', 'C', '-', '"', 'P', '-', 'F', '(', '-', 'F', '")', '-', '"', 'S', '"', '"', 'N', '"', '"', 'D', '"', '"', 'I', '"', '"', 'A', '"', '"', 'F', '."', 'I', '.', '▁"', '"', 'I', 'I', '"', 'I', '▁"', 'III', '-', 'I', 'I', 'FI', '-', 'I', 'FI', 'FI', '-', 'FI', 'I', '-', 'FF', 'I', 'I', '"', 'FF', 'IF', 'IF', 'III', "'", 'I', 'I', 'X', '"', 'I', 'X', '")', '"', 'I', 'FI', 'X', '"', 'FI', 'I', 'X', 'X', '"', '"', 'IN', '"', 'IN', '")', '"', 'IN', 'IN', '"', 'IT', 'I'], ['▁"', '0', ':', '02', '",', '▁"', 'name', '"', ':', '▁"', 'to', 'y', '▁car', '",', '▁"', 'D', 'i', 'stance', '"', ':', '▁', '0.5', '▁"', 'time', '_', 's', 't', 'amp', '"', ':', '▁3.', '0', '"', '▁time', '_', 's', 'p', 'read', '"', ':', '▁2.0', '"', '"', 'f', 'light', '_', 'time', '"', ':', '▁1.', '01', '"', 'd', 'i', 'stance', '="', '0', ':', '03', '"', ':', '▁0.', '01', '▁"', 'time', '_', 'f', 'lag', '"', ':', '▁2.0', '2', '"', 's', 'p', 'lash', '"', ':', '▁4.', '02', '▁"', 's', 'p', 'ray', '"', ':', '▁5.', '02', '▁', "'", 'time', '_', 'frame', "'", ':', '▁', '0', ':', '04', '"', 'g', '▁', '.', '.', '▁"', '▁»', "'", '▁(', '▁', '-', '▁', '/', '▁', '/', '▁', '-', '."', '.', '▁', '_', '"', '"', '▁"', '"', '▁"', '▁"', '▁', '&', '"', "'", '"', ':', '"', '"', '"', ':', '.', '▁"', '"', '."', '"', "'", '"', '"', 's', '"', '"', '&', '"', "'", "'", '"', "'", "'", '"', '&', "'", '"', '▁"', '"', '"', ';', '"', '"', '▁', "'", '"', '(', '"', '"', '(', "'", '"', ':', "'", '"', ';', "'", '"', '-', '"', '"', '-', "'", "'", "'", ':', '"', "'", ':', '▁"', "'", "'", '(', "'", "'", ';', '"', "'", ';', "'", "'", '-', '"', "'", '▁"', '"', "'", '(', '"', "'", '-', "'", '"', '="', "'", "'", '▁"', "'", '"', '▁', '&', "'", "'", '.', '▁"', "'", '."', '▁"', "'", ':', "'", "'", '▁', "'", "'", ',', "'", "'", '&', '"', '"', '▁"', '&', '"', '▁"', '▁"', '&', "'", '▁"', '▁"', '"', '&', '&', '"', '▁', '0', "'", "'", '0', "'", '"', '0', '"', "'", '0', '"', '"', '1', "'", "'", '1', "'", '"', '2', "'", "'", '2', "'", '"', '3', "'", "'", '3', "'", '"', '1', '"', "'", '1', '"', '"', '2', '"', "'", '2', '"', '"', '3', '"', '"', '4"', '"', '4"', '4"', '4"', '"', '"', '0', "'", '4"', '4"', "'", '"', '4"', "'", "'", '6', '"', '"', '6', '"', '▁"', '1', '"', '▁"', '2', '"', '▁"', '3', '"', '▁"', '0', '"', '▁"', '4"', '"', '▁"', '7', '"', '▁"', '8', '"', '▁"', '9', '"', '▁"', '10', '"', '▁"', '11', '"', '▁"', '13', '"', '▁"', '14', '"', '▁"', '6', '"', '▁', '4"', '▁"', '4"', '▁"', '"', '4"', ':', '▁"', '"', '1', '"', '1', '▁"', '"', '2', '▁"', '"', '3', '▁"', '"', '4"', '▁"', '4"', '"', '#', '"', '"', '%', '"', '"', '!', '▁"', '"', '0', '▁"', '0', '▁"', '1', '▁"', '▁"', '0', "'", '▁"', '0', '▁"', '1', '▁"', '2', '"', '0', ':', '0', '"', '▁', '-', '"', '0', '▁', '-', '▁"', '0', '▁', '0', "'", '0', '▁', '0', '▁', '122', '"', '4"', '"', '2', '▁2', '▁', '0', '"', '02', "'", '0', '▁', "'", '0', '&', '"', '1', '▁', '0', '&', "'", '14', '"', '"', '0', '&', '&', '#', '"', '0', ';', '"', '/', '"', '&', '"', '&', '#', "'", '"', '#', "'", "'", '#', '"', "'", '&', '&', '&', "'", '&', '##', '"', '&', '/', '"', '"', '$', '"', '&', '▁', "'", '&', "'", '#', '&', '"', '#', '&', '&', '▁"', '&', '▁"', "'", '&', '▁"', '"', '##', "'", '▁"', '&', '#', ';', '"', '▁"', '#', '"', '▁"', '%', '"', '▁"', '(', '"', '&', ';', '"', '▁"', '"', '/', '"', "'", '/', '"', '&', '▁"', '▁"', "'", '#', '▁"', '"', '▁"', ';', '"', '&', '.', '▁"', '".', '▁"', '▁"', '▁"', '#', '&', "'", '-', '▁"', '"', ':', ':', '"', '▁"', '-', '"', '▁"', ':', '"', '&', '-', '"', '▁', ':', '"', '▁', '%', '"', "'", '▁', '&', '▁"', '▁', '&', '&', '▁', '&', '.', '.', "'", '"', '▁("', "'", '.', "'", '"', '▁', "'", '.', "'", "'", '▁(', "'", "'", '/', "'", "'", ')', '▁"', "'", '(', '▁"', "'", ';', '▁"', "'", '▁"', '(', "'", '.', '&', '"', '(', '▁"', '"', '(', '(', '"', '(', '&', "'", '(', '(', "'", '(', '&', '"', '▁(', "'", '"', '/', "'", '"', '$', "'", '"', '#', "'", '&', '/', "'", '#', "'", '#', '(', "'", '#', '##', '&', '##', '##', '##', '##', '##', '##', '##', '&', '#', '&', '▁"', '##', '0', '&', '#', '0', "'", '&', ';', '##', '$', '&', '#', '$', '##', '%', '&', '#', ':', '&', '#', '!', '&', '#', '.', '&', '#', '▁"', '&', '&', '%', '&', '&', '$', '&', '&', ';', '&', '&', '!', '&', '&', '.', '&', '&', '/', '&', '&', '-', '&', '&', '!"', '&', '"', ';', '&', "'", ';', '&', '"', '-', '&', '"', '+', '"', '&', '!', "'", "'", '!', '&', "'", '!', "'", '"', '!', '&', '"', ':', '&', '"', '!"', "'", '!"', '"', '."', '&', '!"', "'", '"', '%', "'", "'", '%', '"', '&', '%', "'", '".', '&', "'", '.'], ['▁"', 'Win', 'd', 'Speed', '▁(', 'm', '/', 's', ')', '"', ':', '▁16', '"', 'time', '_', 's', 't', 'amp', '"', ':', '▁"', '0', ':', '01', '",', '▁"', 'name', '"', ':', '▁', "'", 'water', "'", ',', '▁"', 'D', 'i', 'stance', '"', ':', '▁1.5', '"', 'D', 'if', 'ference', '"', ':', '▁1', '"', 'd', 'i', 'stance', '_', 'from', '_', 're', 'mot', 'e', '_', 'control', '▁(', 'm', ')', '"', '▁', '0', '"', 'B', 'atter', 'y', '_', 'level', '"', ':', '▁0,', '▁"', 'Win', 'd', 'speed', '"', ':', '▁8', '"', 'wind', 'speed', '▁(', 'm', '▁', '/', 's', '▁', ')', '"', '"', 'name', '"', '▁', ':', '▁"', 'water', '",', '▁"', 'D', 'if', 'fer', 'en', 'n', '---', '▁', '-', '▁"', '-', '.', '-', '-', '▁"', '▁"', '-', '▁"', '"', '▁"', '▁"', '▁"', '-', '"', '_', '-', '-', '"', '-', '"', '▁', "'", '"', '"', '/', '"', "'", "'", '"', ':', '"', '▁', "'", "'", '(', '"', '"', '▁', '/', '"', '▁', '%', '"', '",', '▁"', "'", '"', '(', '(', '(', 'm', '"', '▁', '&', '/', '"', '/', '(', '(', '"', '(', '"', '/', '"', '"', '"', '(', '/', '"', '(', '▁"', '(', '"', '"', ':', ')', '"', '"', '-', '/', '"', '/', '/', '"', "'", '"', "'", "'", "'", ':', '"', '"', "'", '/', "'", '"', '(', "'", "'", '-', '"', '(', '%', '"', "'", '(', '(', "'", '(', "'", '/', '(', "'", '"', '/', "'", "'", ')', "'", "'", ';', '"', "'", ':', "'", "'", '/', '"', '-', "'", "'", '.', "'", "'", '▁"', "'", "'", '▁(', "'", "'", '&', "'", "'", ',', "'", "'", '▁', "'", '(', '"', "'", ';', "'", "'", '0', "'", "'", '1', "'", "'", '6', "'", "'", '8', "'", "'", '7', "'", "'", '9', "'", "'", '10', "'", "'", '11', "'", "'", '12', "'", "'", '13', "'", "'", '14', "'", "'", '2', "'", "'", '3', "'", "'", '4', "'", "'", '5', "'", "'", '00', "'", "'", '01', "'", "'", '09', "'", "'", '04', "'", "'", '08', "'", "'", '03', "'", "'", '02', "'", "'", '06', "'", "'", '07', "'", "'", '72', "'", "'", '69', "'", "'", '82', "'", "'", '84', "'", "'", '86', "'", "'", '88', "'", "'", '89', "'", "'", '91', "'", "'", '90', "'", "'", '81', "'", "'", '64', "'", "'", '71', "'", "'", '61', "'", "'", '62', "'", "'", '59', "'", "'", '66', "'", "'", '68', "'", "'", '67', "'", "'", '79', "'", "'", '76', "'", "'", '73', "'", "'", '83', "'", "'", '87', "'", "'", '63', "'", "'", '58', "'", "'", '49', "'", "'", '51', "'", "'", '32', "'", "'", '33', "'", "'", '70', "'", "'", '75', "'", "'", '60', "'", "'", '65', "'", "'", '78', "'", "'", '74', "'", "'", '77', "'", "'", '80', "'", "'", '93', "'", "'", '92', "'", "'", '28', "'", "'", '29', "'", "'", '34', "'", "'", '55', "'", "'", '57', "'", "'", '54', "'", "'", '36', "'", "'", '44', "'", '"', '64', "'", '"', '69', "'", '"', '71', "'", '"', '61', "'", '"', '62', "'", '"', '72', "'", '"', '63', "'", '"', '66', "'", '"', '68', "'", '"', '67', "'", '"', '86', "'", '"', '84', "'", '"', '82', "'", '"', '59', "'", '"', '6', "'", '"', '4"', ':', '▁"', "'", '"', '1', "'", '"', '2', "'", '"', '3', "'", '"', '0', "'", '"', '12', "'", '"', '11', "'", '"', '32', "'", '"', '28', "'", '"', '8', "'", '"', '10', "'", '"', '13', "'", '"', '14', "'", '"', '09', "'", '"', '29', "'", '"', '01', "'", '"', '7', "'", '"', '9', "'", '"', '4', "'", '"', '34', '"', '"', '2', '"', '"', '1', '"', '"', '3', '"', '"', '0', '"', '"', '6', '"', '"', '8', '"', '"', '11', '"', '"', '12', '"', '"', '13', '"', '"', '14', '"', '"', '4"', '"', '4"', '4"', '4"', '"', '"', '7', '"', '"', '10', '"', '▁"', '1', '"', '▁"', '2', '"', '▁"', '3', '"', '▁"', '"', '1', '▁"', '"', '2', '▁"', '"', '3', '▁"', '"', '4"', '▁"', '"', '"', '▁"', '4"', '▁"', '4"', '"', '▁"', '0', '"', '▁"', "'", '▁"', '"', "'", '▁"', '▁"', '"', '▁"', '(', "'", '")', '▁"', '"', '(', '")', '"', '"', ';', '"', '"', '&', '"', '"', '#', '"', '"', '%', '"', '")', '"', '▁"', '&', '"', '▁"', '#', '"', '▁"', '%', '"', '▁"', '-', '"', '▁"', '▁"', '&', "'", '"', '▁"', ';', '"', '▁"', '▁', "'", '"', '&', "'", ';', '▁"', '"', ';', '▁"', '▁"', "'", ';', '&', '"', ';', "'", '"', ';', ';', '"', '&', '&', '"', "'", '&', '"', '&', '▁"'], ['▁"', 'D', 'i', 'stance', '_', 'from', '_', 're', 'mot', 'e', '_', 'control', '▁(', 'm', ')', '"', ':', '▁48', '05', ',', '▁"', 'B', 'atter', 'y', '_', 'level', '"', ':', '▁30', '"', 'Norm', 'al', '_', 'frame', '"', ':', '▁false', '"', 'time', '_', 's', 't', 'amp', '"', ':', '▁"', '0:00', '",', '▁"', 'name', '"', ':', '▁', "'", 'building', '▁1', "'", ',', '▁"', 'D', 'if', 'ference', '"', ':', '▁50', ',', '▁"', 'S', 't', 'at', 'us', '"', ':', '▁60', ',', '▁"', 'P', 'o', 's', 'ition', '"', ':', '▁40', ',', '▁"', 'Vol', 'um', 'e', '"', ':', '▁1', '"', 'S', 't', 'and', 'ard', '_', 'frame', '(', 's', ')', ':', '▁', '0', '",', '▁"', 'D', 'i', 'r', '.', '.', '_', '_', '....', '[', '[', '[', '.', '_', '[', '[', ']', '[', '[', '_', '[', '(', '**', '[', '[', '(', '(', '(', 'm', ')', ']', '"', ':', '_', '[', '_', '_', '[', "'", "'", '"', ':', "'", "'", '-', '[', '(', '_', '_', '_', "'", "'", "'", '(', '(', "'", "'", '[', '"', ':', "'", "'", ';', '"', '"', ':', '"', "'", '"', '"', "'", "'", ':', '"', '"', '"', ';', '"', '(', '"', '(', '(', '.', '.', "'", "'", ')', "'", "'", ',', "'", "'", '/', "'", '"', '(', "'", '(', '"', "'", '(', "'", '-', '(', '"', '"', '(', '")', '"', '()', '"', '(', '*', '(', '(', '*', ')', "'", '(', '*', '"', '(', '%', "'", "'", '▁(', "'", "'", '.', '(', "'", '.', "'", '(', '.', "'", '"', '▁(', "'", '(', '-', "'", "'", '▁"', "'", "'", '▁', "'", "'", '1', "'", "'", ').', "'", "'", '0', "'", "'", '&', "'", "'", '#', "'", "'", '▁=', "'", "'", '$', "'", "'", '!', "'", "'", '?', "'", "'", '_', "'", '.', '▁(', "'", '.', '-', "'", '.', '_', "'", '"', "'", '.', '."', "'", ')', '"', "'", ';', "'", "'", '"', "'", ':', "'", '"', '"', '(', '"', "'", '(', "'", ')', '"', '"', '-', '"', '"', '(', "'", '"', '-', "'", '"', '▁"', "'", '"', ';', "'", '")', '"', '"', '▁"', '"', "'", '▁"', '"', '"', '▁', "'", '".', '▁"', "'", '.', '▁"', '"', '▁"', '(', "'", ')', '▁"', '"', ':', ':', '"', '▁"', "'", '"', '▁', '0', "'", '"', '0', "'", '(', '0', "'", '0', '"', "'", '-', '0', "'", '-', '-', '0', '"', '-', '-', "'", '-', '"', '-', '0', '▁', '0', '"', '"', '0', '"', '▁', '-', '0', '▁(0', "'", '▁"', '0', "'", '▁', '0', '▁"', '0', '"', '▁"', '1', "'", '"', '1', "'", '0', '▁"', '2', "'", "'", '2', "'", '"', '3', "'", "'", '4"', ':', '▁"', "'", '1', '"', "'", '0', '(', "'", '0', ')', '"', '▁"', '0', '(', '0', '(', '1', "'", ')', ':', '▁"', '1', '"', '"', '2', "'", '00', "'", '1', '▁"', '0', '▁', "'", '0', '▁(', "'", '"', '2', '"', '"', '1', '"', '▁"', '3', '"', '"', '3', '"', "'", '2', '"', '(', '0', '"', '(', '1', '"', '(', '2', "'", '(', '1', '(0)', "'", '"', '6', "'", "'", '6', "'", '"', '8', "'", "'", '8', "'", '"', '9', "'", "'", '9', "'", '"', '10', "'", "'", '3', "'", '"', '7', "'", "'", '7', "'", '"', '5', "'", "'", '5', "'", '"', '4', "'", "'", '4', "'", '"', '11', "'", "'", '10', "'", '"', '12', "'", "'", '11', "'", '"', '13', "'", "'", '14', "'", "'", '09', "'", "'", '01', "'", "'", '00', "'", "'", '69', "'", "'", '72', "'", "'", '61', "'", "'", '64', "'", "'", '71', "'", "'", '59', "'", "'", '91', "'", "'", '12', "'", '"', '01', "'", '"', '14', "'", '"', '4"', "'", "'", '62', "'", "'", '66', "'", "'", '68', "'", "'", '86', "'", "'", '82', "'", "'", '81', "'", "'", '93', "'", "'", '89', "'", "'", '88', "'", "'", '90', "'", "'", '96', "'", "'", '79', "'", "'", '67', "'", "'", '70', "'", "'", '73', "'", "'", '78', "'", "'", '13', "'", '"', '69', "'", '"', '71', "'", '"', '72', "'", '"', '91', "'", '"', '82', "'", '"', '86', "'", '"', '64', "'", '"', '66', "'", '"', '61', "'", '"', '63', "'", "'", '63', "'", '"', '68', "'", '"', '59', "'", '"', '62', "'", '"', '67', "'", '"', '79', "'", '"', '&', "'", '"', '81', "'", '"', '73', "'", '"', '74', "'", '"', '84', "'", '"', '88', "'", '"', '93', '"', '"', '71', '"', '"', '72', '"', '"', '69', '"', '"', '84', '"', '"', '64', '"', '▁"', '64', '"', '▁', '68', '"', '▁"', '▁"', '64', '▁', '69', '"', '▁', '71', '"'], ['▁"', 'B', 'atter', 'y', '_', 'level', '"', ':', '▁20', '"', 'Norm', 'al', '_', 'frame', '"', ':', '▁false', '"', 'time', '_', 's', 't', 'amp', '"', ':', '▁"', '0', ':', '04', '",', '▁"', 'name', '"', ':', '▁', "'", 'hall', 'way', "'", ':', '▁3.', '0', '"', 'd', 'i', 'stance', '"', ':', '▁', '5.0', '▁', "'", 'time', '_', 't', 'amp', "'", ':', '▁', '0', ':', '06', '"', 'name', '_', 'of', '_', 'hall', 'way', '"', ':', '▁1.', '01', '"', 'normal', '_', 'frame', "'", ':', '▁false', '▁"', '0', '0:00', '",', '▁"', 'num', 'ber', '"', ':', '▁2.0', '4"', 'num', 'ber', '_', 'of', '-', 'stair', 's', '"', ':', '.', '.', '▁', '.', '▁', '-', '▁»', '▁--', '..."', '▁("', '▁"', '▁"', '▁', '/', '-', '-', '▁', '-', '▁', "'", '-', "'", '▁', '_', '_', '_', '▁', '_', '_', '▁', ']', '"', '-', '-', '"', '"', '"', '▁"', '▁"', '..."', ':', '"', ';', '..."', '▁"', '"', ':', '"', '"', ':', "'", "'", '"', '"', "'", "'", "'", ':', '"', "'", '"', "'", ':', "'", '"', ':', ':', "'", '"', '(', '"', "'", '▁"', '(', '▁"', '&', '"', '"', '"', '▁"', "'", "'", '▁"', "'", '"', '▁"', '&', "'", '"', ';', '"', "'", ';', '"', '"', ';', "'", "'", ';', "'", '"', '(', "'", "'", '.', "'", "'", '-', "'", "'", ',', "'", "'", "'", "'", '?', "'", "'", '&', "'", "'", '▁', "'", "'", '(', "'", '.', "'", '"', "'", '▁"', '"', '&', "'", '▁"', '"', "'", '.', '▁"', "'", '."', '"', '&', '"', "'", '▁"', '&', '&', '"', '▁"', '"', '&', '&', "'", '.', '▁', "'", '.', '&', '"', '▁', "'", '&', '"', '&', '▁"', '&', '▁', "'", '"', '&', ';', '"', '&', '/', "'", "'", '/', '"', "'", '&', '▁"', "'", '&', '&', '&', '#', "'", "'", '▁(', "'", "'", '!', "'", "'", ')', '▁"', "'", ';', '▁"', "'", '(', '"', "'", '-', '"', "'", '(', '(', "'", '"', '-', '"', '"', '(', '"', '"', '-', "'", '"', '/', "'", '"', '▁(', "'", '".', "'", '".', "'", '▁"', '.', "'", '.', '▁"', '.', '.', "'", '.', '."', '▁"', '."', '▁', '&', '"', '/', '".', '."', "'", ')', '"', '0', "'", "'", '0', "'", '"', '1', "'", "'", '1', "'", '"', '2', "'", "'", '3', "'", "'", '2', "'", '"', '3', "'", '"', '4"', "'", "'", '6', "'", "'", '8', "'", "'", '9', "'", "'", '10', "'", "'", '7', "'", "'", '11', "'", "'", '12', "'", "'", '13', "'", "'", '14', "'", "'", '09', "'", "'", '01', "'", "'", '00', "'", "'", '#', "'", '"', '0', '"', "'", '0', '"', '(', "'", '0', "'", '0', '&', '"', '0', '&', "'", '00', "'", ';', '0', "'", '0', ':', '0', "'", '1', '"', '2', '"', '3', '"', '122', '"', '"', '1', '"', '"', '2', '"', "'", '1', '"', "'", '3', '"', '"', '3', '"', "'", '2', '"', '▁"', '0', '"', '"', '6', '"', '"', '4"', '"', '"', '0', ':', '01', '"', '"', '7', '"', '"', '8', '"', '"', '9', '"', '"', '10', '"', '"', '11', '"', '"', '12', '"', '"', '13', '"', '"', '14', '"', '"', '01', '"', '0', '▁"', '1', '"', '0', '▁', '0', '"', '0', ')', '0', '"', '▁', '0', "'", '4"', '0', '"', '1', '▁"', '0', "'", '9', '"', '▁', '-', '0', '"', '▁"', '14', '"', '▁', '-', '"', '1', '&', '"', '1', '▁01', '3', '"', '03', '4"', '02', '▁', '0', '&', '23', '▁', '0', '▁', "'", '0', '▁3', "'", '▁', '0', ';', '00', '4"', '"', '9', '"', '10', '"', '11', '"', '12', '"', '13', '"', '14', '"', '7', '"', '6', '"', '8', '"', '4"', '▁', '0', '(', '"', '0', ';', '"', '0', '(', '0', '"', ';', '"', '%', '"', '"', '%', '%', '"', '&', '#', ';', '"', '(', '&', '"', '(', '(', '"', '(', '%', '"', '()', '"', '")', '"', '(', ':', '"', '(', '▁"', '"', '"', '#', '"', '"', '/', '"', '"', '+', '"', '".', '▁"', '"', '(', '.', '.', '▁"', '(', '"', '&', '(', '"', '▁"', '(', '(', '(', '▁"'], ['▁Na', 'N', ',', '▁"', 'D', 'i', 'stance', '"', ':', '▁', '0.5', '▁"', 'time', '_', 's', 't', 'amp', '"', ':', '▁"', '0', ':', '07', '"', ':', '▁Na', 'N', "'", 's', '▁"', 'D', '"', ':', '▁1.5', '▁"', '"', '▁"', 'time', '"', ':', '▁1', '"', '▁(', ')', ':', '▁"', '")', ':', '▁"', '1', '"', ':', '"', '0', ':', '08', '"', ':', ':', '"', '12', '"', '()', '"', '"', '"', '(', '")', ':', '"', '"', '1', ':', '"', '(', '"', '")', '"', '"', '"', '(', '0', ':', '01', ')', ':', '▁', '0', ':', '03', ')', '▁', '.', '.', "'", '▁(', '▁"', '.', '.', '▁', "'", '"', '▁', ':', '.', '▁"', '."', '▁"', '▁', '-', '▁', '-', '."', '.', '▁(', '▁(', '▁', '-', '▁(', '▁("', '(', '(', '(', '▁', ')', ':', ':', '▁(', '(', '(', '▁', ')', '"', '(', '▁', "'", '(', '()', '"', '▁', "'", "'", '"', "'", '"', '"', "'", '"', '()', ':', "'", "'", "'", ':', '"', "'", "'", '"', "'", ':', '▁"', '(', '"', '"', '▁"', '(', '(', '"', '(', '▁"', '"', '(', "'", "'", '(', '"', "'", '(', "'", '"', "'", '▁"', "'", '"', '▁"', "'", "'", '▁"', '"', '"', ':', "'", '"', '-', '"', '"', '-', "'", "'", '-', '"', "'", '-', "'", '-', '▁"', "'", '-', '-', '"', '-', '▁"', '"', '-', '-', '▁"', '-', '"', '▁"', '-', '-', "'", '"', '▁(', '-', "'", '(', '-', '"', '(', '-', "'", ')', "'", "'", '▁(', "'", "'", '.', "'", "'", ')', '▁"', "'", '(', '▁"', "'", '.', '▁"', '"', '▁"', '"', "'", ')', '"', '"', '▁("', '")', '"', '▁"', '1', '"', '▁"', '0', '"', '"', ';', '"', "'", ';', '"', '(', ':', '"', ':', ')', '"', ':', '(', '(', "'", '(', ':', "'", '(', '!', "'", "'", ';', "'", "'", ',', "'", "'", 'F', "'", "'", 'E', "'", "'", 'I', "'", "'", 'O', "'", "'", 'S', "'", "'", 'B', "'", "'", 'D', "'", "'", 'C', "'", "'", 'A', "'", "'", 'N', "'", "'", 'Re', "'", "'", 'H', "'", "'", 'P', "'", "'", 'L', "'", "'", 'Y', "'", "'", '#', "'", "'", '▁', "'", 'F', '"', "'", "'", "'", '1', "'", '"', 'F', '"', '"', 'F', "'", '"', 'I', '"', "'", 'F', '(', '"', 'F', '(', "'", ')', 'F', '"', 'F', '.', 'F', '.', '▁"', 'F', '"', '▁"', 'F', '."', 'F', '."', 'F', '.', 'G', '.', 'F', '"', ':', 'F', '"', 'H', '"', 'FF', '"', '(', 'F', '"', 'D', '"', 'F', ')', 'F', '(', 'F', '")', 'F', '"', 'G', '"', 'F', '▁"', 'F', '(', '(', 'F', ')', '▁"', 'F', "'", '(', 'F', "'", 'F', '▁"', '"', 'F', ':', '"', 'F', ';', '"', 'F', '-', 'F', '"', 'O', 'F', '"', 'N', '"', 'F', '".', 'F', '"', 'AF', '"', 'IF', '"', 'FI', 'F', '"', 'S', '"', 'F', '")', '"', 'F', '&', 'F', '"', 'Fi', 'F', '"', 'Re', '"', 'F', '/', 'F', '"', 'P', '"', 'F', '"', 'F', '0', '"', 'F', '2', '"', 'F', '3', '"', 'F', '4"', 'F', '1', 'F', '"', 'FA', '"', 'F', '5', 'F', '"', 'B', '"', 'F', '6', 'F', '"', 'C', '"', 'F', '9', 'F', '"', 'E', '"', 'F', '7', 'F', '"', 'U', '"', 'F', '4"', 'F', '"', 'Est', '"', 'F', '#', 'F', '"', 'L', '"', 'F', '",', 'F', '"', 'De', 'F', '"', 'R', '"', 'F', '▁F', '"', 'W', '"', 'FI', '"', 'FI', '"', 'F', 'IRE', 'IRE', 'IRE', 'F', 'IRE', 'FI', 'FI', '"', 'FI', '▁"', 'F', 'IRE', '"', 'FF', 'I', '"', 'DF', '"', 'FIN', '"', 'F', 'RU', '"', 'FIN', '"', 'F', 'EU', '"', 'F', 'FF', '"', 'FF', 'A', '"', 'FI', 'FI', 'FI', '▁"', 'I', '"', 'F', 'FO', '"', 'F', 'FE', '"', 'F', 'RE', '"', 'F', 'FER', '"', 'FE', 'F', '"', 'F', 'END', '"', 'F', 'IE', '"', 'FI', 'F', '"', 'F', 'Y', '"', 'F', 'FIN', '▁"', 'FF', '(', 'FI', '"', 'I', 'I', '"', '"', 'I', '▁"', 'F', '▁"', 'I', '▁"', 'FI', '"', '"', 'FIN', 'FIN', '"', 'FIN', '▁"', 'FIN', '"', 'FI', 'I', '"', 'FIN', 'FI', '"', 'FIN', 'F', '"', '▁F', '▁"', 'FF', 'I', 'FI', ')', '"', 'FT', 'I', '"', 'FA', 'F', '"', 'FR', 'F', '"', 'F', 'RO', '"', 'F', '...', 'F', '"', 'FE', '"', 'FI', 'FA', '"', 'FA', '▁"', 'F', 'FF', 'FF', '"', 'FIN', 'FA', '"', '"', 'FI', 'FIN', '"', '▁"', 'F', 'IFF', 'I', 'I', 'FI', 'FI', 'IF', 'IF', 'IFF', 'IF', 'IF', 'IF', 'IF', 'FIN', '"'], ['▁"', 'time', '_', 's', 't', 'amp', '"', ':', '▁', '0', ':', '04', '"', 'name', '"', ':', '▁"', 'stair', '",', '▁"', 'D', 'i', 'stance', '"', ':', '▁3.', '0', '"', ':', '▁20', '"', 'b', 'atter', 'y', '_', 'level', '"', ':', '▁20%', '"', 'date', '_', 's', 'lot', '"', ':', '▁3', ':', '06', '"', 'd', 'i', 'stance', '_', ':', '▁', '3.5', '"', '"', 'de', 'fini', 'tion', '"', ':', '▁30', '"', 'data', '_', 'type', '"', ':', '▁40', '"', 'D', 'i', 'men', 'sion', '"', ':', '▁2', ':', '06', '"', ':', '▁4', ':', '06', "'", "'", ':', '▁', '-', '0', ':', '06', ',', '▁"', 'name', '"', '▁', ':', '▁"', '▁"', '.', '▁', '-', '▁(', "'", '-', '▁', '/', '▁', "'", '"', '▁', '_', '▁', '_', '".', '."', '-', '"', '"', '▁', '&', '"', "'", '"', '_', "'", "'", '"', '/', '"', '▁"', '"', '_', '_', '▁"', ':', '"', "'", '"', ':', ':', "'", '"', '"', ':', '"', '"', "'", "'", "'", '▁"', '"', ':', "'", "'", '▁"', "'", "'", '▁', "'", "'", ';', '▁', "'", ':', '▁"', '"', '&', '"', '"', '▁"', '"', '"', '"', '▁', "'", '&', "'", "'", '-', '"', '"', '&', "'", '"', '&', '&', "'", ':', ':', ':', '"', ':', '.', "'", ':', '"', "'", ':', "'", '"', "'", '&', ':', '"', ':', '"', '&', ':', '▁"', "'", '&', '&', '&', ';', ':', '▁"', '&', '&', ':', '▁', "'", ';', ':', ':', "'", ';', "'", "'", '.', "'", "'", '(', "'", "'", '&', '"', "'", ';', '"', "'", '▁"', '&', "'", '&', ';', "'", ';', '▁"', "'", '"', '(', "'", '"', '▁"', "'", ';', '&', "'", ';', ';', "'", '"', ';', "'", ':', ';', "'", '▁"', "'", ':', '&', "'", '▁"', '▁"', "'", '▁"', '▁', "'", '.', '.', '▁', "'", "'", ';', '"', ';', "'", '&', "'", '▁"', "'", '-', "'", "'", '_', "'", "'", ',', "'", "'", '?', "'", "'", ')', "'", "'", '▁(', "'", "'", '/', "'", "'", '#', "'", "'", '%', "'", "'", '0', "'", "'", '1', "'", "'", '2', "'", "'", '6', "'", "'", '8', "'", "'", '9', "'", "'", '10', "'", "'", '11', "'", "'", '12', "'", "'", '13', "'", "'", '14', "'", "'", '09', "'", "'", '00', "'", "'", '01', "'", "'", '02', "'", "'", '03', "'", "'", '04', "'", "'", '08', "'", "'", '91', "'", "'", '93', "'", "'", '86', "'", "'", '89', "'", "'", '84', "'", "'", '88', "'", "'", '90', "'", "'", '96', "'", "'", '92', "'", "'", '94', "'", "'", '69', "'", "'", '82', "'", "'", '72', "'", "'", '64', "'", "'", '66', "'", "'", '71', "'", "'", '81', "'", "'", '83', "'", "'", '87', "'", "'", '79', "'", "'", '68', "'", "'", '59', "'", "'", '61', "'", "'", '62', "'", "'", '63', "'", "'", '67', "'", "'", '58', "'", "'", '65', "'", "'", '76', "'", "'", '28', "'", "'", '32', "'", "'", '29', "'", "'", '33', "'", "'", '36', "'", "'", '49', "'", "'", '7', "'", "'", '73', "'", "'", '78', "'", "'", '75', "'", "'", '▁8', "'", "'", '▁9', "'", "'", '▁7', "'", "'", '4', "'", "'", '3', "'", '"', '2', "'", '"', '1', "'", '"', '3', "'", "'", '4"', ':', '"', '1', '"', '"', '2', '"', '"', '3', '"', '"', '4"', ':', '▁"', '1', '"', '▁"', '2', '"', '▁"', '3', '"', '▁"', '1', '▁"', '"', '1', '"', '1', ':', '▁"', '2', ':', '▁"', '3', ':', '▁"', '4"', ':', '▁', '4"', ':', ':', '▁"', '0', ':', '0', '"', '"', ';', '"', '"', '(', '"', '"', '1', '▁"', '2', '▁"', '"', '2', '▁"', '1', "'", '▁"', '2', "'", '▁"', '0', "'", '"', '0', "'", '0', '"', "'", '1', '"', "'", '2', '"', "'", '3', '"', "'", '4"', '"', '"', '6', '"', '"', '0', '"', '▁"', '0', '"', '▁', '0', "'", '4"', "'", '"', '4"', '"', '4"', '▁"', '"', '4"', '▁', '0', '"', '0', '▁"', '0', '▁"', '12', '"', '▁', '4"', '▁"', '02', "'", '4"', '▁', '4"', '"', '▁"', '4"', '▁"', '4"', '▁', "'", '0', '▁"', '"', '04', '"', '▁"', '1', ':', '0', "'", '1', ':', '2', "'", '0', ':', '01', "'", '"', '6', "'", '"', '7', "'", '"', '8', "'", '"', '9', "'", '"', '10', "'", '"', '11', "'", '"', '12', "'", '"', '13', "'", '"', '14', "'", '"', '09', "'", '"', '01', '"', '"', '11', '"', '"', '13', '"', '"', '14', '"', '"', '12', '"', '"', '10', '"', '"', '01', "'", '▁"'], ['▁"', 'Norm', 'al', '_', 'frame', '"', ':', '▁false', '"', 'time', '_', 's', 't', 'amp', '"', ':', '▁"', '0', ':', '03', '",', '▁"', 'name', '")', ':', '▁"', 's', 'a', 'il', 'boat', '",', '▁"', 'D', 'i', 'stance', '"', ':', '▁', '1.0', '"', 'tim', '_', 's', 'amp', '"', '▁', ':', '▁"', '0:00', '",', '▁', "'", 'name', "'", ':', '▁"', 'per', 'o', 's', 'n', '▁on', '▁sail', 'boat', '"', ':', '▁1', '0.0', '"', 'd', 'i', 'stance', '="', '0', ':', '04', '"', ':', '▁0.', '01', '"', 'date', '_', 'time', '"', ':', '▁1.', '03', '"', 'name', '"', ':', '▁sail', 'boat', ',', '▁"', 'D', 'if', 'ference', '"', ':', '▁2.0', '"', 'text', '_', 's', '.', '.', '▁"', '"', '"', '▁»', '"', '▁"', "'", '▁(', '▁', '/', '.', '▁', '-', '▁', '-', "'", '"', '▁', '."', '"', '▁"', '▁"', '▁', '&', '"', "'", '"', '/', '"', '"', ':', '"', ':', '".', '▁', "'", '"', '"', '▁', "'", "'", '"', '_', '"', "'", '"', "'", "'", '"', '▁"', '"', ':', ':', '▁', "'", ':', '"', '"', '"', '▁"', "'", '"', ':', "'", '"', ';', '▁"', '"', '"', ';', '"', '"', '&', '"', '"', "'", ':', "'", "'", "'", ':', ':', '▁"', "'", "'", ';', '▁"', "'", ':', '."', '"', '(', '▁"', "'", '(', '(', '"', '(', '(', "'", '"', '(', '"', '"', '-', '"', '(', "'", "'", '(', "'", '(', '"', "'", '(', '▁"', '"', "'", '▁"', '"', '▁"', '(', '"', '▁"', ':', '"', '▁"', '-', '"', '▁"', '▁"', '"', '-', "'", "'", '▁"', "'", '▁"', '▁"', "'", '-', "'", '"', '-', '▁"', "'", '.', "'", "'", '-', '"', "'", '-', '▁"', '"', ';', "'", "'", '.', '▁"', "'", ';', '"', "'", ';', "'", '".', "'", '"', '▁(', "'", "'", ')', '▁"', "'", ')', '"', "'", '."', "'", ')', "'", "'", '▁', "'", '.', "'", "'", '▁(', "'", '".', "'", '.', '.', '▁"', '...', "'", ';', '.', '."', ';', '.', '.', ';', '.', "'", ':', '.', '.', '▁"', ';', '"', ';', ';', '"', '(', '.', '.', ':', '.', "'", '(', '.', "'", '▁"', ';', ';', ';', "'", ';', ';', '▁"', ';', "'", '.', ';', '"', '▁"', ';', '▁"', '(', '."', '▁', ';', '▁"', '."', '▁"', '.', '▁"', '"', '▁', '-', '"', '"', '▁("', '▁"', '&', '"', '▁"', '/', '"', "'", '/', '"', '▁"', '0', "'", '"', '&', "'", "'", '&', '"', "'", '&', "'", '"', '/', "'", "'", '/', "'", '"', '#', "'", "'", '0', "'", "'", '1', "'", "'", '2', "'", "'", '3', "'", "'", '#', '"', "'", '#', "'", '"', '0', "'", '▁"', '0', '"', '"', '#', '"', '"', '0', '"', '▁"', '1', '"', '▁"', '2', '"', '▁"', '3', '"', '▁"', '4"', '▁"', '"', '&', '▁"', '"', '/', '"', '&', '&', '"', '▁', '0', "'", '0', '"', '▁', '&', '▁"', '0', '&', '"', '&', '▁', '0', '"', '&', '/', '"', '0', '&', "'", '0', '&', '&', '&', '▁"', '&', '&', "'", '&', '&', '#', '"', '&', '#', '&', '"', ';', '&', '&', ';', '"', '&', ';', '&', '"', '(', '&', '&', '(', '"', '&', '(', "'", '&', '▁"', "'", '&', '(', '(', '(', '&', "'", '(', '&', '"', '▁(', '(', '()', '&', '"', ':', '&', '"', '-', '&', '"', '/', '(', '"', '-', '(', '"', '/', '/', '"', '-', '-', '"', '-', '/', '"', '(', ':', '"', '&', '-', '"', '/', '▁"', '"', '()', '"', '")', '▁"', '"', '▁(', ')', '"', '▁"', '%', '"', '"', '/', '&', '"', '1', '&', '"', '2', '&', '"', '3', '&', '"', '4"', '"', '"', '1', '"', '"', '2', '"', '"', '3', '"', '"', '4', '&', '"', '9', '&', '"', '6', '&', '"', '7', '&', '"', '8', '&', '"', '0', ':', '0', '"', "'", '1', '&', "'", '1', '"', '2', "'", '"', '1', "'", '"', '3', "'", '"', '2', '"', '0', '"', '12', '"', '14', '"', '"', '3', '"', '(', '"', '4"', '&', '"', '4"', '"', '6', '"', '"', '9', '"', '"', '4"', '4"', '4"', '"', '0', '▁"', '0', '▁"', '134', '"', '"', '2', '▁"', '03', '"', '4"', '0', '"', '2', "'", '0', '23', "'", '4"', '0', "'", '2', '"', '00', '"', '0', '▁01', '"', '0', ')', '0', '"', '2', '▁', '0', '▁"', '22', '▁"', '2', "'", '3', '"', ':', ')', '"', ':', ';', '"', '0', ';', '"', '2', ':', '"', '0.0', '"'], ['▁"', 'B', 'atter', 'y', '_', 'level', '"', ':', '▁', '0', '"', 'Norm', 'al', '_', 'frame', '"', ':', '▁false', '"', 'time', '_', 's', 't', 'amp', '"', ':', '▁"', '0', ':', '01', '",', '▁"', 'd', 'i', 'stance', '"', ':', '▁1', '▁km', ',', '▁"', 'f', 'light', '▁distance', '"', ':', '▁2', '▁km', ';', '▁"', 'd', 'if', 'ference', '"', ':', '▁3', '▁km', ';', '▁', "'", 'd', 'if', 'fusion', "'", ':', '▁', '1.0', '▁km', ';', '▁and', '▁', "'", 'f', 'light', "'", ':', '▁1', 'km', ';', '▁"', 'f', 'lash', '"', ':', '▁0.', '01', '▁km', ';', '▁[', 'e', 'mail', '▁protected', ']', ':', '▁false', ';', '▁"', 'time', '_', 't', 'mp', '"', ':', '▁true', ';', '▁"', 'date', '_', 's', '...', '▁"', '▁', '-', '.', '-', '▁[', '[', '[', '.', '.', ']', ']', '[', '."', '"', '[', '.', '_', '_', '[', '."', ':', '"', '-', '_', '"', '".', '.', '_', '.', '.', "'", '"', ';', '▁', "'", '"', "'", '_', '_', '_', "'", "'", '"', ':', '"', ';', '▁"', '"', ':', "'", "'", "'", ':', '▁"', '"', '▁"', "'", ';', '"', '▁', "'", "'", ';', '▁"', '"', ';', '"', '/', '"', '(', '"', '_', '_', ':', '"', ':', ':', '"', '"', '"', '(', ':', '"', '(', "'", "'", '(', "'", '"', '"', "'", '"', "'", "'", '▁"', "'", "'", '-', "'", "'", '/', '"', "'", ':', '"', "'", ';', "'", "'", '.', "'", "'", '_', "'", "'", ',', "'", "'", '&', "'", "'", '▁', "'", ':', "'", '"', '&', "'", '"', ';', ';', "'", ';', '"', "'", '&', ';', "'", '"', '▁"', "'", ';', ';', '"', ';', "'", '▁', ';', "'", '&', '&', "'", ';', '▁', ';', ';', ';', '.', "'", ';', '&', "'", '▁', '&', '&', ';', '▁', '&', "'", '&', '▁', '&', '▁', "'", '&', '"', '▁', '&', '"', '&', '&', '&', '.', '&', '&', '"', "'", '.', '▁(', "'", "'", '▁(', '▁', "'", '.', '▁', "'", ';', '.', '▁', ';', '▁', '/', '▁', "'", '(', "'", '/', "'", "'", ')', '▁', "'", '".', '▁"', '▁', "'", '/', "'", '▁', ':', '▁', "'", '#', "'", '(', "'", '.', '▁(', '.', '(', '.', '/', "'", '.', '▁"', "'", '.', '.', "'", '."', '▁', ':', '▁"', "'", '"', '(', '"', '(', '(', '"', "'", '(', '(', '(', "'", '(', '"', '▁(', '(', '(', '.', '.', '(', "'", '.', '(', '()', "'", '(', '.', "'", '"', '▁(', '.', '.', '▁(', '.', '▁(', '(', '.', '(', '.', '▁"', '...', '▁"', '(', '."', '(', '.', '/', '.', '."', "'", '▁(', '.', "'", '(', '▁"', '.', "'", '▁"', '.', '▁"', '"', "'", '▁"', '(', '"', '"', '▁"', '"', '"', '▁(', "'", '"', '▁("', '.', '.', ')', "'", "'", '")', '▁"', "'", '▁"', '"', '(', '▁"', "'", '(', '")', '"', "'", ')', '"', "'", '/', '(', '"', '/', '"', '()', '"', '"', '&', '"', '"', '/', '/', '"', '/', '(', '(', '/', '/', '/', '(', "'", '/', '/', "'", '"', '/', "'", '/', '.', '/', '/', '.', "'", '/', '▁"', "'", '/', '&', '/', '.', '▁"', '/', '/', '▁//', '▁"', '/', "'", '(', '/', '"', '"', '▁', '/', '"', '▁//', '▁(', "'", '/', '▁(', '▁', '/', '(', '▁', '/', '▁(', '▁(', "'", '(', '▁(', '(', "'", ')', "'", '/', '▁', '0', "'", "'", '▁=', '▁', "'", '-', '/', "'", '▁(', '▁(', '▁(', '(', '▁(', '▁(', '.', '▁', '/', '▁"', '(', '(', '▁(', "'", ')', '▁"', '(', "'", '▁(', '▁"', "'", '▁(', '(', '"', '▁"', '(', '▁"', '"', '▁(', '▁"', '"', '▁"', '/', '"', '▁("', '(', '▁(', '▁"', '(', '▁("', '▁(', '▁("', '")', '▁"', '"', '/', '▁(', '()', '▁"', '/', '(', '▁"', '(', '/', '(', '▁(', '/', '(', '/', '▁(', '/', '/', ')', '▁"', "'", '-', '(', "'", '-', '"', "'", '-', '▁"', "'", '")', "'", "'", '▁("', '(', "'", '")', '▁', "'", ')', '▁(', "'", '▁("', "'", '.', ')', '▁(', '(', '▁"', '/', '.', '▁("', '/', '.', '▁', '0', '(', '(', '0', '(', "'", '0', '(', '0', "'", '(', '1', "'", "'", '0', "'", '"', '0', "'", '0', '"', "'", '0', '▁"', "'", '0', ':', '0', "'", '-', '0', "'", ')', '(', "'", '1', "'", '"', '1', "'", '0', '▁(', '(', '0', '"', '(', '0', ':', '1', "'", '(', '2', "'", "'", '2', "'", '"', '3', "'", "'", '3', "'", '"', '6', "'", "'", '6', "'", '"', '8', "'", "'", '9', "'", "'", '8', "'", '"', '9', "'", '"', '10', "'", "'", '10', "'", '"', '11', "'", "'", '11', "'", '"', '13', "'", "'", '12', "'", "'", '13', "'", '"', '14', "'", "'", '14', "'", '"', '09', "'", "'", '09', "'", '"', '01', "'", "'", '01', "'", '"', '00', "'", "'", '00', "'", '"', '#', "'", "'", '#', '"', "'", '#', "'", '"', '$', "'", "'", '$', '"', "'", '$', "'", '"', '-', "'", '"', '%', "'", "'", ');', '▁"', "'", '$', '(', "'", '##', "'", ')', ':', '▁"', '$', '"'], ['▁"', 'B', 'atter', 'y', '_', 'level', '"', ':', '▁10', '"', 'water', 'proof', '▁drone', '?"', ':', '▁false', ',', '▁"', 'weather', '"', ':', '▁"', 'g', 'loom', 'y', '"', '"', 'time', '_', 's', 't', 'amp', '"', ':', '▁false', '"', 'd', 'i', 'stance', '"', ':', '▁', '5.0', '"', 'f', 'light', '_', 's', 'e', 'que', 'nce', '"', ':', '▁1', '"', 's', 'e', 'rial', '"', ':', '▁3', '"', 'weight', '"', ':', '▁', '5"', 'body', '_', 'size', '"', ':', '▁', '4"', 'wid', 'th', '"', ':', '▁2', '"', 'length', '"', ':', '▁6', '"', 'he', 'ight', '"', ':', '▁8', '"', 'power', '_', 'level', '?', ':', '▁10', '"', ':', '▁11', '"', '[', '[', '[', '_', '[', '[', '"', '[', '[', ']', '"', ']', '[', '[', '...', '[', '[', '.', ']', ']', "'", '"', '[', '"', '[', '...', ']', '"', '/', '"', '[', 'h', ']', '"', '"', "'", "'", '"', '"', '[', "'", "'", '[', "'", '"', '"', '"', "'", "'", "'", "'", '-', '"', '[', 'd', "'", "'", '_', '"', '"', '_', '"', "'", '"', "'", '-', '-', '"', '"', '(', "'", "'", '(', '"', '"', ':', '"', '"', '?"', '"', '&', '"', '"', ';', '"', '")', '"', '"', '-', '"', '&', '&', '"', '&', "'", '"', '(', '"', '&', '/', '"', '"', '#', '"', '&', '#', '"', '"', '/', '"', '&', '▁"', '&', '"', "'", '&', '&', '&', "'", "'", '&', '"', ';', "'", "'", ';', '"', "'", ';', "'", '"', '&', ';', '"', '&', '?', '&', "'", ';', '&', "'", '&', ';', '&', '&', ';', "'", '&', "'", '?', "'", "'", ':', "'", "'", '.', '&', '&', '#', '&', '&', '?"', '&', '▁', '&', '&', '▁', "'", "'", '▁', '&', "'", '&', "'", '▁', "'", '"', ';', '&', '"', '▁', '&', '"', '&', '"', '-', '&', '&', '-', '&', "'", '-', "'", "'", '-', '&', '-', '"', "'", ':', '"', "'", '.', '▁', "'", '&', '-', '-', "'", '"', '-', "'", '-', '▁"', "'", "'", '▁"', '&', "'", '▁"', '"', "'", '▁"', "'", '"', ':', "'", '"', '▁"', "'", '&', '▁"', '"', ':', ':', "'", ';', ';', '"', ';', ';', "'", ';', '▁"', "'", ';', '.', "'", "'", ',', "'", "'", '?', ';', '"', '▁', ';', "'", '."', '▁', "'", ';', '"', ';', "'", ';', '▁', "'", '.', "'", '"', '▁', "'", '&', '#', "'", "'", '▁(', "'", "'", '/', "'", "'", ')', "'", "'", '1', "'", "'", '2', "'", "'", '3', "'", "'", '0', "'", "'", '#', "'", '"', '0', "'", '"', '1', "'", '"', '2', "'", '"', '3', "'", '"', '4"', ';', '"', '▁"', '"', '-', '-', '&', '"', ':', '"', '0', '".', "'", '0', '"', '"', '0', ';', '"', '0', '"', '1', '"', '2', '"', '3', '"', '4"', '6', '"', '8', '"', '9', '"', '7', '"', '5"', '4"', ':', '"', '0', '&', '"', '0', ':', '"', '&', '2', "'", '0', '&', '2', '"', '&', '3', '"', '&', '4"', '&', '"', '2', '"', '"', '1', '"', '"', '2', '&', '"', '1', '&', '"', '3', '&', '"', '4"', '2', '"', '04', '"', '"', '1', ':', '"', '2', '▁"', '0', '"', '0', '▁"', '0', '&', '1', '&', '2', '&', '3', '&', '4"', '4"', ':', '▁"', '12', '"', '2', ':', '"', '1', '▁', '0', '"', '2', '▁', '0', "'", '2', '"', '3', '"', '"', '4"', '"', '"', '6', '"', '"', '4"', '"', '7', '"', '"', '8', '"', '"', '9', '"', '"', '10', '"', '"', '11', '"', '"', '13', '"', '"', '14', '"', '"', '12', '"', '"', '3', ':', '0', '"', "'", '0', ':', '2', '"', "'", '1', '"', "'", '3', '"', "'", '2', '&', '2', '▁"', '"', '2', '#', '"', '3', '#', '"', '24', '"', '"', '34', '"', '"', '4"', '3', '"', '02', '"', '4"', '0', '"', '1', '▁"', '2', '"', '6', '&', '"', '7', '&', '"', '9', '&', '"', '8', '&', '"', '4', '&', '"', '11', '&', '"', '12', '&', '"', '13', '&', '"', '10', '&', '"', '6', "'", '"', '8', "'", "'", '9', "'", "'", '8', "'", '"', '9', "'", '"', '7', "'", "'", '6', "'", "'", '7', "'", '"', '11', "'", "'", '11', "'", '"', '12', "'", "'", '12', "'", '"', '13', "'", "'", '10', "'", "'", '13', "'", '"', '10', "'", '"', '4', "'", "'", '4"', ':', "'", '1', '&', "'", '2', '#', '3', "'", '24', '"', "'", '"', '64', '"', "'", "'", '4"', "'", '8', '"', "'"], ['▁Na', 'N', ',', '▁"', 'D', 'i', 'stance', '"', ':', '▁', '1.0', '▁"', 'time', '_', 's', 't', 'amp', '"', ':', '▁"', '0', ':', '01', '",', '▁"', 'name', '"', ':', '▁Na', 'N', '.', '▁"', 'D', '"', ':', '▁2.0', '.', '▁"', 'time', '"', ':', '▁(0', ':', '08', '),', '▁"', 'name', '")', '▁Na', 'N', ':', '▁2.0', '▁', "'", "'", 'time', "'", "'", ':', '▁(', '(', '0', ':', '07', ')', "'", "'", '▁', "'", "'", "'", '"', "'", "'", '"', '"', '"', "'", "'", '"', '"', "'", '▁"', '"', '▁', '/', '"', "'", '"', '▁', '0', ':', '02', '"', '▁', '-', '▁', '-', '▁', '-', '▁"', '▁(', '...', '▁', "'", '.', ',', ',', '."', '.', "'", "'", '"', '▁»', '▁(', '▁', ')', '▁', '_', '▁--', "'", '_', '▁"', '▁', '/', '▁"', ',', '▁', ':', '▁', ';', '▁', "'", '"', "'", '▁"', '▁"', '▁', "'", '(', '▁"', '▁', "'", ':', '▁', "'", ';', '▁("', '▁', '&', '▁', '")', '!"', '..."', '."', '▁"', '.', '▁', "'", '.', '▁', '&', '"', '▁', ':', '▁"', '"', '▁', '&', "'", '▁', '&', '▁"', '"', '"', '▁"', '"', '▁"', '▁"', '"', "'", '(', '"', '"', '(', '"', '(', '(', '(', "'", "'", '(', "'", '(', '(', '▁"', '(', "'", '"', '(', '▁"', "'", "'", '▁"', "'", '"', '▁"', "'", '(', '▁"', '"', '(', "'", '.', "'", "'", '▁(', "'", "'", ',', "'", "'", '-', "'", "'", '.', '▁"', "'", '...', "'", '"', ':', "'", "'", ';', "'", "'", '!', "'", "'", ')', "'", '"', '▁(', "'", '"', '-', "'", '")', "'", '(', ':', "'", '"', '&', "'", "'", '/', "'", "'", '#', "'", "'", '&', "'", '"', ';', '"', "'", ';', '"', '"', '"', ':', '"', "'", '&', '"', "'", ':', '"', '"', '&', '▁"', "'", '(', '(', '"', "'", '▁"', '(', '(', '"', '(', "'", '(', '()', '"', "'", ')', '"', '"', '▁("', '▁"', "'", ')', '▁"', '"', '(', '"', '/', "'", '"', 'F', "'", "'", 'F', '"', "'", '."', "'", '-', '"', '"', 'F', '"', '"', '-', '"', "'", ',"', "'", 'F', "'", '".', '▁"', '(', '"', '▁"', 'F', '"', '▁"', '.', '."', '▁"', '(', '.', '.', '▁"', '.', '▁"', '"', ':', ':', '"', '▁"', '-', '"', '▁"', ':', '"', '(', ':', '"', ':', '(', '"', ':', ')', '"', ':', '."', '"', ';', '▁"', '"', ';', ':', '"', ';', ';', '"', '()', '"', '(', '!"', '(', '.', "'", '(', '."', '(', '"', '."', '"', '#', '"', '"', '!', '▁"', '"', '#', '▁"', '"', '&', '"', '▁"', '#', '"', '▁"', '&', '"', '"', '!', '▁"', '.', "'", '.', '-', '"', '▁', '%', '"', '▁"', '%', '"', '")', '"', '▁"', '1', '"', '▁"', '0', '"', '▁"', '2', '"', '▁"', '3', '"', '▁"', '4"', '▁"', '6', '"', '▁"', '7', '"', '▁"', '8', '"', '▁"', '9', '"', '▁"', '10', '"', '▁"', '5"', '▁"', '4"', '▁"', '"', '1', '"', '"', '1', '▁"', '"', '2', '"', '"', '2', '▁"', '"', '3', '"', '"', '0', '"', '"', '3', '▁"', '"', '0', "'", '"', '1', "'", '"', '2', "'", '"', '3', "'", '"', '4"', '"', '6', '"', '"', '7', '"', '"', '8', '"', '"', '9', '"', '"', '10', '"', '"', 'N', '"', '▁"', 'N', '"', '"', 'I', '"', '"', 'S', '"', '"', 'D', '"', '"', 'A', '"', '"', 'B', '"', 'N', '▁"', '"', 'N', "'", '"', 'N', ':', '"', 'N', ';', '"', 'N', '.', 'N', '.', 'F', '.', 'F', '"', 'F', '.', 'N', '"', 'F', '▁"', 'F', '.', 'I', '.', 'F', '."', 'F', '."', 'F', '."', 'N', '"', 'N', '(', '"', 'N', ')', 'F', '"', 'N', '-', '"', 'N', '"', 'F', '(', '"', 'I', "'", '"', 'I', '▁"', '"', 'I', 'I', '"', 'N', '▁', "'", 'N', '"', '(', 'I', '"', 'F', ')', '"', 'N', '&', '"', 'N', '/', '"', 'N', '▁("', '"', 'O', '"', 'N', ',"', 'N', '",', '▁"', 'N', '▁"', 'F', '▁"', 'N', '-', 'F', '"', 'I', ':', '"', 'F', ':', '"', 'I', '-', 'F', '.', 'O', '"', 'F', '".', 'F', '"', 'A', '▁"', '"', 'F', '-', 'F', '-', '"', 'F', '/', 'F', '"', 'O', 'F', '"', '(', 'F', '"', 'P', '"', 'F', '"'], ['▁"', '0', ':', '03', '",', '▁"', 'name', '"', ':', '▁"', 'tre', 'e', '▁1', '",', '▁"', 'D', 'i', 'stance', '"', ':', '▁', '1.0', '"', 'time', '_', 's', 't', 'amp', '"', ':', '▁false', '"', '0', ':', '01', '"', ':', '▁true', '"', '1', ':', '03', '"', ':', '▁True', ':', '▁false', ':', '▁false', ';', '▁true', ':', '▁true', ':', '▁false', '▁"', 'Time', '_', 'frame', '"', ':', '▁Fal', 's', 'e', ':', '▁false', '▁"', '1', ':', '02', '"', ':', '▁real', ':', '▁"', '0', '0:00', '"', ';', '▁false', ':', '▁', '0', ':', '02', ';', '▁"', '0', ';', '01', '"', ';', '▁"', '1', '"', ';', '▁', '1.0', ';', '▁"', '1.0', '"', ':', '▁1', ':', '03', ';', '▁"', '.', '.', '▁', '-', '▁»', '"', '"', '-', '-', '."', '"', '▁"', '"', '▁', "'", '"', '▁', '-', '"', "'", '"', ':', '"', ';', '"', '▁"', '.', '.', '▁"', '"', '▁', '-', '▁"', '"', '/', '"', '"', ':', "'", '"', '"', '"', '▁', ':', '▁"', '"', '"', '▁"', '"', ':', ':', '▁"', ';', '▁', ';', '"', ';', '▁"', '"', ';', '"', '"', "'", '"', ';', "'", '"', "'", "'", '"', '▁"', "'", '"', '(', '"', '"', '(', '(', '"', '(', "'", '"', '&', '"', '"', ';', ';', '"', "'", ':', '"', '"', '&', "'", '"', ':', '"', "'", ';', '"', '&', '&', '"', ';', '&', '"', "'", '&', '"', '&', ';', "'", "'", "'", ';', "'", '&', "'", "'", '▁"', "'", "'", '(', "'", "'", '.', "'", "'", ':', '▁"', "'", '&', '&', "'", '▁"', '"', "'", '.', '▁"', "'", '."', "'", '▁"', '&', '"', '▁"', '&', "'", '.', '&', '"', '▁', "'", "'", '&', '▁"', '"', '&', '▁"', "'", ';', '&', "'", ';', '▁"', '&', '▁"', '&', '&', '&', ';', '"', '▁"', '(', '"', "'", '(', '"', '&', '/', "'", "'", ',', "'", "'", '-', "'", "'", '/', "'", '"', '-', '"', "'", '-', '"', '"', '-', "'", '".', "'", '"', '▁', '&', '"', "'", "'", '▁', '&', "'", '-', '▁"', "'", '(', '(', "'", '(', '&', "'", '(', '▁"', '"', '(', '▁"', "'", '/', '"', '(', '▁', "'", '"', '▁(', "'", "'", ')', '"', "'", ')', "'", "'", '▁("', "'", '▁(', '(', "'", '.', '(', '"', '▁(', '(', '(', '(', '.', '.', '(', '.', "'", '(', '."', '(', '.', '▁"', '(', '.', '▁(', '..."', '"', '▁(', '."', '▁"', '."', '.', '.', "'", '.', '.', '▁"', '".', '▁"', '.', "'", '▁"', '.', '▁', "'", '.', '▁(', "'", '.', '-', "'", '.', '.', "'", "'", '"', '(', "'", ')', '▁"', '(', '()', '"', '()', "'", '()', '▁"', '"', '▁("', '(', '"', '(', '&', '"', '(', ':', '"', '(', ';', '"', '(', '▁(', "'", '"', 'F', '"', '(', '")', '"', '(', '%', '"', '(', '0', '"', '(', '1', '"', '(', '2', '"', '(', '3', '"', '(', '6', '"', '(', '7', '"', '(', '8', '"', '(', '9', '"', '(', '10', '"', '(', '11', '"', '(', '13', '"', '(', '12', '"', '(', '14', '"', '(', '15', '"', '(', '17', '"', '(', '19', '"', '(', '33', '"', '(', '31', '"', '(', '38', '"', '(', '34', '"', '(', '35', '"', '(', '36', '"', '(', '37', '"', '(', '44', '"', '(', '69', '"', '(', '71', '"', '(', '73', '"', '(', '91', '"', '(', '93', '"', '(', '94', '"', '(', '89', '"', '(', '86', '"', '(', '88', '"', '(', '92', '"', '(', '98', '"', '(', '96', '"', '(', '97', '"', '(', '00', '"', '(', '90', '")', '"', "'", '")', '"', '"', '1', '"', '"', '3', '"', '"', '2', '"', '"', '6', '"', '"', '8', '"', '"', '9', '"', '"', '0', '"', '"', '10', '"', '"', '13', '"', '"', '14', '"', '"', '11', '"', '"', '12', '"', '"', '15', '"', '"', '17', '"', '"', '7', '"', '"', '19', '"', '"', '30', '"', '"', '31', '"', '"', '33', '"', '"', '35', '"', '"', '34', '"', '"', '37', '"', '"', '93', '"', '"', '94', '"', '"', '91', '"', '"', '92', '"', '"', '90', '"', '"', '96', '"', '"', '97', '"', '"', '98', '"', '"', '99', '"', '"', '86', '"', '"', '89', '"', '"', '95', '"', '"', '00', '"', '"', '100', '"', '"', '01', '"', '"', '09', '"', '"', '03', '"', '"', '#', '"', '0', '")', '"', '0', "'", '"', '0', '&', '"', '0', '▁"', '0', '"', '0', ';', '"', '0', '"', '2', '&', '"', '1', '&', '"', '3', '&', '"', '4"', '0', '"', '&', '#', '"', '"', '4', '&', '"', '2'], ['▁"', 'B', 'atter', 'y', '_', 'level', '"', ':', '▁20', '"', 'time', '_', 's', 't', 'amp', '"', ':', '▁', '0', ':', '03', '"', 'name', '"', ':', '▁"', 'tre', 'e', '▁', '4"', ';', '▁"', 'D', 'i', 'stance', '"', ':', '▁3.', '0', '"', 'length', '"', ':', '▁3', '"', 'wid', 'th', '"', ':', '▁2', '"', 'd', 'i', 'stance', '_', ':', '▁', '0.3', '"', 's', 't', 'ack', '_', 'size', '"', ':', '▁1', '"', 'size', '_', 'of', '_', 'tre', 'e', '"', ':', '▁', '4"', 'weight', '"', ':', '▁6', '"', 'he', 'ight', '"', ':', '▁8', '"', 'power', '_', 'level', ':', '▁20', '"', ':', '▁10', '"', 'speed', '_', '---', '▁', '-', '---', '[', '[', '[', '"', '"', '_', '"', '-', '"', '[', '[', '*', '"', ']', '"', '"', '/', '"', "'", '"', '_', '_', '_', '/', '"', '▁', "'", '"', '▁"', '"', "'", "'", "'", '"', ':', "'", "'", "'", '▁"', '"', '12', '"', ':', '"', '(', '"', '*', "'", '_', "'", '/', "'", '&', "'", "'", ':', '"', "'", '"', "'", "'", ';', "'", "'", '&', '/', '-', "'", '-', '"', '"', '"', '(', '"', '"', '&', "'", '"', '"', ':', ':', '"', '"', "'", ':', "'", '"', '(', "'", "'", '(', "'", '"', '&', '"', "'", ';', '"', "'", '&', '"', '"', ';', '"', '"', '_', "'", "'", '▁"', "'", "'", '.', "'", "'", '▁', "'", "'", '_', "'", ':', '▁"', "'", '."', ':', '"', '▁"', "'", '"', '▁"', "'", '.', '▁', "'", ';', '▁', "'", '"', '▁', "'", ':', '▁', "'", '(', '▁"', "'", ':', "'", ';', ';', "'", '"', ';', "'", ';', ':', "'", ';', '▁"', "'", ';', '(', "'", ';', '&', "'", ';', '/', "'", "'", ',', "'", "'", '/', "'", ';', '_', '_', "'", ';', '-', "'", "'", '-', "'", ';', "'", '(', "'", ')', "'", "'", '▁(', "'", "'", ')', '▁', "'", '(', "'", '(', '(', "'", ';', "'", '(', '▁', "'", '&', '&', "'", '(', ':', "'", '.', '▁"', "'", '(', '.', "'", '"', '▁(', "'", '.', '.', "'", '."', "'", '.', '-', "'", '.', '/', "'", '.', '_', "'", '.', ';', "'", '.', '&', "'", '.', ',', "'", '.', '(', "'", '.', '▁(', "'", ';', '........', '▁"', '.', '."', '.', '▁"', '.', "'", '.', '/', '.', '_', '.', ';', '"', '(', '.', "'", '(', '"', '."', ';', '"', ':', '.', '▁"', '"', "'", '▁"', ':', '"', ';', '▁"', '"', '"', '▁"', '"', '(', '(', '"', "'", ')', '"', "'", '/', '"', "'", '-', '▁"', "'", '-', '-', "'", '"', '-', '"', '-', '-', '"', "'", ',"', '"', '/', "'", '"', '/', '"', '/', '/', "'", '-', '/', "'", '/', '/', '/', '"', '-', '/', '/', '.', "'", '/', '.', '/', '."', '/', '.', '▁"', '/', '.', '▁', '/', '.', '-', '/', '.', '.', '/', '/', '▁//', '/', '/', '_', '/', '/', '(', "'", '/', '(', '/', '/', '▁(', '/', '/', '▁"', '/', '/', '://', ';', '/', '/', '-', '/', '"', '▁', '/', "'", '▁', '/', '▁', "'", '/', '▁', '/', "'", '_', '/', "'", '▁(', '▁', '/', '▁', '/', '(', '▁', '/', '▁(', "'", '/', '▁(', "'", '-', "'", '(', '▁(', '.', '▁', '-', '/', '▁', '-', "'", '/', '-', '▁', '/', '-', '-', '/', '-', "'", '-', '(', '/', "'", '(', '/', '"', '(', '/', '(', '(', '/', '▁(', '(', '(', '(', '.', '/', '(', '"', '/', '(', ':', '/', '(', '▁(', '/', "'", ')', '/', '(', '/', '▁(', '/', '()', '/', "'", ',', '▁', "'", ',', '▁"', "'", ',', '▁(', "'", ',', '(', "'", ',', '/', "'", ':', '(', "'", ':', '▁(', "'", '(', '▁(', "'", ')', '(', "'", ')', '▁(', '(', "'", '▁(', '(', '"', '()', "'", '()', '"', '(', '.', '.', '(', '()', '(', '(', '(', '(', '*', '(', '(', '%', '(', "'", '")', '(', '(', '▁(', '(', '*', '"', '(', '*', "'", '(', '-', '(', '(', '-', '"', '(', '-', "'", '(', '*', ')', "'", '▁("', "'", '(', '%', "'", "'", ').', "'", "'", '!', "'", "'", '1', "'", "'", '0', "'", "'", '6', "'", "'", '3', "'", "'"], ['▁"', 'Norm', 'al', '_', 'frame', '"', ':', '▁false', '"', 'time', '_', 's', 't', 'amp', '"', ':', '▁', '0', '0:00', ',', '▁"', 'name', '"', ':', '▁"', 'tre', 'e', '▁1', '",', '▁"', 'D', 'i', 'stance', '"', ':', '▁', '8.0', '"', '▁distance', '="', '0', '",', '▁"', 'length', '"', ':', '▁0.', '01', '",', '▁"', 'd', 'i', 'stance', '="', '0', '"', ':', '▁1.', '01', '",', '▁', "'", 'name', "'", ':', '▁', '8.0', '"', 'length', '="', '1', '",', '▁"', 'wid', 'th', '"', ':', '▁3.', '0', '",', '▁', '-', '0', ',', '▁0,', '▁"', 'length', '="', '2', '",', '▁', '-1', ',', '▁', '-1', '",', '▁', '-2', ',', '▁--', '0', ';', '▁', '-1', '▁', '-', '▁', '---', '---', '-', '▁', '-', '▁"', '---', '.', '▁', '-', '_', '-', '-', '_', '_', '_', '-', '▁', '_', '_', '▁', '-', '"', '-', '-', '▁"', '▁', '-', '"', '▁', '-', '▁(', '_', '_', '.', '_', '_', '"', '_', '_', '"', '_', '-', '_', '_', '-', '"', '_', "'", '"', '"', '"', '-', '"', '"', '"', '_', '"', '"', ':', '"', '"', ';', '▁', '-', "'", "'", '"', "'", "'", "'", ':', '▁"', "'", "'", ';', '▁"', '"', ';', '▁"', "'", '"', ';', '"', '"', "'", '"', '(', '▁"', "'", '(', '(', "'", "'", '(', '"', "'", '(', '▁"', '"', "'", '▁"', "'", '_', '_', "'", "'", '▁"', '(', "'", '"', '▁"', '"', '"', '▁"', "'", '▁"', '"', '▁"', '(', '"', '"', '/', '"', '"', '(', '"', '(', '(', '(', '"', '/', "'", "'", '.', "'", "'", '-', "'", '-', '"', "'", '-', '-', "'", '"', '-', "'", '.', '▁"', "'", '-', '▁"', "'", '."', "'", ';', '"', "'", ':', '"', "'", '/', '"', "'", ',', '▁"', "'", ';', "'", "'", ',', "'", "'", '▁', "'", "'", '▁(', "'", "'", ')', "'", "'", '/', "'", ';', ';', '"', ';', ';', "'", ';', '▁', "'", ';', '.', "'", ';', ':', "'", "'", '&', "'", "'", ');', "'", "'", '#', "'", "'", '0', "'", "'", '1', "'", "'", '2', "'", "'", '6', "'", "'", '8', "'", "'", '7', "'", "'", '9', "'", "'", '10', "'", "'", '11', "'", "'", '3', "'", "'", '4', "'", "'", '5', "'", "'", '12', "'", "'", '13', "'", "'", '14', "'", "'", '09', "'", "'", '01', "'", "'", '69', "'", "'", '91', "'", "'", '89', "'", "'", '86', "'", "'", '_', "'", ';', '▁(', "'", ';', '(', "'", '(', "'", ';', '&', "'", ';', '#', '"', "'", '&', '"', "'", '.', ';', '"', '(', "'", '...', "'", '"', '&', '"', '"', '&', "'", '".', "'", '.', '▁', "'", '.', '&', '"', '▁"', '&', '"', '&', '&', '"', ';', "'", '"', '#', "'", '"', '▁', "'", '"', '_', "'", '"', ':', '.', "'", '▁"', ';', '"', '▁"', ';', "'", '▁"', '&', "'", '▁"', '▁"', "'", '&', '&', "'", '&', ';', '"', '▁', '&', '"', '▁', '0', "'", '"', '▁(', "'", '.', '(', '"', '▁"', '.', '.', '▁"', '.', '▁"', '"', '..."', '▁"', '▁"', '."', '".', '▁"', '▁"', '".', '."', '."', ';', '."', '(', '.', '.', '(', '."', '&', '.', '.', '/', '.', '.', '&', '."', '▁', '/', '"', '▁"', '▁', '/', "'", '"', '/', '/', '"', '/', '▁"', '/', '/', '/', '▁"', "'", '/', '/', "'", '/', '▁"', '▁"', '/', '"', '(', '/', '/', '(', '(', '/', '(', "'", '/', '(', '/', "'", '(', '/', '▁"', '(', '(', '▁"', '/', '(', '▁"', '.', '/', '/', '▁(', "'", '/', '▁(', '/', '/', '▁', "'", '/', '▁//', '.', '▁"', '/', '▁"', '.', '▁', '/', '▁"', "'", '/', '.', '▁"', '.', '/', '▁"', '▁(', '.', "'", '/', '.', '/', "'", '.', '/', '"', '▁', ';', '▁', '/', '▁(', '▁', '/', '(', '▁', "'", '(', '▁', '/', '▁("', "'", '(', '▁', "'", ')', "'", '"', '▁', ';', '▁"', '▁', '&', '▁', "'", '-', '▁', "'", '_', '▁', "'", '#', '▁', "'", '&', '▁"', '▁', '0', '▁', "'", '")', "'", '(', '"', "'", "'", '▁("', '▁"', "'", '▁(', '▁"', "'", '")', '▁"', "'", '▁("', '"', '▁"', '/', "'", '▁"', '/', '▁', '0', '"', '▁', ':', '▁', "'", ',', '▁', "'", '0', '"', "'", '▁', ':', '▁"', '"', '(', '▁', '0', '▁"', "'", '▁', '0', '▁(', '▁', '0', '")', '▁', '0', ')', '▁', '0', '(', '▁"', '0', '"', '▁"', '1', '"', '▁"', '2', '"', '▁"', '3', '"', '▁"', '4"', '▁"', '"', '1', '"', '"', '1', "'", '"', '0', '"', '"', '0', "'", '0', '▁"', '0', "'", '1', '"', "'", '1', '▁"', '0', '▁"', '1', "'", '00', '"', '▁("', "'", '0', '(', "'", '0', '▁(', "'", '0', ')', '▁"', "'", '0', ';', '▁"', '0', ')', '"', '▁"', '0', '(', '0', '"', '(', '0', "'", '(0)', "'", '"', '2', "'", '"', '3', "'", '"', '4"', '▁"', '0', '")', '▁"', '"', '2', '"', '"', '3', '"', '"', '4"', '"', '"', '6', '"', '"', '8', '"', '"', '9', '"', '"', '10', '"', '"', '11', '"', '"', '13', '"', '"', '14', '"', '"', '7', '"', '"', '12', '"', '▁"', '9', '"', '▁"', '8', '"'], ['▁"', 'Norm', 'al', '_', 'frame', '"', ':', '▁false', '"', 'B', 'atter', 'y', '_', 'level', '"', ':', '▁10', '"', 'Temp', 'era', 'ture', '▁(', 'cel', 'ciu', 's', ')', '"', ':', '▁7', '"', 'time', '_', 's', 't', 'amp', '"', ':', '▁"', '0', ':', '01', '",', '▁"', 'name', '",', '▁"', 'tre', 'e', '▁1', '",', '▁"', 'D', 'i', 'stance', '"', ':', '▁', '5.0', '▁', "'", 'time', '_', 'level', "'", ':', '▁"', '10', '"', 'normal', '_', 'frame', "'", ':', '▁false', '▁"', 'normal', '_', 'frame', '"', ':', '▁true', '"', 'Default', '_', 'frame', ':', '▁false', '"', '"', 'default', '_', 'frame', '_', 'size', '"', ':', '▁', '5"', '---', '▁--', '▁("', '▁"', '"', '▁', '-', '"', '▁(', '---', '▁"', '»', '"', '"', "'", '▁"', '"', '▁', "'", '"', '/', '"', '",', '▁"', "'", '"', '"', ':', '"', '▁"', '▁"', '-', '"', ';', '"', '"', '."', '_', '-', '-', '"', '"', '-', '"', '-', '▁"', '/', '"', ':', '"', '"', '"', '▁', "'", "'", '"', "'", '"', '(', '"', '/', '▁"', "'", '"', '(', '"', '(', '"', '"', '(', '(', '"', '(', '▁"', '(', '(', '(', '▁"', '(', '(', "'", '(', '(', '")', '"', '(', "'", '"', '▁"', '(', '"', "'", '"', ':', ')', '"', '"', "'", "'", "'", ':', '"', "'", ';', '"', '"', ';', '▁"', "'", "'", '(', "'", "'", ';', "'", "'", ')', "'", "'", '-', "'", "'", '.', "'", "'", '▁"', "'", ';', '▁"', '"', '"', '▁"', "'", '(', '"', '▁"', '"', "'", '(', '▁"', "'", '."', "'", '▁"', '(', "'", '.', '▁"', "'", ':', "'", "'", '/', "'", "'", '&', "'", "'", "'", "'", ',', "'", "'", '▁(', "'", "'", '_', "'", "'", ');', "'", "'", ').', "'", "'", '▁', "'", ';', ';', "'", '"', ';', "'", ';', '"', ';', "'", ';', '▁', "'", '(', ';', "'", '.', "'", '.', '▁', "'", ':', '▁', "'", '(', "'", ')', '"', "'", ':', ':', "'", '"', '▁(', "'", '"', '▁', '&', "'", '"', '&', "'", ';', '(', "'", ';', '&', "'", '.', '&', "'", '(', "'", '(', '&', "'", '&', '"', "'", '.', '(', "'", '&', '&', "'", '▁"', ';', '"', "'", ')', '"', '&', '"', '"', '&', '&', '"', '&', ';', '"', '(', '&', '"', ';', '"', '&', '▁"', '"', '&', '/', "'", '"', '/', '"', "'", '/', '"', '".', "'", '"', '."', '"', '/', "'", '.', '/', '"', '&', '.', "'", '.', ';', '"', ';', '&', '"', '(', '.', '.', "'", ';', '/', '"', '(', ':', '"', '&', '(', '"', '&', '#', "'", "'", '")', '"', "'", '▁', '&', '"', '▁"', '&', '"', '▁', '0', "'", '"', '0', "'", "'", '0', "'", ';', '0', "'", '0', '"', "'", '&', '#', '"', "'", '0', '&', '"', '0', '"', '"', '#', '"', '"', '1', "'", '"', '2', "'", "'", '1', "'", "'", '3', "'", "'", '2', "'", '"', '3', "'", '"', '4"', "'", "'", '4"', '"', '"', '2', '&', '"', '2', '"', '"', '3', '&', '"', '1', '"', '"', '4"', '&', '"', '3', '"', '"', '6', '&', '"', '6', '"', '"', '8', '&', '"', '9', '&', '"', '4"', '4"', '4"', '"', '4"', '2', '"', "'", '4"', '4"', "'", '"', '6', "'", "'", '6', '"', "'", '8', '"', "'", '6', "'", '"', '8', "'", "'", '9', '"', "'", '9', "'", "'", '8', "'", '"', '9', '"', '"', '0', ':', '0', '"', '1', '"', '2', '"', '0', '"', '3', '"', '1', '&', '"', '8', '"', '9', '"', '6', '"', '7', '"', '8', '"', '"', '7', '"', '"', '9', "'", '"', '7', "'", "'", '7', '"', "'", '7', "'", '"', '10', '"', "'", '1', '"', "'", '3', '"', "'", '2', '"', '▁"', '0', '"', '▁"', '1', '"', '▁"', '2', '"', '▁', '4"', '"', '2', '▁"', '2', '▁"', '"', '2', '▁', '0', '"', '▁', ':', '▁', '0', ':', '02', '"', '▁"', '3', '"', '▁', '-', '"', '1', '▁"', '"', '1', '▁', '0', '▁"', '"', '3', '▁02', '"', '123', '2', "'", '4"', ':', '▁"', '13', '"', '3', "'", '4"', '"', '4"', '"', '0', '▁"', '03', '4"', '"', '2', '▁"', '0', "'", '4"', '▁3', '▁"', '"', '4"', '▁', "'", '0', ':', '03', '"', '")', '"', '▁"', '4"', '"', '▁"', '"', '%', '"'], ['▁Na', 'N', ',', '▁"', 'D', 'i', 'stance', '"', ':', '▁', '0.2', '▁"', 'time', '_', 's', 't', 'amp', '"', ':', '▁"', '0', ':', '07', '",', '▁"', 'name', '"', ':', '▁Na', 'N', "'", 's', '▁"', 'tre', 'e', '▁trunk', '",', '▁"', 'N', 'a', 'N', '",', '▁"', 'D', 'if', 'ference', '"', ':', '▁', '0.2', '5"', 'Time', '_', 's', 'amp', '"', '▁', ':', '▁"', '0:00', '",', '▁"', 'Name', '"', ':', '▁N', "'", 's', '▁tree', '▁trunk', ',', '▁"', 'N', "'", "'", 's', '▁trunk', '"', ':', '▁1.', '25', '▁"', 'Time', '_', 'S', 't', 'amp', "'", "'", ':', '▁', '0.7', '5"', 'N', "'", 'd', 'i', 'stance', ':', '▁', '0.5', '"', 'T', 're', 'e', '▁leaves', '.', '▁', '.', '▁', '-', '▁"', '▁--', '...', '-', '▁', '---', '▁"', '-', '-', '"', '▁', "'", '"', '-', '-', '"', '-', '"', '"', '"', '"', '▁', "'", '"', '"', '▁', '-', '▁"', '"', '."', '▁', "'", "'", '."', "'", '"', "'", '"', '"', ':', '▁1', '"', '▁"', "'", '▁"', '"', '"', '▁"', '"', ':', '"', '"', ';', '▁"', '"', 'N', '"', '"', "'", "'", '"', ':', "'", '"', '(', '"', '"', '(', '▁"', '"', '(', '(', '(', "'", "'", '(', '(', '▁"', '(', '(', '▁(', '(', '(', '"', '(', "'", '(', '▁"', "'", '(', '▁("', '()', '"', '(', '.', '."', '(', ':', '▁"', '(', "'", '"', "'", '(', '"', '▁"', "'", "'", "'", '▁"', '(', '"', "'", '▁"', "'", '"', '▁"', '(', '▁"', '▁"', "'", '▁"', '▁"', '"', "'", '...', '▁"', "'", '."', ':', '.', '.', "'", '"', '▁', '0', ':', '01', '"', '▁"', '.', '.', ':', '."', '".', '."', '.', '..."', '...', '........', '................', '................', '_', '.', '.', '/', '.', '.', '-', '.', '.', '---', '.', '.', '!', '.....', '".', '.', '▁(', '....', '(', '.', '▁', "'", '.', '▁"', '.....', '▁"', '...', '................', '▁', '0', "'", '"', '...', '........', '................', '................', '▁', '0.2', '▁', '0', '▁', '0', '"', '▁', '0.5', '▁', '0', '"', '(', '▁', '0', '▁"', '"', '▁', '0.2', '"', '▁', '0.3', '"', '▁(', '▁', '0', '")', '"', '"', '▁("', '"', '/', '"', '"', '-', '▁"', '"', '-', "'", '"', '/', '."', '▁"', '-', '"', '-', '."', '-', '/', '"', '-', '---', '"', '-', '->', '"', '-', '_', '"', '-', '(', '-', '-', "'", '-', '"', "'", '-', '-', '(', '"', '-', '▁', '0-', '"', '▁"', '▁"', '-', '▁"', "'", '-', '▁"', '▁"', '▁"', '.', '▁"', '-', "'", "'", '-', "'", '.', '-', "'", '▁"', '-', '0', '"', '-', '0-0', "'", '-', '0', '▁"', '-', '/', '-', '-', '0', ':', '0', "'", "'", '/', '-', '0', ',', '0', '"', "'", '/', '"', '▁"', '0', '"', '▁"', '1', '"', '▁"', ':', '▁"', '"', '▁"', '/', '"', '▁', '/', '▁"', '"', '/', '/', '"', '/', '▁"', '/', '/', '/', "'", '/', '/', '▁', "'", '/', "'", "'", '.', '/', '/', '▁"', "'", '/', '▁"', '▁"', '/', "'", '.', '▁//', '.', '▁"', '/', '.', '/', "'", '"', '▁(', '/', '/', '(', '(', '/', '(', '"', '/', '(', '/', '/', '▁(', '/', '(', "'", '/', '(', '▁"', '/', '(', '▁', '/', '(', '▁(', '/', '"', '(', '/', "'", '(', '/', '"', '▁(', "'", '/', '▁(', "'", "'", '▁(', "'", '.', "'", "'", ')', "'", "'", ',', '▁"', "'", ';', '▁"', "'", ':', '▁"', "'", '")', '▁"', '"', ';', '"', "'", ':', '"', "'", ';', '"', '"', '&', "'", "'", ';', "'", "'", '&', "'", '"', ';', "'", '"', '&', '"', "'", ',', "'", "'", '#', "'", "'", '!', "'", "'", "'", "'", '%', "'", "'", '")', "'", "'", '!"', "'", "'", '?', "'", "'", '".', "'", "'", '1', "'", "'", '",', "'", "'", 'F', "'", "'", '0', "'", '(', "'", '.', '(', "'", '-', '(', "'", ',"', "'", ')', '"', "'", '▁("', "'", '&', '"', '"', '#', "'", '"', '#', '"', '"', '$', '"', "'", '!"', "'", '▁', "'", '(', '.', "'", '.', '▁(', "'", '"', '$', "'", "'", '$', '"', '"', '0', "'", '▁"', ':', '"', '▁"', '$', '"', '▁"', '&', '"', '▁"', ';', '"', '▁"', '#', '"', '▁"', '%', '"', '")', '"', '"', '!"', '"', '!"', '"', '"', '1', '"', '"', '3', '"', '"', '2', '"', '"', '4"', '"', '"', '6', '"', '"'], ['▁"', 'B', 'atter', 'y', '_', 'level', '"', ':', '▁20', '"', 'Norm', 'al', '_', 'frame', '"', ':', '▁false', '"', 'time', '_', 's', 't', 'amp', '"', ':', '▁"', '0', ':', '05', '"', ':', '▁', '0', ':', '01', '"', 'name', '"', ':', '▁«', 'blu', 'e', '▁car', '",', '▁"', 'D', 'i', 'stance', '"', ':', '▁1', '0.0', '"', 'd', 'i', 'stance', '="', '0', ':', '06', '"', ':', '▁0,', '▁"', 'A', 'l', 'titude', '▁(', 'm', ')', '"', ':', '▁8', '0.0', '▁', 'm', ')', ':', '▁', '0.5', '▁', 'm', '"', ':', '▁10', '▁', 'm', '"', 'normal', '_', 'frame', ':', '▁false', '"', ':', '▁true', '"', 'b', 'atter', 'y', '-', 'level', '"', '▁', '0', '▁"', '"', '"', '▁»', '"', '"', '▁"', '"', "'", '"', '-', '"', '."', ':', '"', '▁"', '»', '▁', "'", '"', '»', '»', '»', '▁"', '▁"', '.', '▁"', '[', '"', '[', '[', '[', ']', '"', '_', '"', ';', '"', '(', '"', ']', '"', '"', '"', '▁', ':', '▁"', '"', '"', ':', '"', '"', '▁"', '(', '"', '"', '(', '▁"', '"', '▁', '-', '"', '"', "'", '"', '(', '"', '(', '(', '(', '")', '"', '"', ';', '"', '")', '▁"', '(', '(', '▁"', '(', '▁(', '(', '(', "'", '(', '()', '"', '(', "'", "'", '(', '"', "'", '(', "'", '"', "'", "'", "'", '"', '"', '-', '"', '(', ':', '"', "'", ':', '▁"', "'", "'", '-', "'", "'", ';', '"', "'", ';', "'", "'", ':', '"', ':', "'", "'", '.', "'", "'", '▁"', '"', "'", '.', '▁"', "'", '"', '▁"', "'", '."', "'", '-', '"', "'", '▁"', "'", ':', '▁', "'", "'", ',', "'", "'", ')', "'", "'", '▁', "'", '"', '-', '-', "'", '"', ';', '▁"', "'", ';', '▁"', '"', ';', "'", '"', ':', ':', '"', ';', ':', '"', '(', ';', '"', '(', '.', "'", '"', '▁(', "'", "'", '/', "'", "'", "'", "'", '&', "'", "'", '?', "'", "'", '!', "'", "'", '0', "'", "'", '1', "'", "'", '#', "'", "'", '2', "'", "'", '3', "'", "'", '6', "'", "'", '8', "'", "'", '7', "'", "'", '9', "'", "'", '10', "'", "'", '11', "'", "'", '12', "'", "'", '13', "'", "'", '14', "'", "'", '09', "'", "'", '00', "'", "'", '01', "'", "'", '91', "'", "'", '93', "'", "'", '90', "'", "'", '89', "'", "'", '88', "'", "'", '86', "'", "'", '84', "'", "'", '69', "'", "'", '72', "'", "'", '64', "'", "'", '71', "'", "'", '61', "'", "'", '62', "'", "'", '82', "'", "'", '81', "'", "'", '87', "'", "'", '79', "'", "'", '68', "'", "'", '59', "'", "'", '66', "'", "'", '19', "'", "'", '70', "'", "'", '73', "'", "'", '76', "'", "'", '67', "'", "'", '77', "'", "'", '78', "'", "'", '83', "'", "'", '75', "'", "'", '63', "'", "'", '74', "'", "'", '58', "'", "'", 'A', "'", "'", '65', "'", '"', '69', "'", '"', '71', "'", '"', '72', "'", '"', '64', "'", '"', '86', "'", '"', '84', "'", '"', '82', "'", '"', '62', "'", '"', '66', "'", '"', '68', "'", '"', '73', "'", '"', '63', "'", '"', '61', "'", '"', '81', "'", '"', '91', "'", '"', '1', "'", '"', '2', "'", '"', '3', "'", '"', '4"', ':', '▁"', '1', "'", '▁"', '1', '"', ':', '.', '▁"', '"', ':', '".', '▁"', '"', '▁"', '"', '()', "'", '")', ':', '▁"', '▁"', '"', '▁("', '"', '1', '"', '"', '2', '"', '"', '3', '"', '"', '4"', '"', '"', '0', '"', '"', '6', '"', '"', '8', '"', '"', '9', '"', '"', '10', '"', '"', '11', '"', '"', '12', '"', '"', '13', '"', '"', '14', '"', '"', '09', '"', '"', '01', '"', '"', '03', '"', '"', '02', '"', '"', '04', '"', '"', '07', '"', '"', '08', '"', '"', '29', '"', '"', '28', '"', '"', '34', '"', '"', '23', '"', '"', '37', '"', '"', '31', '"', '"', '32', '"', '"', '44', '"', '"', '35', '"', '"', '36', '"', '"', '38', '"', '"', '59', '"', '"', '63', '"', '"', '62', '"', '"', '64', '"', '"', '69', '"', '"', '71', '"', '"', '61', '"', '"', '72', '"', '"', '91', '"', '"', '73', '"', '"', '84', '"', '"', '82', '"', '"', '83', '"', '"', '93', '"', '"', '86', '"', '"', '92', '"', '"', '87', '"', '"', '88', '"', '"', '79', '"', '"', '&', '"', '"', '81', '"', '"', '66', '"', '"', '68', '"', '"', '74', '"', '"', '78', '"', '"', '77', '"', '"', '75', '"', '"', '76', '"', '"', '67', '"', '"', '7', '"'], ['▁"', 'Norm', 'al', '_', 'frame', '"', ':', '▁false', '"', 'time', '_', 's', 't', 'amp', '"', ':', '▁"', '0', ':', '05', '",', '▁"', 'name', '"', ':', '▁Na', 'N', ',', '▁"', 'boot', 'h', '",', '▁"', 'D', 'i', 'stance', '"', ':', '▁', '0.5', '▁', "'", 'time', '_', 't', 'amp', "'", ':', '▁"', '00', ':', '05', '"', ':', '▁true', '"', 'Time', '_', 'spa', 'mp', '"', '▁', ':', '▁', '0', ':', '02', ',', '▁"', 'name', "'", ':', '▁Na', 'S', ',', '▁"', 'D', 'if', 'ference', '"', ':', '▁0.', '75', '▁', "'", '"', 'time', '-', 's', 't', 'ack', '"', ':', '▁0,', '0', ':', '03', ',', '▁"', 'Name', '"', ':', '▁N', ',', '▁"', '.', '.', '▁', '...', '_', '_', '_', '.', '."', '▁"', '"', '"', '▁"', '▁', '-', '▁--', '.', '▁', '_', '..."', '-', '-', '"', '▁"', '▁"', '"', "'", '"', '/', '"', '"', ':', '"', '"', '".', '▁"', '-', '-', '"', ':', '"', '!"', "'", '▁"', "'", '".', "'", "'", '▁"', '"', ':', ':', '▁"', '"', '"', '▁"', ':', '▁"', '▁"', '▁"', '"', '▁"', '▁"', '▁"', ':', '"', '▁"', '"', '-', '"', '"', '-', '▁"', '"', "'", '"', '"', "'", "'", '"', '▁"', "'", "'", "'", '▁"', "'", '"', ':', "'", "'", ':', '"', '"', ';', '"', '"', '(', '"', '(', "'", '"', '(', '▁"', "'", '(', "'", "'", '(', '"', "'", '▁"', '(', '"', '"', '/', '"', '(', '(', '"', '/', '(', '(', '(', "'", '(', '(', ':', '"', '(', ':', "'", '"', "'", '(', ':', '▁"', '(', ':', ':', '"', "'", '/', '"', "'", '.', '▁"', "'", '.', "'", "'", ';', '▁"', "'", ':', "'", '."', '▁"', '(', "'", '.', '.', "'", '"', '▁', "'", "'", '.', '/', "'", "'", '-', "'", "'", '_', "'", "'", '/', "'", '"', '/', "'", '.', '▁', "'", '.', '-', "'", '"', '▁(', "'", "'", '▁(', "'", '.', '▁(', "'", '"', '-', "'", '(', '-', "'", '-', '"', "'", '-', '▁"', "'", '-', '-', "'", '/', '/', "'", '-', '(', "'", '-', '/', "'", '/', '(', "'", '/', '-', "'", '.', '(', "'", ')', "'", "'", ')', '▁"', "'", ';', '"', "'", ';', "'", "'", ',', "'", "'", '!', "'", "'", '▁', "'", '(', '.', "'", '.', ';', "'", '"', ';', "'", ';', ';', "'", '.', ')', "'", '"', '&', "'", "'", '?', "'", "'", '&', '"', "'", '&', '▁"', "'", '&', '&', "'", ';', '▁', "'", ';', '(', "'", ';', '/', "'", ';', '▁(', "'", ';', '.', "'", ';', ':', "'", ';', '-', "'", ';', '&', "'", '.', '&', "'", '(', '▁"', '(', '.', '.', '&', '"', '(', '."', '(', '!', '(', '.', '▁"', '(', '(', '.', '/', '(', '.', '.', '▁"', '"', '(', '."', '"', '▁(', '.', '.', '▁"', '."', '.', '."', "'", ':', '.', '.', '/', '.', '.', ':', '."', ';', '.', '.', ';', '."', '.', "'", '▁"', ';', '"', ';', '▁"', '"', ';', ';', '"', '▁"', '.', '▁"', ';', '▁"', '.', '▁', '0', "'", '"', ');', '"', "'", ')', '"', '"', '▁', '0', '"', '"', '&', '"', '▁"', '0', "'", "'", '".', '▁"', '"', '0', "'", '▁"', '1', "'", '".', '▁"', '0', '"', '▁"', '1', '"', '▁"', '2', '"', '▁"', '3', '"', '▁"', '4"', '"', '"', '1', '"', '"', '3', '"', '"', '2', '"', '"', '0', '"', "'", '0', '"', '(', '0', '"', '&', '/', '"', '0', ':', '0', '"', '▁', '&', '"', '"', '#', '"', '"', '$', '"', '&', '&', '"', '&', '▁"', '"', '1', '&', '"', '#', '&', '"', '0', '&', '"', '/', '▁"', '0', '&', "'", '"', '1', "'", "'", '0', "'", '0', '&', '&', '&', '▁"', '0', ';', '"', '&', '#', '"', '&', '▁', '0', '&', '▁"', '1', '&', '&', '#', "'", '"', '2', '&', '"', '2', "'", '"', '3', "'", "'", '1', '&', '##', '"', '2', '▁"', '0', '"', '2', '"', '0', "'", "'", '2', "'", "'", '9', "'", "'", '6', "'", "'", '7', "'", "'", '8', "'", "'", '3', "'", '"', '#', "'", "'", '#', '"', "'", '1', "'", '&', '#', '&', '#', ';', '"', '(', '&', '&', '%', '"', '&', '(', '"', '&', ';', '"', '▁', '%', '"', "'", '▁', '&', "'", '▁"', '&', '"', '▁', '/', '"', '▁"', '&', "'", '&', '▁', '&', '▁"', '&', '▁"', '(', '&', '"', '▁("', '()', '"', '(', '/', '"', '▁', '-', '"', '(', '-', '(', '"', '▁"'], ['▁"', 'Norm', 'al', '_', 'frame', '"', ':', '▁false', '"', 'time', '_', 's', 't', 'amp', '"', ':', '▁"', '0', ':', '01', '",', '▁"', 'name', '"', ':', '▁', "'", 'tre', 'e', '▁1', "'", ',', '▁', "'", 'D', 'i', 'stance', "'", ':', '▁2.0', '▁', "'", 'time', '_', 't', 'amp', "'", ':', '▁"', '0-0', ':', '07', '",', '▁"', "'", 'name', "'", ':', "'", "'", "'", '"', 'tre', 'e', '▁2', '",', '▁', "'", "'", 'd', 'i', 'stance', '"', ':', '▁2.0', '"', "'", "'", ':', '▁', '8.0', '▁"', '"', 'normal', '_', 'frame', "'", ':', '▁false', '"', ':', '▁true', '"', '"', 'Time', '_', 'spa', 'mp', '"', '▁', ':', '▁', '.', '.', '▁(', '▁"', '▁».', '▁', "'", '-', '▁', '-', '."', '▁', '_', '▁', '-', "'", '"', '▁"', '▁', '/', '"', "'", '"', '/', '/', '/', '"', '-', '-', '"', '▁"', "'", "'", '"', '"', '."', '"', '."', '▁"', '/', '"', '"', ':', '"', ':', ':', '▁"', '▁"', '"', ':', "'", '"', ':', '.', '▁"', '-', '"', "'", '"', '"', '"', '▁"', "'", '"', "'", '▁"', '"', '"', '▁', "'", '"', '(', '"', '"', '(', "'", '"', '▁"', '(', '▁"', '(', '(', '(', "'", '(', '(', '"', '(', '(', '▁"', "'", '(', '"', "'", '(', "'", "'", '▁"', '(', "'", '/', '"', "'", '/', "'", "'", '/', '/', "'", '"', '/', '"', '/', '/', '"', '(', '/', '/', '/', '(', '(', '/', "'", '/', '(', "'", '.', '/', "'", '.', "'", '.', '▁"', "'", '.', '.', "'", "'", '."', "'", '.', "'", "'", ';', "'", "'", '-', "'", "'", '_', "'", "'", '▁', "'", '.', '-', "'", '.', '▁', "'", ';', '▁"', "'", ';', '"', "'", ':', '"', "'", ';', '▁', "'", '/', '▁"', "'", '/', ';', "'", '"', ';', '"', ';', '▁"', '"', ';', "'", ';', ';', '"', '"', ';', ';', '▁"', ';', '"', '(', ';', '"', '▁', ';', "'", '.', ';', '"', '▁"', '"', '(', ':', '"', ';', '▁', ';', '▁"', '...', '▁"', '".', '.', '▁"', '.', '▁"', '.', "'", '"', '▁(', "'", "'", '▁(', "'", '.', '▁(', "'", '".', '▁"', ';', '▁"', '(', '"', '▁', '&', '▁"', '"', "'", '▁', '&', '"', "'", '▁("', '"', "'", ')', '▁"', '"', '▁"', '."', '▁"', '▁"', '"', '&', '"', '"', '&', '▁"', "'", '&', '"', '▁"', '&', '"', '&', "'", '"', '&', '&', '"', '(', '▁"', '&', '▁"', '&', "'", "'", '(', '▁"', '"', '/', '▁"', '"', '#', '"', '"', '#', "'", '"', '#', '▁"', "'", '#', "'", "'", '#', '"', "'", '&', '▁"', '#', '"', '#', '&', '"', '##', '"', '&', '#', '"', '▁"', '#', '&', '#', '&', '&', '&', '##', '&', "'", '&', '&', "'", '▁"', '&', '#', "'", '&', "'", '.', '&', '#', ';', '"', '&', '▁', '&', '&', '▁"', '.', '&', '&', '▁', "'", '#', '&', '▁"', '▁"', '&', '&', '/', '"', '&', '/', "'", '&', '.', '&', '▁"', '/', '"', '▁"', '▁', "'", '&', '/', '&', '▁"', '▁', '&', "'", '#', '▁"', '&', '/', '.', '&', '"', '▁', '%', '"', '▁', '0', "'", '"', '▁', '-', '"', '▁', '/', '▁"', '▁', '/', "'", '/', "'", '▁"', '▁"', "'", '▁"', '▁', '0', '"', '▁"', '/', "'", '▁', '0', '▁"', '▁', '%', '▁', "'", '(', '▁', "'", ')', "'", '(', '▁', "'", ':', '&', "'", '▁', '%', "'", '▁', '-', '▁"', '▁', '-', "'", '"', '-', '"', "'", '-', '"', '▁"', '-', '"', '"', '-', "'", '-', '▁"', '"', '-', '▁"', "'", '-', '-', '▁"', '-', '-', "'", '▁"', '-', '▁"', '▁"', '-', "'", '/', '-', '"', '-', '/', "'", '-', '/', '"', '-', '(', '"', '-', '&', '"', '-', '0', '"', '-', '%', '"', '-', '#', '"', '-', '▁', '%', '%', "'", '-', '&', "'", '-', '(', "'", '-', '%', "'", "'", ',', '▁"', "'", '%', '"', "'", ',"', '"', '%', '"', '"', '$', '"', "'", '%', "'", '"', '%', "'", '▁"', "'", '!', "'", "'", ')', '"', '"', '/', '&', '"', '/', '.', '▁"', '/', '.', '▁//', '.', '/', '.', "'", '/', '."', '/', '.', '/', '/', '▁', '/', '(', '▁"', '/', '/', '▁"', '.', '/', '▁', '&', '/', '/', '▁(', "'", '/', '▁', '%', '▁"', '/', '▁"', '▁"', '/', '▁', '/', '.', '.', '/', '"', '!', '▁', "'", '!', '▁', '&', '▁', '%', '&', '▁', '/', '&', '▁', '0', '▁', '&', '#', '▁', '&', '▁#', '▁', '&', '▁$', '▁', '&', '$', '▁', '&', '%', '▁', '&', '.', '▁', '&', ';', '▁', '&', '▁(', '▁', '&', '(', '▁', '&', '!', '▁', '%', '#', '▁', "'", '%', '▁', '%', '(', '▁', '%', '▁(', '▁', '%', '▁=', "'", '=', '▁', '&', '▁=', '▁', '%', '/', '!', '▁"', '!', "'", '!', '&', '!'], ['▁"', 'time', '_', 's', 't', 'amp', '"', ':', '▁"', '0', ':', '04', '",', '▁"', 'name', '"', ':', '▁Na', 'N', ',', '▁"', 'boat', '▁1', '",', '▁"', 'D', 'i', 'stance', '"', ':', '▁', '5.0', '"', '▁time', '_', 's', 'p', 'read', ':', '▁', '0', ':', '05', '",', '▁"', 'boat', '_', 'level', '"', ':', '▁0.', '01', '"', 'd', 'i', 'stance', '_', 's', 'table', '"', ':', '▁3.', '0', '"', 'date', '_', 's', 'a', 'mple', '"', ':', '▁', '5.0', '4"', 'day', '_', 'time', '"', ':', '▁1.', '04', '▁"', 'day', '_', 'd', 'if', 'ference', '"', ':', '▁4.', '04', "'", 'time', '_', 'd', 'i', 'stance', '"', ':', '▁2.0', '4', '▁"', '.', '---', '▁--', '---', '---', '---', '▁', '-', '▁', "'", '-', '-', '"', '▁--', '"', '▁', '-', '."', '-', '..."', '"', '"', "'", '"', '▁"', '"', '▁("', '(', '"', '_', '_', '_', '"', ':', '▁1', '"', "'", '"', '"', '"', ':', '.', '."', '"', '▁', "'", "'", '"', "'", "'", "'", '▁"', "'", '"', '▁"', '"', '"', "'", '▁"', '"', "'", '-', '"', '"', '▁"', "'", "'", '-', "'", '"', '-', '"', '-', '-', "'", "'", '_', '"', '"', '(', '"', '(', "'", "'", '(', '"', "'", '(', '(', "'", '"', '(', '(', '(', '"', '/', '"', '(', '▁"', '(', '()', '"', '()', "'", '"', ':', '"', '"', '&', '"', '"', '1', '"', '2', '"', '3', '"', '0', '"', '"', '0', "'", '"', '1', "'", '"', '2', "'", "'", '0', "'", "'", '1', "'", "'", '2', "'", '"', '0', ':', '0', "'", '0', '"', "'", '0', ':', '1', "'", '0', "'", "'", '3', "'", "'", '4', "'", "'", '6', "'", "'", '8', "'", "'", '9', "'", "'", '10', "'", "'", '11', "'", "'", '12', "'", "'", '13', "'", "'", '14', "'", "'", '09', "'", "'", '7', "'", "'", '00', "'", "'", '01', "'", "'", '5', "'", "'", '.', "'", "'", ';', "'", "'", ':', "'", "'", ',', "'", "'", '#', "'", "'", '&', "'", "'", '▁(', "'", "'", '$', "'", "'", ')', "'", "'", '▁', "'", '"', '$', "'", '"', '&', "'", '"', '#', "'", '"', ';', "'", '"', '(', "'", '(', "'", '#', '"', "'", ';', '"', "'", ')', '"', "'", ':', '"', "'", '.', '▁"', "'", ';', ';', '"', '"', ';', '"', ';', '▁"', '"', ';', ';', '▁"', "'", '(', '▁"', "'", '&', '"', "'", ',', '▁"', "'", ',"', "'", '&', '▁"', "'", ':', '▁"', "'", '.', '▁', "'", '.', '-', "'", '."', '▁"', '▁"', "'", '-', '▁"', "'", '▁"', '(', '"', '"', '#', '"', '"', '$', '"', "'", '$', '"', '"', '-', "'", '(', '-', '"', '(', '-', "'", '-', '(', '-', '(', '(', '-', ')', "'", '(', ':', '"', '(', ':', "'", '"', '▁(', "'", '")', '"', ':', "'", '()', '▁"', "'", ')', '▁"', '"', '(', '")', '"', '"', '▁("', '(', '0', '"', '▁"', '0', '"', '▁', '0', '"', '(', '▁', '0', "'", '")', '▁"', '"', '1', '"', '▁"', '1', '"', '"', '2', '"', '"', '3', "'", '"', '3', '"', '"', '4"', '"', '"', '6', "'", '"', '6', '"', '"', '8', "'", '"', '4"', "'", '"', '4', "'", '"', '9', "'", '"', '10', "'", '"', '11', "'", '"', '12', "'", '"', '13', "'", '"', '14', "'", '"', '7', "'", '"', '5', "'", '"', '8', '"', '"', '9', '"', '"', '10', '"', '"', '11', '"', '"', '12', '"', '"', '13', '"', '"', '14', '"', '"', '7', '"', '"', '4"', '"', '5"', '"', '20', '"', '"', '01', '"', '"', '00', '"', '"', '09', '"', '"', '30', '"', '"', '100', '"', '"', '200', '"', '"', '35', '"', '"', '40', '"', '"', '29', '"', '"', '32', '"', '"', '34', '"', '"', '37', '"', '"', '44', '"', '"', '36', '"', '"', '23', '"', '"', '24', '"', '"', '28', '"', '"', '31', '"', '"', '33', '"', '"', '17', '"', '"', '18', '"', '"', '19', '"', '"', '21', '"', '"', '22', '"', '"', '16', '"', '"', '26', '"', '"', '91', '"', '"', '93', '"', '"', '92', '"', '"', '96', '"', '"', '86', '"', '"', '94', '"', '"', '97', '"', '"', '98', '"', '"', '99', '"', '"', '69', '"', '"', '84', '"', '"', '82', '"', '"', '88', '"', '"', '89', '"', '"', '87', '"', '"', '90', '"', '"', '71', '"', '"', '72', '"', '"', '83', '"', '"', '79', '"', '"', '64', '"', '"', '66', '"', '"', '68', '"', '"', '63', '"', '"'], ['▁"', 'water', 'proof', '▁drone', '?"', ':', '▁false', ',', '▁"', 'weather', '"', ':', '▁"', 'g', 'loom', 'y', '"', '"', 'time', '_', 's', 't', 'amp', '"', ':', '▁', '8.5', '.', 'time', '_', 'stor', 'age', '"', ':', '▁9.', '0', '."', 'd', 'i', 'stance', '"', ':', '▁1', '0.0', '.', '▁"', '0', ':', '11', '"', '."', 'time', '-', 's', 't', 'ack', '"', ':', '▁', '8.0', '.', '.', '▁time', '_', 's', 'tim', '"', ':', '▁7.', '7', '...', '▁', "'", '0', ':', '14', "'", '.', '-', '-', '▁', "'", 'time', '_', 't', 'amp', "'", ':', '▁', '7.0', '.', '.', '▁', '---', '"', '▁"', '"', '▁', '&', '▁"', '-', '-', '"', '▁', "'", '"', '"', '»', '▁', '-', '▁"', '▁»', '"', '-', '.', '▁(', '.', ',', '."', '"', '▁"', '.', '/', '.', '_', '"', '▁("', '_', '.', ':', '▁', '/', '"', ':', '"', "'", '"', '/', '"', ':', '▁"', "'", "'", '▁"', '-', '-', "'", '"', ':', "'", "'", '"', '"', "'", '"', "'", "'", "'", '▁"', '"', '"', '"', '▁"', "'", '"', '(', '"', '"', '(', '▁"', '"', '(', ':', '"', '"', ';', '"', '"', '&', '"', '"', ':', ':', "'", '"', '&', "'", "'", ';', '"', "'", ';', "'", "'", '(', '"', "'", '(', "'", "'", ',', "'", "'", '-', "'", "'", ':', "'", '▁"', "'", "'", '.', "'", "'", '▁', "'", "'", '&', "'", '"', ';', "'", '"', '▁"', '"', "'", ':', '"', "'", '▁"', '(', "'", '.', '▁"', "'", ':', '▁"', '"', '▁"', '▁"', "'", '."', '"', '▁', "'", '"', '-', '"', '"', '-', '▁"', '"', '-', "'", '.', '-', '"', "'", '-', '▁"', "'", '-', '"', '-', '-', '▁"', '-', '"', '▁"', '-', "'", '▁"', '▁"', '-', '▁"', '.', '▁"', '"', ':', '.', '▁"', ':', '"', '▁"', ':', ':', '"', '▁', ':', '▁"', '▁"', ':', '▁"', '.', '.', '▁"', '."', '▁"', '.', "'", '".', '▁"', '▁"', '"', '▁', '-', '▁"', '▁"', '.', '▁', '-', '"', '▁', ';', '▁"', '"', ';', '▁"', "'", ';', '▁"', '▁"', ';', '"', '▁"', '(', '"', '▁"', '#', '"', '▁"', '0', '"', '▁"', '1', '"', '▁"', '!', '▁"', '▁"', '0', "'", '"', '".', '▁"', '"', '#', '"', '"', '#', "'", "'", '#', "'", '"', '#', '▁"', "'", '#', '"', "'", '#', '▁"', '"', '0', "'", "'", '0', '"', "'", '.', ':', '▁"', '#', "'", '▁"', '##', '"', '#', '##', '▁"', '#', '0', '"', '#', '!', '▁"', '#', '!"', '▁"', '&', '"', '#', '&', '"', '▁', '&', '"', '▁"', '▁', '&', '▁"', '"', '&', '▁"', '&', '▁"', '▁"', '&', '&', '"', '&', '▁', '&', '&', '▁"', "'", '&', '"', "'", '&', '▁"', '#', '&', "'", '&', '&', "'", '▁"', '&', "'", '.', '▁', '&', '#', '"', '&', '&', '&', '##', '&', '#', '&', '&', ';', '"', '&', '#', "'", '&', '#', ';', '"', '▁', '0', '"', '&', '!', '&', '"', '0', '"', '"', '1', '"', '"', '2', '"', '"', '3', '"', '"', '4"', '"', '"', '0', '▁"', '"', '1', '▁"', '"', '2', '▁"', '"', '3', '▁"', '"', '4"', '4"', '"', '▁"', '3', '"', '▁"', '2', '"', '▁"', '9', '"', '▁"', '%', '"', '"', '/', '"', '"', '%', '"', '▁"', '/', '"', "'", '/', '"', '&', '/', "'", '"', '/', "'", "'", '/', '▁"', '"', '/', '▁"', "'", '/', '/', '"', '/', '/', "'", '/', "'", '▁"', '/', '/', '/', '▁"', '/', "'", '.', '/', '"', '▁', '/', '"', '#', '/', '"', '▁"', '4"', '▁"', '".', '▁"', '#', '/', '/', '.', '▁"', '/', '.', '▁', '/', '▁"', '▁"', '/', '▁"', '.', '/', '.', '/', '/', '▁', "'", '/', '.', '.', '/', '▁"', '▁', "'", '.', '."', '/', '."', '!', '▁"', '.', '&', '▁"', '.', '#', '"', '▁', '%', '"', '?"', '▁"', '!"', "'", '".', '!', '▁', "'", '.', "'", '▁', '&', '▁', "'", '#', '▁', "'", '(', '▁', "'", ';', '▁', "'", '&', '.', '&', '▁', '/', '."', "'", '▁', '/', '.', "'", '▁"', "'", "'", '/', "'", '▁(', "'", '"', '_', '"', '"', '_', "'", '"', '%', "'", "'", '%', '"', "'", '%', "'", '"', '!', "'", '"', '!"', '"', '"', '!"', '"', '!"', '▁"', '"', '!', '▁"', "'", '▁"', '!"', "'", '!', '▁"', '"', '(', '"', '▁', '4"', '"', "'", '!"', '▁"', "'", '!'], ['▁"', 'B', 'atter', 'y', '_', 'level', '"', ':', '▁10', '"', 'Norm', 'al', '_', 'frame', '"', ':', '▁false', '"', 'time', '_', 's', 't', 'amp', '"', ':', '▁"', '0', ':', '04', '",', '▁"', 'name', '"', '▁', ':', '▁"', 'f', 'ence', '",', '▁"', 'D', 'i', 'stance', '"', ':', '▁3.', '0', '"', 'name', '_', 'of', '_', 'wall', '▁1', '"', ':', '▁', "'", 'f', 'ence', "'", ';', '▁', "'", 'd', 'i', 'stance', "'", ':', '▁', '5.0', '▁', "'", 'time', '_', 't', 'amp', "'", ':', '▁"', '00', ':', '04', ';', '▁"', 'name', ';', '▁"', 'wall', '▁1', '",', '▁', "'", 'D', 'if', 'ference', "'", ':', '▁2.0', '"', '"', 'normal', '_', 'frame', '▁', '-', '▁', '.', '▁', '-', "'", '▁"', '"', '▁»', '"', '."', '"', '▁"', '"', '▁"', "'", '"', '▁"', '▁"', '▁(', '▁"', '-', '"', '▁("', '-', '-', '"', '-', '-', '▁"', '▁', ']', '"', '»', '▁', ';', '.', "'", '"', '/', '"', ';', '▁"', '/', '"', ':', '"', ';', '▁"', '.', '."', ':', "'", '"', ':', '"', '"', ':', '.', '▁"', '.', '/', '/', '▁"', '/', '▁', '&', '"', '...', '▁', '/', '.', '_', '_', '_', '"', "'", '"', '"', '_', '_', "'", "'", '"', "'", "'", "'", '▁"', "'", "'", "'", "'", '.', ';', "'", ';', '"', '(', "'", '(', '"', '_', "'", '.', "'", "'", ':', "'", '-', "'", "'", ';', "'", "'", '_', "'", '"', ';', "'", '"', '(', "'", "'", '(', "'", '"', '▁"', "'", ';', '"', "'", ';', '▁"', "'", '.', '▁"', "'", ':', "'", "'", ')', '▁"', "'", '"', '▁', "'", "'", '▁', "'", '.', '▁', "'", '"', '▁(', "'", "'", '▁(', "'", '.', '.', "'", '.', '-', "'", '.', '_', "'", ';', '-', "'", ';', '_', "'", '-', '-', "'", '"', '-', "'", ':', '"', "'", ':', ':', "'", ';', ';', "'", ';', '&', "'", "'", '-', '"', ';', '"', ';', ';', '"', '"', ';', '&', '"', "'", '.', ';', '"', '(', '"', '"', '(', '(', '"', "'", '(', '(', '(', "'", '(', '"', '()', "'", "'", '&', '"', '"', '"', '&', "'", '"', '&', '"', '&', '▁"', "'", '&', "'", '.', '&', '"', '▁"', '&', '"', '(', '▁"', '"', '&', '&', '"', '▁', '&', '▁"', '"', "'", '&', '▁"', '&', "'", '&', '&', "'", ';', '(', "'", '.', '(', "'", '&', ';', "'", '."', '"', "'", '▁"', '"', '"', '▁"', '(', '"', '&', '/', "'", "'", '/', '"', "'", '/', "'", '"', '/', '"', '"', '/', '/', '"', '(', '&', '"', '/', '(', '"', '/', "'", '/', '(', "'", '/', '/', '(', '(', '.', '.', '/', "'", '.', '/', '/', "'", '(', '.', '/', '.....', '▁"', '.', "'", '/', '.', "'", '".', '.', '&', '.', '.', '(', '.', "'", ';', '.', '.', ';', '.', "'", '&', '/', '.', '&', '&', '.', '&', ';', '.', '&', "'", '▁"', '.', '&', '#', "'", '.', '▁(', '.', '.', '!', "'", '.', ')', "'", '"', '_', '"', "'", '.', "'", '▁"', '(', '."', '(', '.', '&', '▁"', '.', '▁"', '"', ';', '."', ';', '."', "'", '_', '"', '".', '▁"', "'", '/', '▁"', "'", '(', '.', '&', '.', '/', '"', '▁', ';', '"', '▁', '-', '"', '▁', '/', '"', '▁"', '"', '▁"', '."', '▁"', "'", '"', '▁"', "'", '▁"', '▁', '/', "'", '▁', '/', '▁', '/', '▁"', '.', '▁//', '▁', "'", '/', '.', '▁(', '▁', "'", '(', '▁"', "'", '▁', '&', "'", '(', '▁"', "'", '▁(', '.', '(', '▁', "'", ';', '▁(', '▁', '/', '(', '▁', '/', '▁(', '▁(', '▁', '&', '(', '▁', '0', "'", '▁', '0', '"', '▁', '0', '▁', '0', '▁"', '▁', '0', '▁"', '▁"', '▁', '&', "'", '▁', '-', '▁"', '▁', '-', "'", '▁', '%', '▁', "'", '-', '▁', "'", ',', '▁', "'", ')', '▁', "'", ':', '▁(', '▁', '0', '&', '▁', '0', ';', '▁', '0', ':', '▁', '0', '▁=', '▁', '0', '▁(', '▁', '-', '0', "'", "'", '▁=', '▁', "'", '0', "'", '▁=', '▁"', "'", '/', '▁(', "'", '"', '▁=', '▁', ')', '▁', '")', '▁', "'", '")', '▁"', "'", '-', '▁"', "'", '/', "'", '-', '/', "'", '▁"', '/', '"', '/', '▁"', '/', '/', '▁"', '"', '/', '▁', '0', '(', '"', '▁"', '/', '▁"', '▁"', '/', "'", ')', '"', '▁"', '▁"', '"', '▁', '4"', '▁"', '"', '▁("', '"'], ['▁"', 'Norm', 'al', '_', 'frame', '"', ':', '▁false', '"', 'time', '_', 's', 't', 'amp', '"', ':', '▁"', '0', ':', '03', '"', ':', '▁', "'", "'", 'hall', 'way', "'", "'", ';', '▁', "'", 'd', 'i', 'stance', "'", ':', '▁', '8.0', '▁', "'", 'time', '_', 't', 'amp', "'", ':', '▁"', "'", "'", '0', ':', '04', "'", "'", "'", '▁', ':', '▁', '0', ':', '02', "'", "'", ':', '▁false', "'", "'", '"', 'name', '"', '▁', ':', '▁"', 'hall', 'way', '"', "'", "'", '"', 'D', 'i', 'stance', '"', ':', '▁2.0', '"', 'd', 'ist', 'ant', '"', ':', '▁2.0', '"', 'length', '"', ':', '▁3.', '0', '▁"', 'd', 'i', 'stance', '▁', '.', '▁', '-', '▁"', '▁(', '▁', "'", '"', '▁».', '"', '▁"', '".', '▁', '/', '"', '"', "'", "'", '▁"', "'", '"', '/', '"', '▁"', "'", "'", '"', ':', '"', "'", '"', '"', ':', '"', "'", '▁', ';', '"', '_', '"', ':', "'", '"', ';', '▁', ';', '.', '.', '▁"', '"', '."', ':', ':', '▁', '="', '"', ';', '▁"', '"', '▁"', '▁"', '_', '▁', ':', '"', ';', '"', ':', '▁"', "'", '▁"', ';', "'", ';', '"', "'", ':', '"', '"', "'", ';', "'", '"', '▁"', "'", ':', "'", "'", '.', "'", '"', "'", '.', ';', "'", ';', ';', "'", "'", '(', "'", "'", '▁(', "'", "'", ')', "'", "'", '-', "'", "'", '_', '_', "'", "'", '/', "'", "'", '&', "'", "'", ',', "'", "'", '?', "'", "'", '...', "'", "'", '#', "'", "'", '%', "'", "'", '▁=', "'", "'", '=', "'", "'", 'y', "'", "'", '!', "'", "'", '1', "'", "'", '2', "'", "'", '3', "'", "'", '6', "'", "'", '8', "'", "'", '9', "'", "'", '4', "'", "'", '7', "'", "'", '10', "'", "'", '11', "'", "'", '12', "'", "'", '13', "'", "'", '14', "'", "'", '09', "'", "'", '01', "'", "'", '69', "'", "'", '64', "'", "'", '59', "'", "'", '72', "'", "'", '91', "'", "'", '88', "'", "'", '89', "'", "'", '86', "'", "'", '90', "'", "'", '81', "'", "'", '82', "'", "'", '84', "'", "'", '▁8', "'", "'", '▁9', "'", "'", '▁7', "'", "'", '▁6', "'", "'", '▁1', "'", "'", '▁2', "'", "'", '▁3', "'", "'", '▁4', "'", "'", '▁5', "'", "'", '5', "'", "'", '▁10', "'", '"', '1', "'", '"', '2', "'", '"', '3', "'", '"', '4', "'", '"', '0', "'", '"', '6', '"', "'", '0', "'", '0', '"', '0', '"', '1', '"', '2', '"', '3', '"', '2', '▁"', '2', '"', '1', '"', '2', '"', '0', ':', '0', "'", "'", '4"', ':', '▁"', '123', '"', '34', '"', '"', '2', ':', '▁"', '2', ':', '2', '"', '"', '3', ':', '"', '1', ':', '"', '2', ';', '"', '"', '1', ';', '"', '2', '&', '"', '1', '&', '2', '"', '2', '▁', '0', '"', '3', '&', '2', "'", '2', '"', '0', '▁"', '0', "'", '2', '&', '2', '&', '3', "'", '2', '▁"', '0', '"', '"', '0', ';', '"', '0', '&', '"', '00', "'", '1', '"', '00', "'", '4"', '0', '"', "'", '14', '"', '0', "'", '3', '"', '"', '20', '"', '0', '"', '▁"', '0', '&', '0', '"', '▁', '0', "'", '01', '"', '"', '4"', '"', '"', '"', '&', '"', '"', '(', '"', '"', ';', '"', '&', "'", '"', '(', '(', '"', "'", '(', '"', '(', '▁"', '"', '")', '"', '"', '▁"', '"', "'", '▁"', '"', '▁"', '&', '"', '▁"', '▁"', '"', ':', '0', '"', ':', '1', '"', ':', '▁1', '"', ':', '2', "'", '0', '▁"', '"', ';', ':', '"', ';', ';', '"', ';', '&', '"', ';', "'", '&', '"', "'", '&', '&', '"', '&', '&', "'", ';', '&', "'", '&', ';', '"', '▁"', ';', '"', '▁', '&', '"', ':', '&', '"', '▁', "'", ';', '▁"', "'", ';', ':', '&', "'", '▁"', ';', "'", '▁"', '(', "'", '"', '-', '"', "'", '-', '"', '"', '-', "'", '"', '&', '▁"', "'", '-', '▁"', "'", '&', '▁"', '"', '-', '-', '"', '-', '▁"', '"', '&', '-', '"', '▁"', '(', '"', '&', '/', '"', '"', '#', '"', '"', '/', "'", '"', '#', "'", '"', '/', '"', "'", '#', '"', "'", '/', '"', '&', '#', '"', '&', '%', '"', "'", '▁', '&', '&', '&', '#', ';', '"', '#', '&', '#', '&', '"', '##', "'", '&', '#', '##', '&', "'", '##', '"', '#', '▁"', '&', '&', '▁"', '&', '#', "'", '#', '&', '&', '%', '&', '&', ';', "'", '#', ';', "'", '.', '&', "'", '.', '▁"', '&', "'", '(', '&', "'", '-', '&', "'", '/', '&', "'", ':', '&', '&', '.', '&', '&', '▁', '&', "'", '0', '&', '&', '-', "'", '&', '▁', "'", '&', '.', "'", "'"], ['▁"', 'water', 'proof', '▁drone', '?"', ':', '▁false', ',', '▁"', 'weather', '"', ':', '▁"', 'g', 'loom', 'y', '"', '"', 'time', '_', 's', 't', 'amp', '"', '▁', ':', '▁"', '0', ':', '04', '"', '▁', '-', '▁"', 'name', '"', ':', '▁Na', 'N', ',', '▁"', 'D', 'i', 'stance', '"', ':', '▁2.', '75', '▁"', 'time', '_', '_', 's', '"', '▁', '0', ':', '08', '".', '▁"', 'time', '"', ':', '▁', '1.0', '▁', '-', '▁', "'", '"', 'time', "'", '"', ':', '▁1.', '75', '▁"', '"', '▁time', '_', 's', "'", '"', '▁', "'", '0', ':', '13', "'", '▁', '-', '▁', '0.7', '5"', '▁', '/', "'", '-', '.', '▁(', '▁', '/', '▁', ',', '▁(', '▁--', '▁', '-', '▁(', '.', '▁"', '▁', ')', '▁', '-', '▁', '/', '"', '▁', '-', '"', '"', '."', '▁', '&', '"', '▁"', ',', '▁"', '▁', '-', ':', "'", "'", "'", '▁"', '▁"', '"', "'", '"', '"', '"', "'", '"', '▁"', '"', '"', '▁"', '."', '"', ':', '"', '▁"', "'", "'", '"', "'", "'", "'", '▁"', "'", '"', '(', '"', '"', '(', '▁"', '"', '(', '(', '"', '▁"', '(', '"', "'", '(', '"', '(', "'", "'", '(', '(', "'", '(', '▁"', "'", '(', "'", '"', '▁(', "'", "'", '-', "'", "'", ':', '▁"', "'", '▁"', '"', "'", '▁"', '▁"', "'", '.', '▁"', "'", ':', '"', '"', '-', "'", '"', '-', '"', "'", '-', '"', '-', '▁"', '"', '-', '-', '"', '▁"', '-', "'", '-', '▁"', "'", '-', '-', "'", '(', '-', "'", '▁"', '-', '"', '(', '-', '"', ':', "'", "'", '.', "'", "'", ')', ':', '▁"', '"', ':', ':', '"', "'", ':', ':', '▁"', '▁"', '"', '▁"', ':', '▁"', ':', '"', ':', '.', '.', '▁"', ':', ':', '.', '▁"', '"', '▁"', "'", '"', "'", "'", "'", ',', "'", "'", '(', "'", '."', "'", ';', "'", "'", ';', '"', "'", ',', '▁"', "'", ';', '▁"', "'", '!', "'", '"', ';', '▁"', '"', ';', '"', '"', ';', "'", '".', "'", '"', '&', "'", "'", '&', '"', '"', '&', '"', "'", '&', "'", '"', '#', "'", "'", '#', '"', "'", '!', '▁"', "'", '#', "'", '"', '!', '▁"', '"', '#', '"', '"', '#', '▁"', "'", '&', '▁"', '"', '&', '▁"', "'", '?', "'", "'", '?', '▁"', "'", '!"', '▁"', '"', '!"', '▁"', '.', '."', '▁"', '▁"', '.', '▁"', '."', '".', '▁"', '▁"', '▁"', '#', '"', '▁"', '&', '"', '▁"', '#', '▁"', '▁"', '&', '▁"', '....', '▁', "'", '.', '.', "'", '.', '▁', '&', '▁"', "'", '"', '▁', '&', "'", '▁"', '.', "'", '▁"', ';', '"', '▁"', ';', '▁"', '▁"', '.', "'", '▁', '&', '▁', "'", '&', '&', '▁"', '▁"', ';', '▁', "'", '#', '▁"', '"', '!', '▁"', '▁"', '!', '▁"', '▁', '0', '"', '▁"', '!"', '▁', '%', '"', '▁"', '!', '▁', "'", '!', '.', '&', '▁"', '&', "'", '▁', '0', '▁"', '▁"', '▁"', '▁"', '-', '▁"', '▁', '%', '▁"', '▁', '/', '▁"', '▁"', '/', '"', '▁"', '/', '▁"', '"', '/', '"', '"', '/', "'", '"', '/', '▁"', "'", '/', '"', '/', '/', '"', "'", '/', '▁"', '/', "'", "'", '/', '/', "'", '/', "'", '▁"', '/', '/', '▁"', '.', '/', '/', '▁//', '/', '.', '/', '"', '/', '/', '/', '"', '&', '/', '"', '##', '"', '&', '&', '"', '&', '#', '##', '&', "'", '&', '#', '"', '#', '&', '"', '#', '▁', "'", '/', '&', '"', '&', '"', '/', '&', '▁"', '#', '&', '&', '&', '#', '&', '#', '▁"', '&', '&', "'", '##', "'", '&', '▁', '&', '#', "'", '▁"', '&', '#', '▁', '&', '&', '/', "'", '&', '/', '▁"', '&', '/', '&', '&', '▁', '/', '&', '▁', '&', '&', ';', '"', '&', '.', '/', "'", '#', '&', '▁"', '/', '&', "'", '/', '▁', "'", '(', '&', "'", '.', '&', '"', '(', '&', '"', '▁("', '&', '.', '▁"', '&', '.', '.', '&', '&', '."', '&', '!', '&', '"', '!', '&', '▁"', '!"', '&', '!"', '"', '"', '!', "'", "'", '!"', "'", '.', '(', '"', '&', '(', "'", '&', '!', "'", '&', '.', "'", '&', ';', "'", '&', '!"', "'", "'", '!"', '"', "'", '#', '(', '"', '#', '(', "'", '#', '!', "'", '▁"', '(', "'", '▁"', '#', "'", '.', '/', '▁"', '(', '(', '(', '&', '&', '(', '(', '.', '.', '(', '."', '(', '.', "'", '(', '.', '▁"', '(', '.', '(', '(', '!"', '"', '▁(', '.', '.', '!', '▁"', '.', '&', "'", '(', '!', '▁"', '(', '▁"', '(', '&', '▁"', '(', '"', '▁', '")', '"', '"', '!"', '"', '▁"', '0', '"', '"'], ['▁Na', 'N', ',', '▁"', 'D', 'i', 'stance', '"', ':', '▁', '7.0', '▁"', 'time', '_', 's', 't', 'amp', '"', ':', '▁"', '0', ':', '11', '",', '▁"', 'name', '"', ':', '▁Na', 'N', '▁', '-', '▁"', 'd', 'i', 'stance', '")', ':', '▁', '8.0', '▁"', 'time', '_', 'level', '"', ':', '▁40', ',', '▁"', 'Temp', 'era', 'ture', '▁(', 'cel', 'ciu', 's', ')', '"', ':', '▁10', '"', 'temperatur', 'e', '"', ':', '▁20', '"', 'temperatur', 'e', '"', ':', '▁30', '"', 'col', 'd', '_', 'level', ':', '▁40', '"', '"', 'clock', '_', 'level', "'", ':', '▁', '0', ':', '10', '"', '-', '-', '0', ':', '12', '"', '"', 'time', '-', 's', 't', '▁', '---', '▁--', '▁"', '▁', '---', '---', '-', '---', '▁"', '-', '-', '"', '▁', "'", '-', '-', '"', '▁', '-', '▁--', '"', '▁"', '"', '▁("', '▁"', '▁"', '"', "'", '"', '"', ':', '"', '"', '"', "'", '"', "'", '▁"', '"', '"', '-', '"', '"', '(', "'", '"', '(', '"', '(', '(', '"', '")', '"', '"', ';', '"', '"', '▁"', "'", '"', '-', '(', '"', "'", "'", '"', ':', "'", "'", "'", ':', '"', "'", ':', '▁"', "'", "'", '(', '"', '-', "'", "'", ';', '"', "'", ';', "'", '"', ';', "'", "'", ',', "'", "'", '-', '"', "'", '(', "'", "'", '.', "'", "'", ')', "'", "'", '▁"', "'", ':', "'", '"', '▁"', '"', "'", '."', "'", '-', '▁"', "'", '(', '(', '(', "'", '.', '-', "'", '.', '.', "'", '.', '▁"', "'", ';', '▁"', "'", '.', '▁', "'", "'", '▁(', "'", "'", '/', "'", "'", '!', "'", "'", '▁', "'", '"', '▁(', "'", '"', '▁', "'", '(', '▁"', "'", ')', '"', "'", ')', '▁"', '"', '▁"', '(', '"', '▁"', '1', "'", "'", '".', '▁"', '"', ':', ':', '"', '▁"', '0', "'", "'", '")', '▁"', "'", '0', "'", '"', '="', "'", "'", '="', "'", '"', '&', "'", "'", '&', "'", '"', '0', "'", '(', '1', "'", '"', '1', "'", '(', '0', "'", ';', '0', "'", '0', '"', "'", '0', ':', '0', '"', '"', '0', '"', '(', '0', '"', ';', '▁"', '0', ';', '"', '0', ':', '1', "'", '0', '"', '0', ';', '0', '"', '0', "'", "'", '0', '(', "'", '0', ';', "'", '0', '▁"', '0', '"', ':', '0', "'", '1', "'", '▁"', '0', '(', '"', '0', '(', '00', '"', '0', "'", ')', '0', '"', '▁', '0', "'", '2', "'", '"', '2', "'", "'", '1', '"', '"', '1', '"', "'", '2', '"', '"', '2', '"', '1', '"', '3', '"', '2', '"', "'", '3', '"', '"', '3', "'", '"', '4"', '"', '"', '4"', "'", '"', '6', '"', '"', '6', "'", '"', '3', '"', '▁"', '3', '"', '0', '▁"', '1', '"', '0', ')', '"', '1', '▁"', '"', '2', '▁"', '"', '1', '▁', '0', '"', '▁"', '2', '"', '▁', ':', '▁"', '"', '0', '▁', '0', '▁"', '"', '3', '▁"', '0', '▁"', '3', "'", "'", '2', '▁"', '3', '▁"', '32', '"', '▁"', '4"', ':', '▁"', '12', '▁"', '0', '233', '"', '2', "'", '00', '"', '-', '0', '"', '1-0', "'", '3-0', '3', '-', '"', '0-0', '-', '"', '-', '▁"', '0-', "'", '0-', '-', "'", '-', '0', '▁"', '-', '0', '▁', '-', '0', ',', '0', '"', '▁--', '0', '"', '▁(', '-', '0', ')', '"', '(', '-', '0', '(', '(', '-', '-', '(', '-', '"', '()', '"', '(', ':', '"', '(', '▁"', '(', '()', "'", '")', '▁"', '()', '▁"', '0', ')', '▁"', '1', '-', '"', '▁"', '-', '"', '1', '(', '"', '1', '&', '"', '"', '#', '"', '"', '&', '"', '(', '2', '"', '(', '1', '"', '(', '3', '"', '(', '4"', '(', '"', '2', '(', '"', '3', '(', '")', "'", '(', '2', "'", '(', '3', "'", '(', '9', "'", '(', '6', "'", '(', '7', "'", '(', '8', "'", '(', '11', "'", '(', '10', "'", '(', '12', "'", '(', '13', "'", '(', '14', "'", '(', '19', "'", '(', '4', "'", '(', '5', "'", '(', '4"', "'", "'", '3', "'", ')', ':', '"', '1', ';', '"', '(', '6', '"', '(', '7', '"', '(', '8', '"', '(', '9', '"', '(', '11', '"', '(', '10', '"', '(', '12', '"', '(', '13', '"', '(', '14', '"', '(', '19', '"', '(', '4"', '(', '")', '"', '(', '00', '"', '(', '%', '(', '0', '")', '"', "'", '")', '"', '"', '")', '"', '▁"', '9', '"', '"', '10', '"', '"', '11', '"', '"', '13', '"', '"', '12', '"', '"', '14', '"', '"', '17', '"', '"', '18', '"', '"', '19', '"', '"', '20', '"', '"', '15', '"', '"', '16', '"', '"', '7', '"', '"', '8', '"', '"', '9', "'", '"', '10', "'", "'", '10', '"', "'", '10', "'", '"', '12', "'", '"', '11', "'", "'", '12', "'", "'", '11', '"', "'", '11', "'", '"', '14', "'", "'", '09', "'", "'", '9', "'", "'", '8', "'", "'", '6', "'", "'", '7', "'", "'", '14', "'", '"', '9', '"', "'", '9', '"', ':', '1', '"', ':', '2', "'", '9', '"', '9', "'", '09', '"', '▁"', '10', '"'], ['▁"', 'Norm', 'al', '_', 'frame', '"', ':', '▁false', '"', 'time', '_', 's', 't', 'amp', '"', ':', '▁', '0', '0:00', ',', '▁"', 'name', '"', ':', '▁"', 'tre', 'e', '▁1', '",', '▁"', 'D', 'i', 'stance', '"', ':', '▁', '8.0', '"', '▁distance', '="', '0', '",', '▁"', 'length', '"', ':', '▁0.', '01', '",', '▁"', 'd', 'i', 'stance', '="', '0', '"', ':', '▁1.', '01', '",', '▁', "'", 'name', "'", ':', '▁', '8.0', '"', 'length', '="', '1', '",', '▁"', 'wid', 'th', '"', ':', '▁3.', '0', '",', '▁', '-', '0', ',', '▁0,', '▁"', 'length', '="', '2', '",', '▁', '-1', ',', '▁', '-1', '",', '▁', '-2', ',', '▁--', '0', ';', '▁', '-1', '▁', '-', '▁', '---', '---', '-', '▁', '-', '▁"', '---', '.', '▁', '-', '_', '-', '-', '_', '_', '_', '-', '▁', '_', '_', '▁', '-', '"', '-', '-', '▁"', '▁', '-', '"', '▁', '-', '▁(', '_', '_', '.', '_', '_', '"', '_', '_', '"', '_', '-', '_', '_', '-', '"', '_', "'", '"', '"', '"', '-', '"', '"', '"', '_', '"', '"', ':', '"', '"', ';', '▁', '-', "'", "'", '"', "'", "'", "'", ':', '▁"', "'", "'", ';', '▁"', '"', ';', '▁"', "'", '"', ';', '"', '"', "'", '"', '(', '▁"', "'", '(', '(', "'", "'", '(', '"', "'", '(', '▁"', '"', "'", '▁"', "'", '_', '_', "'", "'", '▁"', '(', "'", '"', '▁"', '"', '"', '▁"', "'", '▁"', '"', '▁"', '(', '"', '"', '/', '"', '"', '(', '"', '(', '(', '(', '"', '/', "'", "'", '.', "'", "'", '-', "'", '-', '"', "'", '-', '-', "'", '"', '-', "'", '.', '▁"', "'", '-', '▁"', "'", '."', "'", ';', '"', "'", ':', '"', "'", '/', '"', "'", ',', '▁"', "'", ';', "'", "'", ',', "'", "'", '▁', "'", "'", '▁(', "'", "'", ')', "'", "'", '/', "'", ';', ';', '"', ';', ';', "'", ';', '▁', "'", ';', '.', "'", ';', ':', "'", "'", '&', "'", "'", ');', "'", "'", '#', "'", "'", '0', "'", "'", '1', "'", "'", '2', "'", "'", '6', "'", "'", '8', "'", "'", '7', "'", "'", '9', "'", "'", '10', "'", "'", '11', "'", "'", '3', "'", "'", '4', "'", "'", '5', "'", "'", '12', "'", "'", '13', "'", "'", '14', "'", "'", '09', "'", "'", '01', "'", "'", '69', "'", "'", '91', "'", "'", '89', "'", "'", '86', "'", "'", '_', "'", ';', '▁(', "'", ';', '(', "'", '(', "'", ';', '&', "'", ';', '#', '"', "'", '&', '"', "'", '.', ';', '"', '(', "'", '...', "'", '"', '&', '"', '"', '&', "'", '".', "'", '.', '▁', "'", '.', '&', '"', '▁"', '&', '"', '&', '&', '"', ';', "'", '"', '#', "'", '"', '▁', "'", '"', '_', "'", '"', ':', '.', "'", '▁"', ';', '"', '▁"', ';', "'", '▁"', '&', "'", '▁"', '▁"', "'", '&', '&', "'", '&', ';', '"', '▁', '&', '"', '▁', '0', "'", '"', '▁(', "'", '.', '(', '"', '▁"', '.', '.', '▁"', '.', '▁"', '"', '..."', '▁"', '▁"', '."', '".', '▁"', '▁"', '".', '."', '."', ';', '."', '(', '.', '.', '(', '."', '&', '.', '.', '/', '.', '.', '&', '."', '▁', '/', '"', '▁"', '▁', '/', "'", '"', '/', '/', '"', '/', '▁"', '/', '/', '/', '▁"', "'", '/', '/', "'", '/', '▁"', '▁"', '/', '"', '(', '/', '/', '(', '(', '/', '(', "'", '/', '(', '/', "'", '(', '/', '▁"', '(', '(', '▁"', '/', '(', '▁"', '.', '/', '/', '▁(', "'", '/', '▁(', '/', '/', '▁', "'", '/', '▁//', '.', '▁"', '/', '▁"', '.', '▁', '/', '▁"', "'", '/', '.', '▁"', '.', '/', '▁"', '▁(', '.', "'", '/', '.', '/', "'", '.', '/', '"', '▁', ';', '▁', '/', '▁(', '▁', '/', '(', '▁', "'", '(', '▁', '/', '▁("', "'", '(', '▁', "'", ')', "'", '"', '▁', ';', '▁"', '▁', '&', '▁', "'", '-', '▁', "'", '_', '▁', "'", '#', '▁', "'", '&', '▁"', '▁', '0', '▁', "'", '")', "'", '(', '"', "'", "'", '▁("', '▁"', "'", '▁(', '▁"', "'", '")', '▁"', "'", '▁("', '"', '▁"', '/', "'", '▁"', '/', '▁', '0', '"', '▁', ':', '▁', "'", ',', '▁', "'", '0', '"', "'", '▁', ':', '▁"', '"', '(', '▁', '0', '▁"', "'", '▁', '0', '▁(', '▁', '0', '")', '▁', '0', ')', '▁', '0', '(', '▁"', '0', '"', '▁"', '1', '"', '▁"', '2', '"', '▁"', '3', '"', '▁"', '4"', '▁"', '"', '1', '"', '"', '1', "'", '"', '0', '"', '"', '0', "'", '0', '▁"', '0', "'", '1', '"', "'", '1', '▁"', '0', '▁"', '1', "'", '00', '"', '▁("', "'", '0', '(', "'", '0', '▁(', "'", '0', ')', '▁"', "'", '0', ';', '▁"', '0', ')', '"', '▁"', '0', '(', '0', '"', '(', '0', "'", '(0)', "'", '"', '2', "'", '"', '3', "'", '"', '4"', '▁"', '0', '")', '▁"', '"', '2', '"', '"', '3', '"', '"', '4"', '"', '"', '6', '"', '"', '8', '"', '"', '9', '"', '"', '10', '"', '"', '11', '"', '"', '13', '"', '"', '14', '"', '"', '7', '"', '"', '12', '"', '▁"', '9', '"', '▁"', '8', '"'], ['▁"', '0', ':', '01', '",', '▁"', 'name', '"', ':', '▁"', 'wall', '▁in', '▁castle', '▁2', '",', '▁"', 'D', 'i', 'stance', '"', ':', '▁1.5', '"', 'time', '_', 's', 't', 'amp', '"', ':', '▁', '0', ':', '03', '"', 'normal', '_', 'frame', '"', ':', '▁false', '"', 'standard', '_', 'frame', '1', '"', ':', '▁true', '"', 'default', '_', 'frame', '2', '"', ':', '▁no', '"', 'n', 'o', '"', 'd', 'i', 'stance', '_', 'level', '"', ':', '▁zero', '"', 'num', 'ber', '"', ':', '▁0.', '01', '"', 'name', '_', 'name', ':', '▁', '0.5', '"', 'wall', '▁in', '▁palace', '▁1', '",', '▁"', 'wall', '"', ':', '▁1', '"', '"', 'd', 'if', 'ference', '_', 'level', ':', '...', '▁"', '▁"', '▁»', "'", '/', '▁', '-', '▁(', '."', '▁', '-', '."', '.', '-', '"', '▁', '-', '"', '"', '-', '"', '"', "'", '"', '/', '"', '"', ':', '"', '"', '"', "'", "'", '"', '"', ';', '"', "'", '"', "'", '"', '▁"', '"', ':', '.', '."', '"', '▁', "'", '"', '▁"', 'wall', "'", '"', ';', '▁"', '"', '"', '▁"', "'", '"', '▁', '&', '"', '"', '(', '"', '"', '&', "'", '"', '(', "'", '"', '&', '"', '(', '(', '"', '(', '▁"', '(', '(', '(', '...', "'", '"', ':', ':', '"', '(', ':', '"', '&', '▁"', '(', '"', '&', '/', '"', '&', '&', "'", "'", '(', '(', "'", "'", "'", ';', '"', "'", '(', '"', "'", ';', "'", "'", '▁"', "'", "'", '.', "'", "'", ':', '"', "'", ':', "'", "'", '&', '"', "'", '."', "'", '▁"', '"', "'", '&', "'", '.', '▁"', "'", '.', '.', '▁"', '.', '.', '&', '"', ';', '"', '".', '".', '▁"', '".', '.', '!', '...', '................', '................', '................', '................', '................', '................', '................', '................', '................', '................', '................', '_', '.', '.', '---', '.', '.', '-', '.', '.', '..."', '........', '................', '................', '................', '................', '---', '---', '---', '.', '......', '---', '---', '......', '................', '................', '................', '................', '................', '................', '................', '................', '................', '................', '................', '................', '................', '................', '▁', "'", '......', '................', '▁', "'", '....', "'", '.', '................', "'", '.', '▁', "'", "'", '...', "'", '...', '▁"', "'", '......', "'", '"', '...', "'", "'", "'", '..."', "'", '..."', '..."', '...', '▁"', '..."', '"', '......', '▁"', '.', '..."', '▁"', '......', '"', '...', '▁"', '"', '......', '".', '...', '▁"', '"', '_', '"', '"', '-', '-', '"', "'", '-', '"', '▁"', '."', '▁"', '-', '"', '-', "'", "'", '-', "'", '"', '-', '▁"', '"', '-', '...', "'", '-', '-', "'", '.', '-', "'", '-', '▁"', "'", '-', '..."', '-', '.', "'", '-', '."', '-', '---', '"', '-', '_', '"', '-', ':', '"', '-', '&', '"', '-', '"', '&', '-', '"', '...', '&', '"', '&', '.', '.', ':', '"', '▁"', '&', '&', '"', '▁"', '▁"', '&', '"', '▁', '0', '"', '▁"', '0', '"', '"', '1', '"', '"', '2', '"', '"', '3', '"', '"', '0', '"', '1', '"', '1', '&', '"', '2', '"', '0', '&', '"', '0', '"', '2', '&', '"', '3', '"', '3', '&', '"', '1', "'", '"', '0', "'", '"', '2', "'", '"', '3', "'", "'", '1', "'", "'", '2', "'", "'", '0', "'", "'", '3', "'", '"', '1', '▁"', '"', '2', '▁"', '"', '1', '-', '"', '2-', '"', '0', '▁"', '0', "'", '0', '"', '-', '0', '"', '&', '#', '"', '"', '6', '"', '&', '2', '"', '&', '1', '"', '&', '3', '"', '&', '4"', '&', '"', '4"', '&', '&', '#', '&', '"', '6', '&', '&', '&', '▁"', '2', '&', '&', '2', '&', '##', '&', '&', '%', '&', '&', '!', '&', '&', ';', '"', '&', '%', '"', '&', ';', '&', '"', '#', '&', "'", '&', '&', '!"', '&', '"', ':', '&', '"', '%', '&', "'", '▁"', '&', "'", '(', '&', '"', '!', '&', '"', '?', '&', '"', '&', '"', '$', '"', '&', '▁', '&', '&', '.', '&', '&', '/', '&', '"', '+', '"', '&', '(', '&', '&', '(', "'", '&', '▁"', '&', '▁"', '"', '&', '!', '▁"', '&', '▁', "'", '&', '#', "'", '"', '#', "'", "'", '#', '"', "'", '#', '&', '#', ';', '&', '&', '-', '&', '&', '+', '&', '&', ':', '&', "'", '##', "'", '&', ';', "'", '&', '%', "'", "'", '▁', '&', "'", ';', '&', "'", '-', '&', "'", '▁', "'", ';', ';', "'", '"', '$', "'", "'", '!', "'", "'", '?', "'", "'", ',', "'", "'"], ['▁"', 'Norm', 'al', '_', 'frame', '"', ':', '▁false', '"', 'time', '_', 's', 't', 'amp', '"', ':', '▁', '0', '0:00', '"', 'tre', 'e', '▁1', '"', ':', '▁"', 'tre', 'e', '▁', '0', '",', '▁"', 'd', 'i', 'stance', '"', ':', '▁', '5"', '"', 'name', '"', ':', '▁tree', '▁1', '",', '▁"', 'D', 'i', 'stance', '2', '"', ':', '▁3', '"', '▁distance', '"', ':', '▁1', '"', 'd', 'i', 'stance', '"', ':', '▁2', '"', 'length', '"', ':', '▁', '4"', 'wid', 'th', '"', ':', '▁6', '"', 's', 'e', 'rial', '_', 'time', '"', ':', '▁true', '"', 'inter', 'val', '"', ':', '▁0.', '01', '"', 'num', 'ber', '"', ':', '▁1.', '...', '▁"', '▁--', '▁(', '▁»', "'", '[', '[', '[', '"', '▁', ']', '"', '▁"', '"', '▁', '-', '▁', '-', '"', "'", '"', '_', '_', '_', '"', ':', '"', '"', '/', '"', '_', '_', '_', '"', ':', "'", "'", '"', ':', '"', '"', '"', ':', '="', "'", '"', '"', "'", "'", "'", ':', '▁"', "'", "'", '▁"', "'", '"', '▁"', '"', '"', ';', '▁"', '"', ':', '▁$', '▁"', '"', ';', '▁', "'", "'", ';', '▁"', "'", "'", '_', '_', "'", '"', "'", '"', ';', "'", '"', '(', '"', '"', '(', '(', '"', '(', "'", "'", '(', '(', '(', "'", '(', '"', "'", '(', '▁"', "'", '(', "'", '/', '"', "'", '/', "'", "'", '-', '"', "'", '-', "'", "'", '/', '/', "'", '-', '-', "'", '-', '/', "'", '"', '-', "'", '"', '&', "'", "'", '.', "'", "'", ',', "'", "'", '▁(', "'", "'", '&', "'", ';', '"', "'", ';', "'", "'", ')', "'", "'", '?', "'", "'", '▁', "'", ';', ';', "'", ';', '.', "'", '"', '▁', "'", '"', '▁(', '▁', "'", '(', '▁', "'", '/', '(', "'", '"', '/', "'", '(', ':', "'", '".', "'", '.', '▁"', "'", '.', '▁', "'", ':', '▁', "'", '."', "'", '.', '.', "'", ';', '.', "'", ':', '.', "'", '(', '...', '▁"', '.', '.', '_', '.', '."', "'", '_', "'", '.', "'", '.', '_', "'", "'", "'", '_', '"', "'", '"', '(', "'", '(', '"', '_', "'", '_', '(', "'", '.', '-', "'", '.', '▁(', "'", '.', '(', "'", ';', '"', ';', "'", ';', '_', "'", ';', '&', '"', "'", ':', '"', "'", '▁"', '"', "'", ')', '"', "'", '&', '"', '"', '&', '"', '&', '&', '"', '▁"', '&', '"', ';', '"', '"', '▁"', "'", ';', '(', '"', ';', ';', '"', ';', '&', "'", '.', ';', '"', '&', ';', '"', '(', '▁"', '"', '&', '/', '"', '"', '▁', '&', "'", '"', '▁', '="', "'", '▁', '&', '"', '▁', '/', '"', '(', '▁', '/', '(', '"', '/', '/', '"', '/', '(', '(', '/', '/', '/', '(', '/', '(', '.', '/', '/', '.', '/', '.', '▁', '/', "'", '.', '/', "'", '/', '▁"', '/', '/', '▁//', '▁"', '.', '/', '".', '/', '.', '▁"', '/', "'", '▁', '/', '▁', "'", '.', '▁(', '.', '▁', '&', "'", '"', "'", '(', '▁"', "'", '/', '▁(', "'", '/', '▁', '&', '▁"', '▁', '/', '▁(', '▁', '/', '▁"', '▁"', '▁', '&', '▁(', '▁', '&', '(', '▁', '&', '/', "'", '▁"', '▁"', '▁"', "'", '▁"', '/', '▁', '0', '▁', '&', '.', '▁', '0', "'", '▁', '0', '"', '▁', '0', '▁"', '▁', '0', '▁(', '▁', '0', '(', '▁', '0', '")', '▁', '0', ')', '▁', '0', '▁=', '▁', '0', ';', '▁', '0', '"', '▁"', '▁', '-', "'", '▁', '-', '▁"', "'", '▁', '%', '▁', "'", '-', '▁', "'", ')', "'", '=', '▁', "'", '_', "'", '-', '-', '▁', "'", ',', '▁', "'", '#', '▁', "'", '!', '▁', "'", '0', "'", "'", '▁=', '▁"', "'", '0', '▁', "'", '1', "'", "'", '▁--', '▁"', "'", '#', "'", "'", '#', '"', "'", '#', '▁"', '"', '#', "'", '"', '#', '"', '"', '##', '"', '▁"', '0', "'", '"', '$', '"', '"', '$', "'", "'", '$', '"', '$', '$', '"', '&', '▁"', "'", '$', '$', "'", '"', '0', "'", '0', '"', '▁"', '$', '$', '$', '0', '"', "'", '$', "'", '▁"', '$', '"', '▁"', '#', '"', '$', '&', '"', '#', '$', '"', '#', '&', '"', '$', '#', '"', '&', '#', "'", '▁"', '&', '▁"', '&', '&', '#', '&', '&', '&', "'", '&', '▁"', '"', '▁"', '▁"', '&', "'", '▁"', '.', '▁"', '&', '▁', '&', '#', '"', '#', '▁"', "'", '&', '##', '&', '#', ';', '"', '▁"', '(', '"']]

len(pred[0])

721

In [81]:
refer=[[['▁Risk', '▁of', '▁physical', '▁damage', '!', '▁The', '▁drone', '▁has', '▁', 'a', '▁damaged', '▁frame', '▁and', '▁low', '▁battery', '.', '▁It', '’', 's', '▁flying', '▁at', '▁an', '▁al', 'titude', '▁of', '.', '5', 'm', '.', '▁[', 'S', 'EP', ']', '▁(0', ':', '01', ')', '▁Risk', '▁of', '▁physical', '▁damage', '!', '▁There', "'", 's', '▁are', '▁2', '▁ships', '▁only', '▁2', 'm', '▁away', '.', '▁[', 'S', 'EP', ']'], ['▁Risk', '▁of', '▁physical', '▁damage', '!', '▁The', '▁frame', '▁is', '▁in', '▁need', '▁of', '▁repair', '▁and', '▁the', '▁battery', '▁is', '▁empty', '.', '▁The', '▁drone', '▁is', '▁flying', '▁at', '▁', 'a', '▁3', 'm', '▁al', 'titude', '▁in', '▁strong', '▁winds', '.', '▁[', 'S', 'EP', ']', '▁(', '0:00', ')', '▁Risk', '▁of', '▁physical', '▁damage', '!', '▁The', '▁drone', '▁is', '▁flying', '▁toward', '▁cargo', '▁which', '▁is', '▁5', 'm', '▁ahead', '.', '▁[', 'S', 'EP', ']', '▁(0', ':', '03', ')', '▁Risk', '▁of', '▁physical', '▁damage', '!', '▁Now', '▁the', '▁cargo', '▁in', '▁the', '▁flight', '▁path', '▁is', '▁1', 'm', '▁from', '▁the', '▁drone', '.', '▁There', '▁is', '▁also', '▁', 'a', '▁store', 'house', '▁2', 'm', '▁from', '▁the', '▁drone', '.', '▁[', 'S', 'EP', ']', '▁(0', ':', '04', ')', '▁Risk', '▁of', '▁physical', '▁damage', '!', '▁The', '▁cargo', '▁in', '▁the', '▁drone', "'", 's', '▁path', '▁is', '▁only', '.', '5', 'm', '▁away', '.', '▁[', 'S', 'EP', ']'], ['▁Risk', '▁of', '▁physical', '▁and', '▁internal', '▁damage', '!', '▁The', '▁drone', '▁has', '▁', 'a', '▁damaged', '▁frame', '▁and', '▁low', '▁battery', '.', '▁It', "'", 's', '▁flying', '▁indoor', 's', '▁and', '▁back', 'ward', 's', '.', '▁It', "'", 's', '▁flying', '▁at', '▁an', '▁al', 'titude', '▁of', '▁1', 'm', '▁in', '▁', 'a', '▁low', '-', 'vis', 'ibility', '▁setting', '.', '▁The', '▁weather', '▁is', '▁over', 'cast', '▁and', '▁the', '▁drone', '▁is', '▁not', '▁waterproof', '.', '▁[', 'S', 'EP', ']', '▁(0', ':', '02', ')', '▁Risk', '▁of', '▁physical', '▁damage', '!', '▁There', "'", 's', '▁', 'a', '▁moving', '▁to', 'y', '▁car', '▁in', '▁the', '▁drone', "'", 's', '▁flight', '▁path', '▁at', '▁', 'a', '▁distance', '▁of', '.', '5', 'm', '.', '▁There', "'", 's', '▁also', '▁', 'a', '▁package', '▁on', '▁the', '▁shelf', '▁only', '.', '5', 'm', '▁away', '.', '▁[', 'S', 'EP', ']', '▁(0', ':', '03', ')', '▁Risk', '▁of', '▁physical', '▁damage', '!', '▁There', '’', 's', '▁another', '▁moving', '▁to', 'y', '▁car', '▁in', '▁the', '▁drone', "'", 's', '▁flight', '▁path', '▁at', '▁', 'a', '▁distance', '▁of', '▁1', 'm', '.', '▁[', 'S', 'EP', ']', '▁(0', ':', '09', ')', '▁Risk', '▁of', '▁physical', '▁damage', '!', '▁Now', '▁there', "'", 's', '▁', 'a', '▁exit', '▁door', '▁in', '▁the', '▁drone', "'", 's', '▁flight', '▁path', '▁at', '▁', 'a', '▁distance', '▁of', '▁5', 'm', '.', '▁There', "'", 's', '▁also', '▁an', '▁orange', '▁shelf', '▁only', '▁2', 'm', '▁away', '.', '▁[', 'S', 'EP', ']'], ['▁Risk', '▁of', '▁physical', '▁damage', '!', '▁The', '▁drone', '▁has', '▁an', '▁empty', '▁battery', '.', '▁It', '’', 's', '▁flying', '▁in', '▁strong', '▁wind', '▁at', '▁an', '▁al', 'titude', '▁of', '▁1', 'm', '.', '▁[', 'S', 'EP', ']', '▁(0', ':', '01', ')', '▁Risk', '▁of', '▁physical', '▁damage', '!', '▁There', "'", 's', '▁', 'a', '▁water', '▁in', '▁the', '▁drone', "'", 's', '▁flight', '▁path', '▁at', '▁', 'a', '▁distance', '▁of', '▁1.5', 'm', '.', '▁[', 'S', 'EP', ']', '▁(0', ':', '02', ')', '▁Risk', '▁of', '▁physical', '▁damage', '!', '▁Now', '▁the', '▁water', '▁is', '▁only', '.', '5', 'm', '▁away', '.', '▁[', 'S', 'EP', ']'], ['▁Risk', '▁of', '▁physical', '▁damage', '▁and', '▁lost', '▁connection', '!', '▁The', '▁drone', '▁has', '▁', 'a', '▁damaged', '▁frame', '▁and', '▁low', '▁battery', '.', '▁It', '’', 's', '▁flying', '▁at', '▁an', '▁al', 'titude', '▁of', '▁20', 'm', '.', '▁The', '▁drone', '▁is', '▁48', '05', 'm', '▁away', '▁from', '▁the', '▁remote', '▁control', '▁and', '▁nearly', '▁out', '▁of', '▁range', '.', '▁[', 'S', 'EP', ']', '▁(', '0:00', ')', '▁Risk', '▁of', '▁physical', '▁damage', '!', '▁There', "'", 's', '▁', 'a', '▁building', '▁only', '▁1.5', 'm', '▁away', '.', '▁[', 'S', 'EP', ']'], ['▁Risk', '▁of', '▁physical', '▁damage', '!', '▁The', '▁drone', "'", 's', '▁frame', '▁needs', '▁', 'repairing', '▁and', '▁the', '▁battery', '▁low', '.', '▁It', "'", 's', '▁flying', '▁at', '▁', 'a', '▁2', 'm', '▁al', 'titude', '.', '▁[', 'S', 'EP', ']', '▁(', '0:00', ')', '▁Risk', '▁of', '▁physical', '▁damage', '!', '▁There', '▁are', '▁stairs', '▁in', '▁5', 'm', '▁and', '▁', 'a', '▁lamp', '▁in', '▁6', 'm', '▁in', '▁the', '▁drone', "'", 's', '▁flight', '▁path', '.', '▁[', 'S', 'EP', ']', '▁(0', ':', '04', ')', '▁Risk', '▁of', '▁physical', '▁damage', '!', '▁Now', '▁the', '▁drone', '▁is', '▁flying', '▁toward', '▁', 'a', '▁hallway', '▁in', '▁its', '▁path', '▁3', 'm', '▁away', '.', '▁[', 'S', 'EP', ']'], ['▁Risk', '▁of', '▁physical', '▁damage', '!', '▁The', '▁drone', '▁has', '▁low', '▁battery', '.', '▁It', '’', 's', '▁flying', '▁at', '▁an', '▁al', 'titude', '▁of', '▁80', 'm', '▁in', '▁', 'a', '▁very', '▁low', '-', 'temperatur', 'e', '▁setting', '.', '▁[', 'S', 'EP', ']', '▁(', '0:00', ')', '▁Risk', '▁of', '▁physical', '▁damage', '!', '▁There', "'", 's', '▁', 'a', '▁moving', '▁bird', '▁only', '▁', '0.5', 'm', '▁away', '.', '▁There', "'", 's', '▁another', '▁moving', '▁bird', '▁only', '▁1.5', 'm', '▁away', '.', '▁There', "'", 's', '▁another', '▁moving', '▁bird', '▁only', '▁2', 'm', '▁away', '.', '▁[', 'S', 'EP', ']', '▁(0', ':', '01', ')', '▁Risk', '▁of', '▁physical', '▁damage', '!', '▁There', "'", 's', '▁', 'a', '▁moving', '▁bird', '▁only', '▁', '0.5', 'm', '▁away', '.', '▁There', "'", 's', '▁another', '▁moving', '▁bird', '▁only', '▁1.5', 'm', '▁away', '.', '▁There', "'", 's', '▁another', '▁moving', '▁bird', '▁only', '▁2', 'm', '▁away', '.', '▁[', 'S', 'EP', ']', '▁(0', ':', '02', ')', '▁Risk', '▁of', '▁physical', '▁damage', '!', '▁There', "'", 's', '▁', 'a', '▁moving', '▁bird', '▁only', '▁', '0.5', 'm', '▁away', '.', '▁There', "'", 's', '▁another', '▁moving', '▁bird', '▁only', '▁1.5', 'm', '▁away', '.', '▁There', "'", 's', '▁another', '▁moving', '▁bird', '▁only', '▁2', 'm', '▁away', '.', '▁[', 'S', 'EP', ']', '▁(0', ':', '03', ')', '▁Risk', '▁of', '▁physical', '▁damage', '!', '▁There', "'", 's', '▁', 'a', '▁moving', '▁bird', '▁only', '▁', '0.5', 'm', '▁away', '.', '▁There', "'", 's', '▁another', '▁moving', '▁bird', '▁only', '▁1.5', 'm', '▁away', '.', '▁There', "'", 's', '▁another', '▁moving', '▁bird', '▁only', '▁2', 'm', '▁away', '.', '▁[', 'S', 'EP', ']', '▁(0', ':', '04', ')', '▁Risk', '▁of', '▁physical', '▁damage', '!', '▁There', "'", 's', '▁', 'a', '▁moving', '▁bird', '▁only', '▁', '0.5', 'm', '▁away', '.', '▁There', "'", 's', '▁another', '▁moving', '▁bird', '▁only', '▁1.5', 'm', '▁away', '.', '▁There', "'", 's', '▁another', '▁moving', '▁bird', '▁only', '▁2', 'm', '▁away', '.', '▁[', 'S', 'EP', ']', '▁(0', ':', '05', ')', '▁Risk', '▁of', '▁physical', '▁damage', '!', '▁There', "'", 's', '▁', 'a', '▁moving', '▁bird', '▁only', '▁', '0.5', 'm', '▁away', '.', '▁There', "'", 's', '▁another', '▁moving', '▁bird', '▁only', '▁1.5', 'm', '▁away', '.', '▁There', "'", 's', '▁another', '▁moving', '▁bird', '▁only', '▁2', 'm', '▁away', '.', '▁[', 'S', 'EP', ']', '▁(0', ':', '06', ')', '▁Risk', '▁of', '▁physical', '▁damage', '!', '▁There', "'", 's', '▁', 'a', '▁moving', '▁bird', '▁only', '▁', '0.5', 'm', '▁away', '.', '▁There', "'", 's', '▁another', '▁moving', '▁bird', '▁only', '▁1.5', 'm', '▁away', '.', '▁There', "'", 's', '▁another', '▁moving', '▁bird', '▁only', '▁2', 'm', '▁away', '.', '▁[', 'S', 'EP', ']', '▁(0', ':', '07', ')', '▁Risk', '▁of', '▁physical', '▁damage', '!', '▁There', "'", 's', '▁', 'a', '▁moving', '▁bird', '▁only', '▁', '0.5', 'm', '▁away', '.', '▁There', "'", 's', '▁another', '▁moving', '▁bird', '▁only', '▁1.5', 'm', '▁away', '.', '▁There', "'", 's', '▁another', '▁moving', '▁bird', '▁only', '▁2', 'm', '▁away', '.', '▁[', 'S', 'EP', ']', '▁(0', ':', '08', ')', '▁Risk', '▁of', '▁physical', '▁damage', '!', '▁There', "'", 's', '▁', 'a', '▁moving', '▁bird', '▁only', '▁', '0.5', 'm', '▁away', '.', '▁There', "'", 's', '▁another', '▁moving', '▁bird', '▁only', '▁1.5', 'm', '▁away', '.', '▁There', "'", 's', '▁another', '▁moving', '▁bird', '▁only', '▁2', 'm', '▁away', '.', '▁[', 'S', 'EP', ']', '▁(0', ':', '09', ')', '▁Risk', '▁of', '▁physical', '▁damage', '!', '▁There', "'", 's', '▁', 'a', '▁moving', '▁bird', '▁only', '▁', '0.5', 'm', '▁away', '.', '▁There', "'", 's', '▁another', '▁moving', '▁bird', '▁only', '▁1.5', 'm', '▁away', '.', '▁There', "'", 's', '▁another', '▁moving', '▁bird', '▁only', '▁2', 'm', '▁away', '.', '▁[', 'S', 'EP', ']', '▁(0', ':', '10)', '▁Risk', '▁of', '▁physical', '▁damage', '!', '▁There', "'", 's', '▁', 'a', '▁moving', '▁bird', '▁only', '▁', '0.5', 'm', '▁away', '.', '▁There', "'", 's', '▁another', '▁moving', '▁bird', '▁only', '▁1.5', 'm', '▁away', '.', '▁There', "'", 's', '▁another', '▁moving', '▁bird', '▁only', '▁2', 'm', '▁away', '.', '▁[', 'S', 'EP', ']', '▁(0', ':', '11', ')', '▁Risk', '▁of', '▁physical', '▁damage', '!', '▁There', "'", 's', '▁', 'a', '▁moving', '▁bird', '▁only', '▁', '0.5', 'm', '▁away', '.', '▁There', "'", 's', '▁another', '▁moving', '▁bird', '▁only', '▁1.5', 'm', '▁away', '.', '▁There', "'", 's', '▁another', '▁moving', '▁bird', '▁only', '▁2', 'm', '▁away', '.', '▁[', 'S', 'EP', ']', '▁(0', ':', '12', ')', '▁Risk', '▁of', '▁physical', '▁damage', '!', '▁There', "'", 's', '▁', 'a', '▁moving', '▁bird', '▁only', '▁', '0.5', 'm', '▁away', '.', '▁There', "'", 's', '▁another', '▁moving', '▁bird', '▁only', '▁1.5', 'm', '▁away', '.', '▁There', "'", 's', '▁another', '▁moving', '▁bird', '▁only', '▁2', 'm', '▁away', '.', '▁[', 'S', 'EP', ']', '▁(0', ':', '13', ')', '▁Risk', '▁of', '▁physical', '▁damage', '!', '▁There', "'", 's', '▁', 'a', '▁moving', '▁bird', '▁only', '▁', '0.5', 'm', '▁away', '.', '▁There', "'", 's', '▁another', '▁moving', '▁bird', '▁only', '▁1.5', 'm', '▁away', '.', '▁There', "'", 's', '▁another', '▁moving', '▁bird', '▁only', '▁2', 'm', '▁away', '.', '▁[', 'S', 'EP', ']', '▁(0', ':', '14', ')', '▁Risk', '▁of', '▁physical', '▁damage', '!', '▁There', "'", 's', '▁', 'a', '▁moving', '▁bird', '▁only', '▁', '0.5', 'm', '▁away', '.', '▁There', "'", 's', '▁another', '▁moving', '▁bird', '▁only', '▁1.5', 'm', '▁away', '.', '▁There', "'", 's', '▁another', '▁moving', '▁bird', '▁only', '▁2', 'm', '▁away', '.', '▁[', 'S', 'EP', ']', '▁(0', ':', '15', ')', '▁Risk', '▁of', '▁physical', '▁damage', '!', '▁There', "'", 's'], ['▁Risk', '▁of', '▁physical', '▁damage', '!', '▁The', '▁drone', '▁is', '▁flying', '▁5', 'm', '▁high', '▁with', '▁low', '▁battery', '.', '▁[', 'S', 'EP', ']', '▁(0', ':', '04', ')', '▁Risk', '▁of', '▁physical', '▁damage', '!', '▁A', '▁building', '▁in', '▁3', 'm', '▁is', '▁in', '▁the', '▁drone', "'", 's', '▁path', '.', '▁[', 'S', 'EP', ']', '▁(0', ':', '06', ')', '▁Risk', '▁of', '▁physical', '▁damage', '!', '▁Now', '▁there', '▁are', '▁stairs', '▁in', '▁3', 'm', '▁in', '▁the', '▁drone', "'", 's', '▁path', '.', '▁[', 'S', 'EP', ']', '▁(0', ':', '08', ')', '▁Risk', '▁of', '▁physical', '▁damage', '!', '▁The', '▁stairs', '▁are', '▁now', '.', '5', 'm', '▁from', '▁the', '▁drone', '.', '▁[', 'S', 'EP', ']', '▁(0', ':', '09', ')', '▁Risk', '▁of', '▁physical', '▁damage', '!', '▁The', '▁drone', '▁is', '▁flying', '▁2', 'm', '▁near', '▁to', '▁', 'a', '▁', 'pillar', '.', '▁[', 'S', 'EP', ']'], ['▁Risk', '▁of', '▁physical', '▁damage', '!', '▁The', '▁frame', '▁needs', '▁', 'repairing', '.', '▁[', 'S', 'EP', ']', '▁(', '0:00', ')', '▁Risk', '▁of', '▁physical', '▁damage', '!', '▁There', "'", 's', '▁', 'a', '▁sail', 'boat', '▁in', '▁the', '▁flight', '▁path', '▁in', '▁8', 'm', '.', '▁[', 'S', 'EP', ']', '▁(0', ':', '03', ')', '▁Risk', '▁of', '▁physical', '▁damage', '!', '▁The', '▁sail', 'boat', '▁in', '▁the', '▁flight', '▁path', '▁is', '▁now', '.', '5', 'm', '▁away', '.', '▁The', '▁drone', '▁is', '▁also', '▁flying', '▁near', '▁', 'a', '▁person', '▁1', 'm', '▁away', '.', '▁[', 'S', 'EP', ']'], ['▁Risk', '▁of', '▁physical', '▁damage', '!', '▁The', '▁drone', "'", 's', '▁needs', '▁to', '▁be', '▁repaired', '▁and', '▁its', '▁battery', '▁is', '▁empty', '.', '▁It', "'", 's', '▁flying', '▁at', '▁', 'a', '▁1', 'm', '▁al', 'titude', '.', '▁[', 'S', 'EP', ']', '▁(0', ':', '01', ')', '▁Risk', '▁of', '▁physical', '▁damage', '!', '▁The', '▁drone', '▁is', '▁flying', '▁1', 'm', '▁next', '▁to', '▁', 'a', '▁container', '▁ship', '.', '▁[', 'S', 'EP', ']'], ['▁Risk', '▁of', '▁physical', '▁and', '▁internal', '▁damage', '!', '▁The', '▁drone', '▁is', '▁flying', '▁with', '▁an', '▁empty', '▁battery', ',', '▁and', '▁at', '▁2', 'm', '▁al', 'titude', '.', '▁The', '▁drone', '▁is', 'n', "'", 't', '▁waterproof', ',', '▁and', '▁the', '▁weather', '▁is', '▁over', 'cast', '.', '▁[', 'S', 'EP', ']', '▁(0', ':', '01', ')', '▁Risk', '▁of', '▁physical', '▁damage', '!', '▁There', "'", 's', '▁', 'a', '▁box', '▁in', '▁5', 'm', '▁in', '▁the', '▁drone', "'", 's', '▁path', '.', '▁[', 'S', 'EP', ']', '▁(0', ':', '07', ')', '▁Risk', '▁of', '▁physical', '▁damage', '!', '▁The', '▁drone', '▁is', '▁flying', '▁near', '▁an', '▁iron', '▁frame', '.', '5', 'm', '▁away', '.', '▁[', 'S', 'EP', ']', '▁(0', ':', '08', ')', '▁Risk', '▁of', '▁physical', '▁damage', '!', '▁The', '▁drone', '▁is', '▁still', '▁flying', '.', '5', 'm', '▁away', '▁from', '▁the', '▁iron', '▁frame', '.', '▁[', 'S', 'EP', ']'], ['▁Risk', '▁of', '▁physical', '▁damage', '!', '▁The', '▁drone', '▁has', '▁low', '▁battery', '.', '▁It', '’', 's', '▁flying', '▁at', '▁an', '▁al', 'titude', '▁of', '▁80', 'm', '▁in', '▁', 'a', '▁low', '-', 'temperatur', 'e', '▁setting', '.', '▁[', 'S', 'EP', ']', '▁(', '0:00', ')', '▁Risk', '▁of', '▁physical', '▁damage', '!', '▁There', "'", 's', '▁', 'a', '▁moving', '▁bird', '▁only', '▁1', 'm', '▁away', '.', '▁There', "'", 's', '▁another', '▁moving', '▁bird', '▁only', '▁2', 'm', '▁away', '.', '▁[', 'S', 'EP', ']', '▁(0', ':', '01', ')', '▁Risk', '▁of', '▁physical', '▁damage', '!', '▁There', "'", 's', '▁', 'a', '▁moving', '▁bird', '▁only', '▁1', 'm', '▁away', '.', '▁There', "'", 's', '▁another', '▁moving', '▁bird', '▁only', '▁2', 'm', '▁away', '.', '▁[', 'S', 'EP', ']', '▁(0', ':', '02', ')', '▁Risk', '▁of', '▁physical', '▁damage', '!', '▁There', "'", 's', '▁', 'a', '▁moving', '▁bird', '▁only', '▁1', 'm', '▁away', '.', '▁There', "'", 's', '▁another', '▁moving', '▁bird', '▁only', '▁2', 'm', '▁away', '.', '▁[', 'S', 'EP', ']', '▁(0', ':', '03', ')', '▁Risk', '▁of', '▁physical', '▁damage', '!', '▁There', "'", 's', '▁', 'a', '▁moving', '▁bird', '▁only', '▁1', 'm', '▁away', '.', '▁There', "'", 's', '▁another', '▁moving', '▁bird', '▁only', '▁2', 'm', '▁away', '.', '▁[', 'S', 'EP', ']', '▁(0', ':', '04', ')', '▁Risk', '▁of', '▁physical', '▁damage', '!', '▁There', "'", 's', '▁', 'a', '▁moving', '▁bird', '▁only', '▁1', 'm', '▁away', '.', '▁There', "'", 's', '▁another', '▁moving', '▁bird', '▁only', '▁2', 'm', '▁away', '.', '▁[', 'S', 'EP', ']', '▁(0', ':', '05', ')', '▁Risk', '▁of', '▁physical', '▁damage', '!', '▁There', "'", 's', '▁', 'a', '▁moving', '▁bird', '▁only', '▁1', 'm', '▁away', '.', '▁There', "'", 's', '▁another', '▁moving', '▁bird', '▁only', '▁2', 'm', '▁away', '.', '▁[', 'S', 'EP', ']', '▁(0', ':', '06', ')', '▁Risk', '▁of', '▁physical', '▁damage', '!', '▁There', "'", 's', '▁', 'a', '▁moving', '▁bird', '▁only', '▁1', 'm', '▁away', '.', '▁There', "'", 's', '▁another', '▁moving', '▁bird', '▁only', '▁2', 'm', '▁away', '.', '▁[', 'S', 'EP', ']', '▁(0', ':', '07', ')', '▁Risk', '▁of', '▁physical', '▁damage', '!', '▁There', "'", 's', '▁', 'a', '▁moving', '▁bird', '▁only', '▁1', 'm', '▁away', '.', '▁There', "'", 's', '▁another', '▁moving', '▁bird', '▁only', '▁2', 'm', '▁away', '.', '▁[', 'S', 'EP', ']', '▁(0', ':', '08', ')', '▁Risk', '▁of', '▁physical', '▁damage', '!', '▁There', "'", 's', '▁', 'a', '▁moving', '▁bird', '▁only', '▁1', 'm', '▁away', '.', '▁There', "'", 's', '▁another', '▁moving', '▁bird', '▁only', '▁2', 'm', '▁away', '.', '▁[', 'S', 'EP', ']', '▁(0', ':', '09', ')', '▁Risk', '▁of', '▁physical', '▁damage', '!', '▁There', "'", 's', '▁', 'a', '▁moving', '▁bird', '▁only', '▁1', 'm', '▁away', '.', '▁There', "'", 's', '▁another', '▁moving', '▁bird', '▁only', '▁2', 'm', '▁away', '.', '▁[', 'S', 'EP', ']', '▁(0', ':', '10)', '▁Risk', '▁of', '▁physical', '▁damage', '!', '▁There', "'", 's', '▁', 'a', '▁moving', '▁bird', '▁only', '▁1', 'm', '▁away', '.', '▁There', "'", 's', '▁another', '▁moving', '▁bird', '▁only', '▁2', 'm', '▁away', '.', '▁[', 'S', 'EP', ']', '▁(0', ':', '11', ')', '▁Risk', '▁of', '▁physical', '▁damage', '!', '▁There', "'", 's', '▁', 'a', '▁moving', '▁bird', '▁only', '▁1', 'm', '▁away', '.', '▁There', "'", 's', '▁another', '▁moving', '▁bird', '▁only', '▁2', 'm', '▁away', '.', '▁[', 'S', 'EP', ']', '▁(0', ':', '12', ')', '▁Risk', '▁of', '▁physical', '▁damage', '!', '▁There', "'", 's', '▁', 'a', '▁moving', '▁bird', '▁only', '▁1', 'm', '▁away', '.', '▁There', "'", 's', '▁another', '▁moving', '▁bird', '▁only', '▁2', 'm', '▁away', '.', '▁[', 'S', 'EP', ']', '▁(0', ':', '13', ')', '▁Risk', '▁of', '▁physical', '▁damage', '!', '▁There', "'", 's', '▁', 'a', '▁moving', '▁bird', '▁only', '▁1', 'm', '▁away', '.', '▁There', "'", 's', '▁another', '▁moving', '▁bird', '▁only', '▁2', 'm', '▁away', '.', '▁[', 'S', 'EP', ']', '▁(0', ':', '14', ')', '▁Risk', '▁of', '▁physical', '▁damage', '!', '▁There', "'", 's', '▁', 'a', '▁moving', '▁bird', '▁only', '▁1', 'm', '▁away', '.', '▁There', "'", 's', '▁another', '▁moving', '▁bird', '▁only', '▁2', 'm', '▁away', '.', '▁[', 'S', 'EP', ']'], ['▁Risk', '▁of', '▁physical', '▁damage', '!', '▁The', '▁drone', '▁has', '▁', 'a', '▁damaged', '▁frame', '▁and', '▁low', '▁battery', '.', '▁It', '’', 's', '▁flying', '▁at', '▁an', '▁al', 'titude', '▁of', '▁1', 'm', '.', '▁[', 'S', 'EP', ']', '▁(0', ':', '01', ')', '▁Risk', '▁of', '▁physical', '▁damage', '!', '▁There', "'", 's', '▁', 'a', '▁tree', '▁in', '▁the', '▁drone', "'", 's', '▁flight', '▁path', '▁at', '▁', 'a', '▁distance', '▁of', '▁5', 'm', '.', '▁[', 'S', 'EP', ']', '▁(0', ':', '03', ')', '▁Risk', '▁of', '▁physical', '▁damage', '!', '▁Now', '▁the', '▁tree', '▁is', '▁in', '▁the', '▁drone', "'", 's', '▁flight', '▁path', '▁at', '▁', 'a', '▁distance', '▁of', '▁1', 'm', '.', '▁[', 'S', 'EP', ']'], ['▁Risk', '▁of', '▁physical', '▁damage', '!', '▁The', '▁drone', '▁has', '▁low', '▁battery', '.', '▁It', '’', 's', '▁flying', '▁at', '▁an', '▁al', 'titude', '▁of', '▁1', 'm', '.', '▁[', 'S', 'EP', ']', '▁(0', ':', '03', ')', '▁Risk', '▁of', '▁physical', '▁damage', '!', '▁There', "'", 's', '▁', 'a', '▁tree', '▁in', '▁the', '▁drone', "'", 's', '▁flight', '▁path', '▁at', '▁', 'a', '▁distance', '▁of', '▁3', 'm', '.', '▁[', 'S', 'EP', ']', '▁(0', ':', '04', ')', '▁Risk', '▁of', '▁physical', '▁damage', '!', '▁There', "'", 's', '▁another', '▁tree', '▁only', '▁1', 'm', '▁away', '.', '▁[', 'S', 'EP', ']'], ['▁Risk', '▁of', '▁physical', '▁damage', '!', '▁The', '▁drone', "'", 's', '▁frame', '▁is', '▁in', '▁need', '▁of', '▁repair', '.', '▁[', 'S', 'EP', ']', '▁(', '0:00', ')', '▁Risk', '▁of', '▁physical', '▁damage', '!', '▁There', "'", 's', '▁', 'a', '▁tree', '▁8', 'm', '▁ahead', '▁in', '▁the', '▁flight', '▁path', '.', '▁[', 'S', 'EP', ']', '▁(0', ':', '03', ')', '▁Risk', '▁of', '▁physical', '▁damage', '!', '▁There', "'", 's', '▁another', '▁tree', '▁2', 'm', '▁from', '▁where', '▁the', '▁drone', '▁is', '▁flying', '.', '▁[', 'S', 'EP', ']'], ['▁Risk', '▁of', '▁physical', '▁damage', '!', '▁The', '▁battery', '▁is', '▁empty', '▁and', '▁the', '▁frame', '▁is', '▁damaged', '.', '▁The', '▁drone', '▁is', '▁flying', '▁at', '▁', 'a', '▁5', 'm', '▁al', 'titude', '▁in', '▁low', '▁temperature', '.', '▁[', 'S', 'EP', ']', '▁(0', ':', '01', ')', '▁Risk', '▁of', '▁physical', '▁damage', '!', '▁There', "'", 's', '▁', 'a', '▁fallen', '▁tree', '▁5', 'm', '▁ahead', '▁in', '▁the', '▁drone', "'", 's', '▁path', '.', '▁[', 'S', 'EP', ']', '▁(0', ':', '02', ')', '▁Risk', '▁of', '▁physical', '▁damage', '!', '▁Now', '▁the', '▁drone', '▁is', '▁flying', '▁1', 'm', '▁from', '▁that', '▁fallen', '▁tree', '.', '▁[', 'S', 'EP', ']', '▁(0', ':', '05', ')', '▁Risk', '▁of', '▁physical', '▁damage', '!', '▁There', "'", 's', '▁', 'a', '▁tree', '▁in', '▁the', '▁drone', "'", 's', '▁path', '▁in', '▁8', 'm', '.', '▁[', 'S', 'EP', ']', '▁(0', ':', '06', ')', '▁Risk', '▁of', '▁physical', '▁damage', '!', '▁Now', '▁the', '▁drone', '▁is', '▁flying', '▁2', 'm', '▁from', '▁that', '▁tree', '▁that', '▁was', '▁in', '▁its', '▁path', '.', '▁[', 'S', 'EP', ']', '▁(0', ':', '08', ')', '▁Risk', '▁of', '▁physical', '▁damage', '!', '▁There', "'", 's', '▁another', '▁tree', '▁in', '▁5', 'm', '▁that', "'", 's', '▁in', '▁the', '▁drone', "'", 's', '▁path', '.', '▁[', 'S', 'EP', ']'], ['▁Risk', '▁of', '▁physical', '▁damage', '!', '▁The', '▁drone', '▁has', '▁', 'a', '▁damaged', '▁frame', '▁and', '▁low', '▁battery', '.', '▁It', '’', 's', '▁flying', '▁at', '▁an', '▁al', 'titude', '▁of', '▁20', 'm', '.', '▁[', 'S', 'EP', ']', '▁(', '0:00', ')', '▁Risk', '▁of', '▁physical', '▁damage', '!', '▁There', '▁are', '▁leaves', '▁in', '▁the', '▁drone', "'", 's', '▁flight', '▁path', '▁at', '▁', 'a', '▁distance', '▁of', '.', '5', 'm', '.', '▁[', 'S', 'EP', ']', '▁(0', ':', '01', ')', '▁Risk', '▁of', '▁physical', '▁damage', '!', '▁Now', '▁the', '▁leaves', '▁are', '▁in', '▁the', '▁drone', "'", 's', '▁flight', '▁path', '▁at', '▁', 'a', '▁distance', '▁of', '.', '2', 'm', '.', '▁[', 'S', 'EP', ']', '▁(0', ':', '03', ')', '▁Risk', '▁of', '▁physical', '▁damage', '!', '▁Now', '▁the', '▁leaves', '▁are', '▁in', '▁the', '▁drone', "'", 's', '▁flight', '▁path', '▁at', '▁', 'a', '▁distance', '▁of', '.', '2', 'm', '.', '▁[', 'S', 'EP', ']', '▁(0', ':', '07', ')', '▁Risk', '▁of', '▁physical', '▁damage', '!', '▁There', "'", 's', '▁', 'a', '▁tree', '▁trunk', '▁in', '▁the', '▁drone', "'", 's', '▁flight', '▁path', '▁at', '▁', 'a', '▁distance', '▁of', '▁1.5', 'm', '.', '▁[', 'S', 'EP', ']', '▁(0', ':', '09', ')', '▁Risk', '▁of', '▁physical', '▁damage', '!', '▁Now', '▁the', '▁tree', '▁trunk', '▁is', '▁in', '▁the', '▁drone', "'", 's', '▁flight', '▁path', '▁at', '▁', 'a', '▁distance', '▁of', '.', '5', 'm', '.', '▁[', 'S', 'EP', ']'], ['▁Risk', '▁of', '▁physical', '▁damage', '!', '▁The', '▁drone', '▁has', '▁', 'a', '▁damaged', '▁frame', '▁and', '▁low', '▁battery', '.', '▁It', '’', 's', '▁flying', '▁at', '▁an', '▁al', 'titude', '▁of', '▁80', 'm', '.', '▁[', 'S', 'EP', ']', '▁(0', ':', '01', ')', '▁Risk', '▁of', '▁physical', '▁damage', '!', '▁There', "'", 's', '▁', 'a', '▁sky', 's', 'cra', 'per', '▁only', '.', '5', 'm', '▁away', '.', '▁[', 'S', 'EP', ']'], ['▁Risk', '▁of', '▁physical', '▁damage', '!', '▁The', '▁drone', '▁has', '▁', 'a', '▁damaged', '▁frame', '.', '▁[', 'S', 'EP', ']', '▁(', '0:00', ')', '▁Risk', '▁of', '▁physical', '▁damage', '!', '▁The', '▁drone', '▁is', '▁flying', '.', '5', 'm', '▁from', '▁', 'a', '▁roller', '▁coast', 'er', '.', '▁[', 'S', 'EP', ']', '▁(0', ':', '02', ')', '▁Risk', '▁of', '▁physical', '▁damage', '!', '▁Now', '▁there', "'", 's', '▁', 'a', '▁booth', '.', '5', 'm', '▁away', '▁from', '▁the', '▁drone', '.', '▁[', 'S', 'EP', ']', '▁(0', ':', '05', ')', '▁Risk', '▁of', '▁physical', '▁damage', '!', '▁The', '▁roller', '▁coast', 'er', '▁is', '.', '5', 'm', '▁away', '▁again', '.', '▁[', 'S', 'EP', ']'], ['▁Risk', '▁of', '▁physical', '▁damage', '!', '▁The', '▁frame', '▁is', '▁damaged', '.', '▁[', 'S', 'EP', ']', '▁(0', ':', '01', ')', '▁Risk', '▁of', '▁physical', '▁damage', '!', '▁The', '▁drone', '▁is', '▁approaching', '▁', 'a', '▁tree', '▁8', 'm', '▁away', '▁in', '▁its', '▁path', '.', '▁[', 'S', 'EP', ']', '▁(0', ':', '03', ')', '▁Risk', '▁of', '▁physical', '▁damage', '!', '▁Now', '▁the', '▁tree', '▁is', '▁2', 'm', '▁away', '▁and', '▁still', '▁in', '▁the', '▁path', '.', '▁[', 'S', 'EP', ']', '▁(0', ':', '04', ')', '▁Risk', '▁of', '▁physical', '▁damage', '!', '▁The', '▁drone', '▁is', '▁flying', '▁2', 'm', '▁from', '▁another', '▁tree', '.', '▁[', 'S', 'EP', ']', '▁(0', ':', '07', ')', '▁Risk', '▁of', '▁physical', '▁damage', '!', '▁Now', '▁the', '▁tree', '▁is', '▁2', 'm', '▁away', '▁and', '▁in', '▁the', '▁drone', "'", 's', '▁path', '.', '▁[', 'S', 'EP', ']'], ['▁Risk', '▁of', '▁physical', '▁damage', '!', '▁The', '▁drone', "'", 's', '▁battery', '▁is', '▁empty', '▁and', '▁it', "'", 's', '▁flying', '▁30', 'm', '▁high', '.', '▁[', 'S', 'EP', ']', '▁(', '0:00', ')', '▁Risk', '▁of', '▁physical', '▁damage', '!', '▁The', '▁drone', '▁is', '▁flying', '▁next', '▁to', '▁', 'a', '▁building', '▁which', '▁is', '▁only', '▁', '0.5', 'm', '▁away', '.', '▁[', 'S', 'EP', ']', '▁(0', ':', '04', ')', '▁Risk', '▁of', '▁physical', '▁damage', '!', '▁Now', '▁there', "'", 's', '▁', 'a', '▁boat', '▁in', '▁5', 'm', '▁in', '▁the', '▁flight', '▁path', '.', '▁[', 'S', 'EP', ']'], ['▁Risk', '▁of', '▁internal', '▁damage', '!', '▁The', '▁weather', '▁is', '▁over', 'cast', '▁and', '▁the', '▁drone', '▁is', '▁not', '▁waterproof', '.', '▁[', 'S', 'EP', ']', '▁(0', ':', '09', ')', '▁Risk', '▁of', '▁physical', '▁damage', '!', '▁The', '▁ground', '▁is', '▁in', '▁the', '▁drone', "'", 's', '▁flight', '▁path', '▁at', '▁', 'a', '▁distance', '▁of', '▁', '9.5', 'm', '.', '▁[', 'S', 'EP', ']', '▁(0', ':', '10)', '▁Risk', '▁of', '▁physical', '▁damage', '!', '▁Now', '▁the', '▁ground', '▁is', '▁in', '▁the', '▁drone', "'", 's', '▁flight', '▁path', '▁at', '▁', 'a', '▁distance', '▁of', '▁9', 'm', '.', '▁[', 'S', 'EP', ']', '▁(0', ':', '11', ')', '▁Risk', '▁of', '▁physical', '▁damage', '!', '▁Now', '▁the', '▁ground', '▁is', '▁in', '▁the', '▁drone', "'", 's', '▁flight', '▁path', '▁at', '▁', 'a', '▁distance', '▁of', '▁', '8.5', 'm', '.', '▁[', 'S', 'EP', ']', '▁(0', ':', '12', ')', '▁Risk', '▁of', '▁physical', '▁damage', '!', '▁Now', '▁the', '▁ground', '▁is', '▁in', '▁the', '▁drone', "'", 's', '▁flight', '▁path', '▁at', '▁', 'a', '▁distance', '▁of', '▁8', 'm', '.', '▁[', 'S', 'EP', ']', '▁(0', ':', '13', ')', '▁Risk', '▁of', '▁physical', '▁damage', '!', '▁Now', '▁the', '▁ground', '▁is', '▁in', '▁the', '▁drone', "'", 's', '▁flight', '▁path', '▁at', '▁', 'a', '▁distance', '▁of', '▁', '7.5', 'm', '.', '▁[', 'S', 'EP', ']', '▁(0', ':', '14', ')', '▁Risk', '▁of', '▁physical', '▁damage', '!', '▁Now', '▁the', '▁ground', '▁is', '▁in', '▁the', '▁drone', "'", 's', '▁flight', '▁path', '▁at', '▁', 'a', '▁distance', '▁of', '▁7', 'm', '.', '▁[', 'S', 'EP', ']'], ['▁Risk', '▁of', '▁physical', '▁damage', '!', '▁The', '▁frame', '▁is', '▁damaged', '▁and', '▁the', '▁battery', '▁is', '▁empty', '.', '▁The', '▁drone', '▁is', '▁also', '▁flying', '▁at', '▁an', '▁al', 'titude', '▁of', '▁8', 'm', '▁while', '▁indoor', 's', '.', '▁[', 'S', 'EP', ']', '▁(', '0:00', ')', '▁Risk', '▁of', '▁physical', '▁damage', '!', '▁There', "'", 's', '▁', 'a', '▁rail', 'ing', '▁in', '▁the', '▁drone', "'", 's', '▁path', '▁in', '▁2', 'm', '.', '▁[', 'S', 'EP', ']', '▁(0', ':', '04', ')', '▁Risk', '▁of', '▁physical', '▁damage', '!', '▁Now', '▁there', "'", 's', '▁', 'a', '▁wall', '▁in', '▁5', 'm', '▁in', '▁the', '▁flight', '▁path', '.', '▁[', 'S', 'EP', ']', '▁(0', ':', '05', ')', '▁Risk', '▁of', '▁physical', '▁damage', '!', '▁The', '▁drone', '▁is', '▁flying', '▁1', 'm', '▁from', '▁the', '▁wall', '▁now', '.', '▁[', 'S', 'EP', ']', '▁(0', ':', '06', ')', '▁Risk', '▁of', '▁physical', '▁damage', '!', '▁There', '▁are', '▁stairs', '▁1.5', 'm', '▁from', '▁where', '▁the', '▁drone', '▁is', '▁flying', '.', '▁[', 'S', 'EP', ']', '▁(0', ':', '07', ')', '▁Risk', '▁of', '▁physical', '▁damage', '!', '▁The', '▁drone', '▁is', '▁flying', '▁next', '▁to', '▁', 'a', '▁wall', '▁only', '▁', '0.5', 'm', '▁away', '.', '▁[', 'S', 'EP', ']', '▁(0', ':', '18', ')', '▁Risk', '▁of', '▁physical', '▁damage', '!', '▁There', "'", 's', '▁another', '▁rail', 'ing', '▁in', '▁the', '▁flight', '▁path', '▁3', 'm', '▁ahead', '.', '▁[', 'S', 'EP', ']'], ['▁Risk', '▁of', '▁physical', '▁damage', '!', '▁The', '▁frame', '▁on', '▁the', '▁drone', '▁is', '▁in', '▁need', '▁of', '▁repair', '.', '▁[', 'S', 'EP', ']', '▁(0', ':', '03', ')', '▁Risk', '▁of', '▁physical', '▁damage', '!', '▁There', "'", 's', '▁', 'a', '▁hallway', '▁up', '▁ahead', '▁in', '▁the', '▁flight', '▁path', '▁at', '▁an', '▁8', 'm', '▁distance', '.', '▁[', 'S', 'EP', ']', '▁(0', ':', '04', ')', '▁Risk', '▁of', '▁physical', '▁damage', '!', '▁Now', '▁the', '▁hallway', '▁in', '▁the', '▁drone', "'", 's', '▁path', '▁is', '▁2', 'm', '▁away', '.', '▁[', 'S', 'EP', ']'], ['▁Risk', '▁of', '▁physical', '▁and', '▁internal', '▁damage', '!', '▁The', '▁drone', '▁is', '▁flying', '▁back', 'ward', 's', '.', '▁The', '▁weather', '▁is', '▁over', 'cast', '▁and', '▁the', '▁drone', '▁is', '▁not', '▁waterproof', '.', '▁[', 'S', 'EP', ']', '▁(', '0:00', ')', '▁Risk', '▁of', '▁physical', '▁damage', '!', '▁The', '▁ground', '▁is', '▁in', '▁the', '▁drone', "'", 's', '▁flight', '▁path', '▁at', '▁', 'a', '▁distance', '▁of', '▁4', 'm', '.', '▁[', 'S', 'EP', ']', '▁(0', ':', '01', ')', '▁Risk', '▁of', '▁physical', '▁damage', '!', '▁Now', '▁the', '▁ground', '▁is', '▁in', '▁the', '▁drone', "'", 's', '▁flight', '▁path', '▁at', '▁', 'a', '▁distance', '▁of', '▁3.', '75', 'm', '.', '▁[', 'S', 'EP', ']', '▁(0', ':', '02', ')', '▁Risk', '▁of', '▁physical', '▁damage', '!', '▁Now', '▁the', '▁ground', '▁is', '▁in', '▁the', '▁drone', "'", 's', '▁flight', '▁path', '▁at', '▁', 'a', '▁distance', '▁of', '▁', '3.5', 'm', '.', '▁[', 'S', 'EP', ']', '▁(0', ':', '03', ')', '▁Risk', '▁of', '▁physical', '▁damage', '!', '▁Now', '▁the', '▁ground', '▁is', '▁in', '▁the', '▁drone', "'", 's', '▁flight', '▁path', '▁at', '▁', 'a', '▁distance', '▁of', '▁3.', '25', 'm', '.', '▁[', 'S', 'EP', ']', '▁(0', ':', '04', ')', '▁Risk', '▁of', '▁physical', '▁damage', '!', '▁Now', '▁the', '▁ground', '▁is', '▁in', '▁the', '▁drone', "'", 's', '▁flight', '▁path', '▁at', '▁', 'a', '▁distance', '▁of', '▁3', 'm', '.', '▁[', 'S', 'EP', ']', '▁(0', ':', '05', ')', '▁Risk', '▁of', '▁physical', '▁damage', '!', '▁Now', '▁the', '▁ground', '▁is', '▁in', '▁the', '▁drone', "'", 's', '▁flight', '▁path', '▁at', '▁', 'a', '▁distance', '▁of', '▁2.', '75', 'm', '.', '▁[', 'S', 'EP', ']', '▁(0', ':', '06', ')', '▁Risk', '▁of', '▁physical', '▁damage', '!', '▁Now', '▁the', '▁ground', '▁is', '▁in', '▁the', '▁drone', "'", 's', '▁flight', '▁path', '▁at', '▁', 'a', '▁distance', '▁of', '▁2.5', 'm', '.', '▁[', 'S', 'EP', ']', '▁(0', ':', '07', ')', '▁Risk', '▁of', '▁physical', '▁damage', '!', '▁Now', '▁the', '▁ground', '▁is', '▁in', '▁the', '▁drone', "'", 's', '▁flight', '▁path', '▁at', '▁', 'a', '▁distance', '▁of', '▁2.', '25', 'm', '.', '▁[', 'S', 'EP', ']', '▁(0', ':', '08', ')', '▁Risk', '▁of', '▁physical', '▁damage', '!', '▁Now', '▁the', '▁ground', '▁is', '▁in', '▁the', '▁drone', "'", 's', '▁flight', '▁path', '▁at', '▁', 'a', '▁distance', '▁of', '▁2', 'm', '.', '▁[', 'S', 'EP', ']', '▁(0', ':', '09', ')', '▁Risk', '▁of', '▁physical', '▁damage', '!', '▁Now', '▁the', '▁ground', '▁is', '▁in', '▁the', '▁drone', "'", 's', '▁flight', '▁path', '▁at', '▁', 'a', '▁distance', '▁of', '▁1.', '75', 'm', '.', '▁[', 'S', 'EP', ']', '▁(0', ':', '10)', '▁Risk', '▁of', '▁physical', '▁damage', '!', '▁Now', '▁the', '▁ground', '▁is', '▁in', '▁the', '▁drone', "'", 's', '▁flight', '▁path', '▁at', '▁', 'a', '▁distance', '▁of', '▁1.5', 'm', '.', '▁[', 'S', 'EP', ']', '▁(0', ':', '11', ')', '▁Risk', '▁of', '▁physical', '▁damage', '!', '▁Now', '▁the', '▁ground', '▁is', '▁in', '▁the', '▁drone', "'", 's', '▁flight', '▁path', '▁at', '▁', 'a', '▁distance', '▁of', '▁1.', '25', 'm', '.', '▁[', 'S', 'EP', ']', '▁(0', ':', '12', ')', '▁Risk', '▁of', '▁physical', '▁damage', '!', '▁Now', '▁the', '▁ground', '▁is', '▁in', '▁the', '▁drone', "'", 's', '▁flight', '▁path', '▁at', '▁', 'a', '▁distance', '▁of', '▁1', 'm', '.', '▁[', 'S', 'EP', ']', '▁(0', ':', '13', ')', '▁Risk', '▁of', '▁physical', '▁damage', '!', '▁Now', '▁the', '▁ground', '▁is', '▁in', '▁the', '▁drone', "'", 's', '▁flight', '▁path', '▁at', '▁', 'a', '▁distance', '▁of', '▁0.', '75', 'm', '.', '▁[', 'S', 'EP', ']'], ['▁Risk', '▁of', '▁physical', '▁damage', '▁and', '▁internal', '▁damage', '!', '▁The', '▁drone', "'", 's', '▁battery', '▁is', '▁low', '▁and', '▁it', '’', 's', '▁flying', '▁at', '▁an', '▁al', 'titude', '▁of', '▁4', 'm', '.', '▁The', '▁weather', '▁is', '▁very', '▁cold', '▁and', '▁snow', 'y', ',', '▁and', '▁the', '▁drone', '▁is', 'n', "'", 't', '▁waterproof', '.', '▁[', 'S', 'EP', ']', '▁(0', ':', '10)', '▁Risk', '▁of', '▁physical', '▁damage', '!', '▁There', "'", 's', '▁', 'a', '▁tree', '▁in', '▁8', 'm', '▁in', '▁the', '▁drone', "'", 's', '▁flight', '▁path', '.', '▁[', 'S', 'EP', ']', '▁(0', ':', '11', ')', '▁Risk', '▁of', '▁physical', '▁damage', '!', '▁Now', '▁the', '▁tree', '▁in', '▁the', '▁drone', "'", 's', '▁path', '▁is', '▁only', '▁2', 'm', '▁away', ',', '▁and', '▁another', '▁tree', '▁in', '▁5', 'm', '▁is', '▁also', '▁in', '▁the', '▁flight', '▁path', '.', '▁[', 'S', 'EP', ']', '▁(0', ':', '15', ')', '▁Risk', '▁of', '▁physical', '▁damage', '!', '▁The', '▁drone', '▁is', '▁flying', '▁3', 'm', '▁from', '▁', 'a', '▁forest', '.', '▁[', 'S', 'EP', ']', '▁(0', ':', '16', ')', '▁Risk', '▁of', '▁physical', '▁damage', '!', '▁Now', '▁the', '▁drone', '▁is', '▁flying', '▁near', '▁', 'a', '▁forest', '▁and', '▁', 'a', '▁tree', ',', '▁each', '▁1', 'm', '▁away', '.', '▁[', 'S', 'EP', ']', '▁(0', ':', '17', ')', '▁Risk', '▁of', '▁physical', '▁damage', '!', '▁Now', '▁the', '▁forest', '▁is', '▁only', '▁1.5', 'm', '▁from', '▁the', '▁drone', '.', '▁[', 'S', 'EP', ']', '▁(0', ':', '18', ')', '▁Risk', '▁of', '▁physical', '▁damage', '!', '▁The', '▁drone', '▁is', '▁flying', '▁2', 'm', '▁from', '▁the', '▁forest', '▁now', '.', '▁[', 'S', 'EP', ']'], ['▁Risk', '▁of', '▁physical', '▁damage', '!', '▁The', '▁drone', '▁has', '▁', 'a', '▁damaged', '▁frame', '.', '▁[', 'S', 'EP', ']', '▁(', '0:00', ')', '▁Risk', '▁of', '▁physical', '▁damage', '!', '▁There', "'", 's', '▁', 'a', '▁tree', '▁in', '▁the', '▁drone', "'", 's', '▁flight', '▁path', '▁at', '▁', 'a', '▁distance', '▁of', '▁8', 'm', '.', '▁[', 'S', 'EP', ']', '▁(0', ':', '03', ')', '▁Risk', '▁of', '▁physical', '▁damage', '!', '▁There', "'", 's', '▁another', '▁tree', '▁only', '▁2', 'm', '▁away', '.', '▁[', 'S', 'EP', ']'], ['▁Risk', '▁of', '▁physical', '▁damage', '!', '▁The', '▁drone', "'", 's', '▁frame', '▁is', '▁damaged', '▁and', '▁the', '▁battery', '▁is', '▁running', '▁low', ',', '▁while', '▁flying', '▁at', '▁', 'a', '▁5', 'm', '▁al', 'titude', '.', '[', 'S', 'EP', ']', '▁(', '0:00', ')', '▁Risk', '▁of', '▁physical', '▁damage', '!', '▁The', '▁castle', '▁wall', '▁is', '▁only', '.', '5', 'm', '▁from', '▁the', '▁drone', '.', '▁[', 'S', 'EP', ']', '▁(0', ':', '01', ')', '▁Risk', '▁of', '▁physical', '▁damage', '!', '▁There', "'", 's', '▁another', '▁castle', '▁wall', '▁which', '▁is', '▁in', '▁the', '▁drone', "'", 's', '▁path', '▁and', '▁only', '▁1.5', 'm', '▁away', '.', '▁[', 'S', 'EP', ']', '▁(0', ':', '03', ')', '▁Risk', '▁of', '▁physical', '▁damage', '!', '▁Now', '▁that', '▁wall', '▁is', '▁only', '▁1', 'm', '▁away', '.', '▁[', 'S', 'EP', ']'], ['▁Risk', '▁of', '▁physical', '▁damage', '!', '▁The', '▁frame', '▁on', '▁the', '▁drone', '▁is', '▁damaged', '.', '▁[', 'S', 'EP', ']', '▁(', '0:00', ')', '▁Risk', '▁of', '▁physical', '▁damage', '!', '▁A', '▁tree', '▁up', '▁ahead', '▁is', '▁in', '▁the', '▁drone', "'", 's', '▁path', '▁and', '▁5', 'm', '▁away', '.', '▁[', 'S', 'EP', ']', '▁(0', ':', '04', ')', '▁Risk', '▁of', '▁physical', '▁damage', '!', '▁The', '▁drone', '▁is', '▁flying', '▁1.5', 'm', '▁from', '▁another', '▁tree', '.', '▁[', 'S', 'EP', ']', '▁(0', ':', '06', ')', '▁Risk', '▁of', '▁physical', '▁damage', '!', '▁Now', '▁the', '▁drone', '▁is', '▁flying', '▁at', '▁', 'a', '▁close', '▁distance', '▁of', '.', '5', 'm', '▁from', '▁another', '▁tree', '.', '▁[', 'S', 'EP', ']']]]
len(refer[0][0])

57